# LONESTAR PURCHASE AND SCRAP REPORT

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from shutil import copyfile
from splinter import Browser
from bs4 import BeautifulSoup as soup
from datetime import datetime
from shutil import copyfile
import os
import shutil
import time
import re
import plotly.graph_objects as go



# PLOTLY INTERACTIVE CREDS:
import chart_studio

# Get environment variables
username = os.getenv('plotly_chart_studio_username')
api_key = os.environ.get('plotly_chart_studio_api_key')

chart_studio.tools.set_credentials_file(username=username, api_key=api_key)
import chart_studio.plotly as py
import chart_studio.tools as tls

In [2]:
Start_time = time.time()

In [3]:
# Get Current and Last Year as well as Current and Last Month as Integer:
now = datetime.now()
print("now:", now)
print("--------")

# Since we need to only report completed months, we will find the last completed month
Current_Month = int(now.strftime("%Y%m"))
Last_Month = Current_Month - 1
print(f"Last_Month: {Last_Month}")
print('----------')

# Now we'll get the month name so we can use it as a variable for graph titles and more:
Last_Month_One_or_Two_Digit = now.month-1
Last_Month_Descriptive = "January February March April May June July August September October November December".split()[Last_Month_One_or_Two_Digit-1]
print(f'Last_Month_Descriptive: {Last_Month_Descriptive}')
print('----------')

# We'll use this version for the first banner title of the website/report:
Last_Month_Descriptive_Short = "JAN FEB MAR APR MAY JUN JUL AUG SEPT OCT NOV DEC".split()[Last_Month_One_or_Two_Digit-1]

Current_Year = int(now.strftime("%Y"))
print(f"Current_Year: {Current_Year}")
print('----------')

Last_Year = Current_Year -1
print(f'Last_Year: {Last_Year}')
print('----------')

# Here we will create the numerical months in YYYYMM format for column name references later:


# Finally, create a variable to list all the month abbreviations as strings:
Months_Strings = ["JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "AUG", "SEPT", "OCT", "NOV", "DEC"]
print('-----------')

now: 2021-11-04 16:27:35.203144
--------
Last_Month: 202110
----------
Last_Month_Descriptive: October
----------
Current_Year: 2021
----------
Last_Year: 2020
----------
-----------


In [4]:
# Connect to DB2 Databse:
import pyodbc
cnxn = pyodbc.connect("DSN=MaddenCo;UID=U212READO;PWD=stmffe")
cursor = cnxn.cursor()

In [5]:
# Depending on the chosen Year in the format YYYY, we will create variables for each month accordingly.
# For example, if we had selected Current_Year = 2021, Jan will become 202101.  This is useful because the
# Maddenco database has the dates formated in this manner YYYYMM.

Jan = Current_Year * 100 + 1
Feb = Current_Year * 100 + 2
Mar = Current_Year * 100 + 3
Apr = Current_Year * 100 + 4
May = Current_Year * 100 + 5
Jun = Current_Year * 100 + 6
Jul = Current_Year * 100 + 7
Aug = Current_Year * 100 + 8
Sept = Current_Year * 100 + 9
Oct = Current_Year * 100 + 10
Nov = Current_Year * 100 + 11
Dec = Current_Year * 100 + 12

# We can also define a list of these numerical Months:
# Again, these will be in format like Jan = 202101.
Months = [Jan, Feb, Mar, Apr, May, Jun, Jul, Aug, Sept, Oct, Nov, Dec]
print("Months:")
for x in Months:
    print(f'    ', x)
print("---------")

Months:
     202101
     202102
     202103
     202104
     202105
     202106
     202107
     202108
     202109
     202110
     202111
     202112
---------


# Prepare a single column Sales Table df (we'll add to it later):

In [6]:
# Create Dataframe of just categories.  Then we'll loop through and add each Location column to the dataframe:
Data = {
    
'Category': ['SERVICE',
                    'Flat Repair',
                    'Tire Rotation',
                    'Mount and Dismount',
                    'Service Call Hours', 
                    'Scrap',
                    'BANDAG RETREADS',
                    '245/70R17.5 RETREAD',
                    '295/75R22.5 RETREAD',
                     'RETREAD + CASINGS', 
                     '10.00-20 CAP & CASING',
                     '11R22.5 CAP & CASING', 
                     '295/75R22.5 CAP & CASING', 
                     'REPAIR ONLY', 
                     'NEW TIRES',
                     'BRIDGESTONE',
                     'Light Truck BRIDGESTONE',
                     '225/70R19.5 BRIDGESTONE',
                     '245/70R17.5 BRIDGESTONE',
                     '9R22.5 BRIDGESTONE',
                     '295/75R22.5 BRIDGESTONE', 
                     '11R22.5 BRIDGESTONE',
                     'OTHER BRIDGESTONE',
                     '10.00-20',
                     'YOKOHAMA TIRES',
                     'CONTINENTAL TIRES',
                     'HANKOOK TIRES',
                     'MISC NEW TIRES',
                     'BS STEER-NEW TIRE',
                     'BS DRIVE-NEW TIRE', 
                     'BS TRAILER-NEW TIRE']
}

# We'll create a starting df for each State (and the combination of all 3) based on that first column:
Sales_Table_df = pd.DataFrame(Data)

# Loop through last year and current year for Swift Transportation data and add each one to a separate dataframe:

In [7]:
years = [Last_Year, Current_Year]


for year in years:
    print('-------------------------------------------')
    print(f'year: {year}')
    Months_Numerical = []
    for x in range(1,13):
        Months_Numerical.append(year*100 + x)
    print("Months_Numerical: ", Months_Numerical)

    # ================================================================================================================================
    # +++++++++++++++++++++++++++++++++++++ SERVICES +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    # ================================================================================================================================
    # Flat Repair:

    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '4990085')
    AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE (PDNUMBER = '100' OR PDNUMBER = '104') AND PDDESCRIP LIKE 'FLAT%')
    AND TSCODDEL <> 'D'
    """

    Flat_Repair_Columns = {}
    #--------------------------------------------------------------------
    # Each Month values:
    for i, month in enumerate(Months_Numerical):
        value = 0
        if month <= Last_Month:
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
        Month_Name = Months_Strings[i]
        Flat_Repair_Columns[Month_Name] = value
    # Get SUM of YTD Values by adding up each month value:
    Sum_YTD = 0
    for i in Flat_Repair_Columns.values():
        Sum_YTD = Sum_YTD + i
    Flat_Repair_Columns['YTD'] = Sum_YTD

    # Print values:
    for column_header_key, value in Flat_Repair_Columns.items():
        print(f"Service of Flat Repairs {column_header_key} = {value}")        

# ============================================================================================================================
    # Tire Rotation
    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '4990085')
    AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE (PDNUMBER = '110') AND PDDESCRIP LIKE 'ROTA%')
    AND TSCODDEL <> 'D'
    """

    Tire_Rotation_Columns = {}
    # Each month values:
    for i, month in enumerate(Months_Numerical):
        value = 0
        if month <= Last_Month:
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
        Month_Name = Months_Strings[i]
        Tire_Rotation_Columns[Month_Name] = value
    # Get SUM of YTD Values by adding up each month value:
    Sum_YTD = 0
    for i in Tire_Rotation_Columns.values():
        Sum_YTD = Sum_YTD + i
    Tire_Rotation_Columns['YTD'] = Sum_YTD
    # Print values:
    for column_header_key, value in Tire_Rotation_Columns.items():
        print(f"Service of Tire Rotation {column_header_key} = {value}")


    # =================================================================================================================================
    # SERVICE: Mount and Dismount

    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '4990085')
    AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE PDNUMBER IN ('120', '120WR', '121', '124', '132', '134LT', '153')) 
    AND TSCODDEL <> 'D'
    """

    Mount_and_Dismount_Columns = {}
    # Each month values:
    for i, month in enumerate(Months_Numerical):
        value = 0
        if month <= Last_Month:
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
        Month_Name = Months_Strings[i]
        Mount_and_Dismount_Columns[Month_Name] = value
    # Get SUM of YTD Values by adding up each month value:
    Sum_YTD = 0
    for i in Mount_and_Dismount_Columns.values():
        Sum_YTD = Sum_YTD + i
    Mount_and_Dismount_Columns['YTD'] = Sum_YTD
    # Print values:
    for column_header_key, value in Mount_and_Dismount_Columns.items():
        print(f"Service of Mount and Dismount {column_header_key} = {value}")

    # =========================================================================================================================================
    # SERVICE CALL HOURS

    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '4990085')
    AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE PDNUMBER IN ('160', '165', '166', '167', '175', '176', '177', '185', '167UPS', 'LAB191'))
    AND TSCODDEL <> 'D'
    """
    Service_Call_Hours_Columns = {}
    # Each month values:
    for i, month in enumerate(Months_Numerical):
        value = 0
        if month <= Last_Month:
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
        Month_Name = Months_Strings[i]
        Service_Call_Hours_Columns[Month_Name] = value
    # Get SUM of YTD Values by adding up each month value:
    Sum_YTD = 0
    for i in Service_Call_Hours_Columns.values():
        Sum_YTD = Sum_YTD + i
    Service_Call_Hours_Columns['YTD'] = Sum_YTD
    # Print values:
    for column_header_key, value in Service_Call_Hours_Columns.items():
        print(f"Service of Service Call Hours {column_header_key} = {value}")


    # ==================================================================================================================================
    # SCRAP

    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '4990085')
    AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE PDNUMBER IN ('209', '210') OR PDNUMBER LIKE '%SCRAP%' OR PDDESCRIP LIKE '%SCRAP%' OR PDNUMBER LIKE '%SFM%' OR PDDESCRIP LIKE '%TIRE DISPOSAL%')
    AND TSCODDEL <> 'D'
    """

    Scrap_Columns = {}
    # Each month values:
    for i, month in enumerate(Months_Numerical):
        value = 0
        if month <= Last_Month:
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
        Month_Name = Months_Strings[i]
        Scrap_Columns[Month_Name] = value
    # Get SUM of YTD Values by adding up each month value:
    Sum_YTD = 0
    for i in Scrap_Columns.values():
        Sum_YTD = Sum_YTD + i
    Scrap_Columns['YTD'] = Sum_YTD

    # Print values:
    for column_header_key, value in Scrap_Columns.items():
        print(f"Service of Scrap {column_header_key} = {value}")


    # ============================================================================================================================================================
    # +++++++++++++++++++++++++ BANDAG RETREADS ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    # ============================================================================================================================================================
    

# ==================================================================================================================

    # 245/70R17.5 RETREAD

    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '4990085')
    AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE PDNUMBER LIKE '257%' AND PDNUMBER NOT LIKE '%257RC%')
    AND TSCODDEL <> 'D'
    """

    Retread_245_slash_70R17_point_5_Columns = {}
    # Each month values:
    for i, month in enumerate(Months_Numerical):
        value = 0
        if month <= Last_Month:
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
        Month_Name = Months_Strings[i]
        Retread_245_slash_70R17_point_5_Columns[Month_Name] = value
    # Get SUM of YTD Values by adding up each month value:
    Sum_YTD = 0
    for i in Retread_245_slash_70R17_point_5_Columns.values():
        Sum_YTD = Sum_YTD + i
    Retread_245_slash_70R17_point_5_Columns['YTD'] = Sum_YTD

    # Print values:
    for column_header_key, value in Retread_245_slash_70R17_point_5_Columns.items():
        print(f"RETREADS - 245/70R17.5 RETREAD {column_header_key} = {value}")

# ==================================================================================================================
    # 295/75R22.5 RETREAD

    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '4990085')
    AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE PDNUMBER LIKE '354%' AND PDNUMBER NOT LIKE '%354RC%')
    AND TSCODDEL <> 'D'
    """

    Retread_295_slash_75R22_point_5_Columns = {}
    # Each month values:
    for i, month in enumerate(Months_Numerical):
        value = 0
        if month <= Last_Month:
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
        Month_Name = Months_Strings[i]
        Retread_295_slash_75R22_point_5_Columns[Month_Name] = value
    # Get SUM of YTD Values by adding up each month value:
    Sum_YTD = 0
    for i in Retread_295_slash_75R22_point_5_Columns.values():
        Sum_YTD = Sum_YTD + i
    Retread_295_slash_75R22_point_5_Columns['YTD'] = Sum_YTD

    # Print values:
    for column_header_key, value in Retread_295_slash_75R22_point_5_Columns.items():
        print(f"RETREADS - 295/75R22.5 RETREAD {column_header_key} = {value}")




    # =============================================================================================================================================================
    # +++++++++++++++++++++++ RETREAD + CASING +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    # =============================================================================================================================================================
    # 10.00-20 CAP & CASING

    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '4990085')
    AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE PDNUMBER LIKE 'S821%')
    AND TSCODDEL <> 'D'
    """

    Retread_PLUS_CASING_10_point_00_dash_20_Columns = {}
    # Each month values:
    for i, month in enumerate(Months_Numerical):
        value = 0
        if month <= Last_Month:
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
        Month_Name = Months_Strings[i]
        Retread_PLUS_CASING_10_point_00_dash_20_Columns[Month_Name] = value
    # Get SUM of YTD Values by adding up each month value:
    Sum_YTD = 0
    for i in Retread_PLUS_CASING_10_point_00_dash_20_Columns.values():
        Sum_YTD = Sum_YTD + i
    Retread_PLUS_CASING_10_point_00_dash_20_Columns['YTD'] = Sum_YTD

    # Print values:
    for column_header_key, value in Retread_PLUS_CASING_10_point_00_dash_20_Columns.items():
        print(f"RETREAD + CASINGS - 11R22.5 CAP & CASING {column_header_key} = {value}")

    # ====================================================================================================================================================


    # 11R22.5 CAP & CASING

    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '4990085')
    AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE PDNUMBER LIKE 'S388%')
    AND TSCODDEL <> 'D'
    """

    Retread_PLUS_CASING_11R22_point_5_Columns = {}
    # Each month values:
    for i, month in enumerate(Months_Numerical):
        value = 0
        if month <= Last_Month:
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
        Month_Name = Months_Strings[i]
        Retread_PLUS_CASING_11R22_point_5_Columns[Month_Name] = value
    # Get SUM of YTD Values by adding up each month value:
    Sum_YTD = 0
    for i in Retread_PLUS_CASING_11R22_point_5_Columns.values():
        Sum_YTD = Sum_YTD + i
    Retread_PLUS_CASING_11R22_point_5_Columns['YTD'] = Sum_YTD

    # Print values:
    for column_header_key, value in Retread_PLUS_CASING_11R22_point_5_Columns.items():
        print(f"RETREAD + CASINGS - 11R22.5 CAP & CASING {column_header_key} = {value}")

    # ====================================================================================================================================================
    # 295/75R22.5 CAP & CASING

    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '4990085')
    AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE PDNUMBER LIKE 'S354%')
    AND TSCODDEL <> 'D'
    """

    Retread_PLUS_CASING_295_slash_75R22_point_5_Columns = {}
    # Each month values:
    for i, month in enumerate(Months_Numerical):
        value = 0
        if month <= Last_Month:
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
        Month_Name = Months_Strings[i]
        Retread_PLUS_CASING_295_slash_75R22_point_5_Columns[Month_Name] = value
    # Get SUM of YTD Values by adding up each month value:
    Sum_YTD = 0
    for i in Retread_PLUS_CASING_295_slash_75R22_point_5_Columns.values():
        Sum_YTD = Sum_YTD + i
    Retread_PLUS_CASING_295_slash_75R22_point_5_Columns['YTD'] = Sum_YTD

    # Print values:
    for column_header_key, value in Retread_PLUS_CASING_295_slash_75R22_point_5_Columns.items():
        print(f"RETREAD + CASINGS - 295/75R22.5 CAP & CASING {column_header_key} = {value}")
 
    # ====================================================================================================================================================
 
    # =========================================================================================================================================================================
    # REPAIR ONLY

    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '4990085')
    AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE PDNUMBER LIKE '2R0%' OR PDNUMBER LIKE '2B0%')
    AND TSCODDEL <> 'D'
    """
    Repair_Only_Columns = {}
    # Each month values:
    for i, month in enumerate(Months_Numerical):
        value = 0
        if month <= Last_Month:
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
        Month_Name = Months_Strings[i]
        Repair_Only_Columns[Month_Name] = value
    # Get SUM of YTD Values by adding up each month value:
    Sum_YTD = 0
    for i in Repair_Only_Columns.values():
        Sum_YTD = Sum_YTD + i
    Repair_Only_Columns['YTD'] = Sum_YTD
    # Print values:
    for column_header_key, value in Repair_Only_Columns.items():
        print(f"REPAIR ONLY {column_header_key} = {value}")



    # =====================================================================================================================================================
    # +++++++++++++++++++++++++++ BRIDGESTONE +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    # =====================================================================================================================================================

    # (BRIDGESTONE) LIGHT TRUCK

    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '4990085')
    AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE (PDSIZE LIKE '015%' OR PDSIZE LIKE '016%' OR PDSIZE LIKE '017%') 
    AND PDCLASS IN ('06', '07', '08') 
    AND PDNUMBER IN ('B192659', 'B002038', 'B002043', 'B213518', 'B000027', 'B000017', 'B191826', 'B191860', 'B214606', 'B002036', 'B003489') 
    AND (PDVENDOR = '007') 
    AND (TSCODDEL <> 'D'))
    """
        
    Bridgestone_Light_Truck_Columns = {}
    # Each month values:
    for i, month in enumerate(Months_Numerical):
        value = 0
        if month <= Last_Month:
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
        Month_Name = Months_Strings[i]
        Bridgestone_Light_Truck_Columns[Month_Name] = value
    # Get SUM of YTD Values by adding up each month value:
    Sum_YTD = 0
    for i in Bridgestone_Light_Truck_Columns.values():
        Sum_YTD = Sum_YTD + i
    Bridgestone_Light_Truck_Columns['YTD'] = Sum_YTD

    # Print values:
    for column_header_key, value in Bridgestone_Light_Truck_Columns.items():
        print(f"BRIDGESTONE - Light Truck {column_header_key} = {value}")

    # =================================================================================================================================================
    # (BRIDGESTONE) 225/70R19.5

    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '4990085')
    AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE PDSIZE LIKE '%019500225700%' AND PDDESCRIP NOT LIKE '%BILL HOLD%')
    AND TSVN IN ('007')
    AND TSCODDEL <> 'D'
    """
        
    Bridgestone_225_slash_70R19_point_5_Columns = {}
    # Each month values:
    for i, month in enumerate(Months_Numerical):
        value = 0
        if month <= Last_Month:
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
        Month_Name = Months_Strings[i]
        Bridgestone_225_slash_70R19_point_5_Columns[Month_Name] = value
    # Get SUM of YTD Values by adding up each month value:
    Sum_YTD = 0
    for i in Bridgestone_225_slash_70R19_point_5_Columns.values():
        Sum_YTD = Sum_YTD + i
    Bridgestone_225_slash_70R19_point_5_Columns['YTD'] = Sum_YTD

    # Print values:
    for column_header_key, value in Bridgestone_225_slash_70R19_point_5_Columns.items():
        print(f"BRIDGESTONE - 225/70R19.5 {column_header_key} = {value}")

    # =================================================================================================================================================
    # (BRIDGESTONE) 245/70R17.5

    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '4990085')
    AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE PDSIZE LIKE '%017500245700%' AND PDDESCRIP NOT LIKE '%BILL HOLD%')
    AND TSVN IN ('007')
    AND TSCODDEL <> 'D'
    """
        
    Bridgestone_245_slash_70R17_point_5_Columns = {}
    # Each month values:
    for i, month in enumerate(Months_Numerical):
        value = 0
        if month <= Last_Month:
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
        Month_Name = Months_Strings[i]
        Bridgestone_245_slash_70R17_point_5_Columns[Month_Name] = value
    # Get SUM of YTD Values by adding up each month value:
    Sum_YTD = 0
    for i in Bridgestone_245_slash_70R17_point_5_Columns.values():
        Sum_YTD = Sum_YTD + i
    Bridgestone_245_slash_70R17_point_5_Columns['YTD'] = Sum_YTD

    # Print values:
    for column_header_key, value in Bridgestone_245_slash_70R17_point_5_Columns.items():
        print(f"BRIDGESTONE - 245/70R17.5 {column_header_key} = {value}")

    # =================================================================================================================================================
    # (BRIDGESTONE) 9R22.5

    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '4990085')
    AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE PDVENDOR = '007' AND PDSIZE LIKE '%022500009000%' AND PDDESCRIP NOT LIKE '%BILL HOLD%')
    AND TSVN = '007'
    AND TSCODDEL <> 'D'
    """
        
    Bridgestone_9R22_point_5_Columns = {}
    # Each month values:
    for i, month in enumerate(Months_Numerical):
        value = 0
        if month <= Last_Month:
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
        Month_Name = Months_Strings[i]
        Bridgestone_9R22_point_5_Columns[Month_Name] = value
    # Get SUM of YTD Values by adding up each month value:
    Sum_YTD = 0
    for i in Bridgestone_9R22_point_5_Columns.values():
        Sum_YTD = Sum_YTD + i
    Bridgestone_9R22_point_5_Columns['YTD'] = Sum_YTD

    # Print values:
    for column_header_key, value in Bridgestone_9R22_point_5_Columns.items():
        print(f"BRIDGESTONE - 9R22.5 {column_header_key} = {value}")

    # =================================================================================================================================================
    # (BRIDGESTONE) 295/75R22.5

    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '4990085')
    AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE PDVENDOR = '007' AND PDSIZE LIKE '%022500295750%' AND PDDESCRIP NOT LIKE '%BILL HOLD%')
    AND TSCODDEL <> 'D'
    """
        
    Bridgestone_295_slash_75R22_point_5_Columns = {}
    # Each month values:
    for i, month in enumerate(Months_Numerical):
        value = 0
        if month <= Last_Month:
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
        Month_Name = Months_Strings[i]
        Bridgestone_295_slash_75R22_point_5_Columns[Month_Name] = value
    # Get SUM of YTD Values by adding up each month value:
    Sum_YTD = 0
    for i in Bridgestone_295_slash_75R22_point_5_Columns.values():
        Sum_YTD = Sum_YTD + i
    Bridgestone_295_slash_75R22_point_5_Columns['YTD'] = Sum_YTD

    # Print values:
    for column_header_key, value in Bridgestone_295_slash_75R22_point_5_Columns.items():
        print(f"BRIDGESTONE - 295/75R22.5 {column_header_key} = {value}")

    # =================================================================================================================================================

    # (BRIDGESTONE) 11R22.5

    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '4990085')
    AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE PDVENDOR = '007' AND PDSIZE LIKE '%022500011000%' AND PDDESCRIP NOT LIKE '%BILL HOLD%')
    AND TSCODDEL <> 'D'
    """
    Bridgestone_11R22_point_5_Columns = {}
    # Each month values:
    for i, month in enumerate(Months_Numerical):
        value = 0
        if month <= Last_Month:
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
        Month_Name = Months_Strings[i]
        Bridgestone_11R22_point_5_Columns[Month_Name] = value
    # Get SUM of YTD Values by adding up each month value:
    Sum_YTD = 0
    for i in Bridgestone_11R22_point_5_Columns.values():
        Sum_YTD = Sum_YTD + i
    Bridgestone_11R22_point_5_Columns['YTD'] = Sum_YTD

    # Print values:
    for column_header_key, value in Bridgestone_11R22_point_5_Columns.items():
        print(f"BRIDGESTONE - 11R22.5 {column_header_key} = {value}")

    # =======================================================================================================================================================
    # OTHER Bridgestone Tires:

    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '4990085')
    -- Exclude Light Truck (note, light truck is product classes 06,07,08)
    AND TSPD NOT IN (SELECT PDNUMBER FROM TMPROD WHERE (PDSIZE LIKE '015%' OR PDSIZE LIKE '016%' OR PDSIZE LIKE '017%') AND PDCLASS IN ('06', '07', '08') AND PDNUMBER IN ('B192659', 'B002038', 'B002043', 'B213518', 'B000027', 'B000017', 'B191826', 'B191860', 'B214606', 'B002036', 'B003489'))
    -- Exclude 225/70R19.5
    AND TSPD NOT IN (SELECT PDNUMBER FROM TMPROD WHERE PDSIZE LIKE '%019500225700%' AND PDDESCRIP NOT LIKE '%BILL HOLD%')
    ---- Exclude 245/70R17.5
    AND TSPD NOT IN (SELECT PDNUMBER FROM TMPROD WHERE PDSIZE LIKE '%017500245700%' AND PDDESCRIP NOT LIKE '%BILL HOLD%')
    ---- Exclude 9R225
    AND TSPD NOT IN (SELECT PDNUMBER FROM TMPROD WHERE PDSIZE LIKE '%022500009000%' AND PDDESCRIP NOT LIKE '%BILL HOLD%')
    ---- Exclude 295/75R22.5
    AND TSPD NOT IN (SELECT PDNUMBER FROM TMPROD WHERE PDSIZE LIKE '%022500295750%' AND PDDESCRIP NOT LIKE '%BILL HOLD%')
    ---- Exclude 11R22.5
    AND TSPD NOT IN (SELECT PDNUMBER FROM TMPROD WHERE PDSIZE LIKE '%022500011000%' AND PDDESCRIP NOT LIKE '%BILL HOLD%')
    --ENSURE PDFLDUSR1 does not have any value (or at least doesn't have "DC")... that way we exlcude junk/commission lines AND exclude any govt "FET":
    AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE PDDESCRIP NOT LIKE '%COMM%' AND PDFLDUSR1 = '')
    AND TSVN = '007'
    AND TSCODDEL <> 'D'
    """
    Bridgestone_OTHER_Columns = {}
    # Each month values:
    for i, month in enumerate(Months_Numerical):
        value = 0
        if month <= Last_Month:
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
        Month_Name = Months_Strings[i]
        Bridgestone_OTHER_Columns[Month_Name] = value
    # Get SUM of YTD Values by adding up each month value:
    Sum_YTD = 0
    for i in Bridgestone_OTHER_Columns.values():
        Sum_YTD = Sum_YTD + i
    Bridgestone_OTHER_Columns['YTD'] = Sum_YTD

    # Print values:
    for column_header_key, value in Bridgestone_OTHER_Columns.items():
        print(f"BRIDGESTONE - OTHER {column_header_key} = {value}")


# ==========================================================================================================================================================
# ===================   OTHER NEW TIRES (besides bridgestone)  +++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# ==========================================================================================================================================================

    # 10.00-20 NEW TIRES

    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '4990085')
    AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE PDSIZE LIKE '%020001000000%' AND PDDESCRIP NOT LIKE '%BILL HOLD%' AND PDFLDUSR1 = '')
    AND TSVN IN ('007')
    AND TSCODDEL <> 'D'
    """
    New_Tires_10_point_00_dash_20_Columns = {}
    # Each month values:
    for i, month in enumerate(Months_Numerical):
        value = 0
        if month <= Last_Month:
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
        Month_Name = Months_Strings[i]
        New_Tires_10_point_00_dash_20_Columns[Month_Name] = value
    # Get SUM of YTD Values by adding up each month value:
    Sum_YTD = 0
    for i in New_Tires_10_point_00_dash_20_Columns.values():
        Sum_YTD = Sum_YTD + i
    New_Tires_10_point_00_dash_20_Columns['YTD'] = Sum_YTD

    # Print values:
    for column_header_key, value in New_Tires_10_point_00_dash_20_Columns.items():
        print(f"NEW TIRES - 10.00-20 {column_header_key} = {value}")

# ==========================================================================================================
    # YOKOHAMA NEW TIRES

    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '4990085')
    AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE PDVENDOR = '005' AND UPPER(PDDESCRIP) NOT LIKE '%BILL HOLD%' AND PDNUMBER NOT LIKE '%YOKOCOM%' AND PDFLDUSR1 = '')
    AND TSVN = '005'
    AND TSCODDEL <> 'D'
    """
    New_Tires_Yokohama_New_Tires_Columns = {}
    # Each month values:
    for i, month in enumerate(Months_Numerical):
        value = 0
        if month <= Last_Month:
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
        Month_Name = Months_Strings[i]
        New_Tires_Yokohama_New_Tires_Columns[Month_Name] = value
    # Get SUM of YTD Values by adding up each month value:
    Sum_YTD = 0
    for i in New_Tires_Yokohama_New_Tires_Columns.values():
        Sum_YTD = Sum_YTD + i
    New_Tires_Yokohama_New_Tires_Columns['YTD'] = Sum_YTD

    # Print values:
    for column_header_key, value in New_Tires_Yokohama_New_Tires_Columns.items():
        print(f"NEW TIRES - Yokohama {column_header_key} = {value}")

# ============================================================================================================
    # CONTINENTAL NEW TIRES:
    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '4990085')
    AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE PDVENDOR = '021' AND PDDESCRIP NOT LIKE '%BILL HOLD%' AND PDFLDUSR1 = '')
    AND TSVN = '021'    
    AND TSCODDEL <> 'D'
    """
    New_Tires_Continental_New_Tires_Columns = {}
    # Each month values:
    for i, month in enumerate(Months_Numerical):
        value = 0
        if month <= Last_Month:
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
        Month_Name = Months_Strings[i]
        New_Tires_Continental_New_Tires_Columns[Month_Name] = value
    # Get SUM of YTD Values by adding up each month value:
    Sum_YTD = 0
    for i in New_Tires_Continental_New_Tires_Columns.values():
        Sum_YTD = Sum_YTD + i
    New_Tires_Continental_New_Tires_Columns['YTD'] = Sum_YTD

    # Print values:
    for column_header_key, value in New_Tires_Continental_New_Tires_Columns.items():
        print(f"NEW TIRES - Continental {column_header_key} = {value}")

# ============================================================================================================
    # HANKOOK NEW TIRES:
    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) AS TOTAL FROM TMSALE
    -- Get rid of Bill Hold, get rid of anything that is commission lines or FET lines (For govt transactions)
    WHERE TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE TRIM(PDNUMBER) NOT LIKE '%BH' AND PDFLDUSR4 <> 'BH' AND PDNUMBER NOT LIKE 'FET%' AND PDDESCRIP NOT LIKE '%COMM%' AND PDFLDUSR1 = '')
    -- Get rid of assemblies
    AND TSCC <> 'A'
    -- Exclude customer classes 6 (govt support), 8 (intercompany), and 9 (wholesale intercompany)
    AND TSCC NOT IN ('6', '8', '9')
    -- Specify which vendor:
    AND TSVN IN ('027')
    AND TSCODDEL <> 'D'
    """
    New_Tires_Hankook_New_Tires_Columns = {}
    # Each month values:
    for i, month in enumerate(Months_Numerical):
        value = 0
        if month <= Last_Month:
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
        Month_Name = Months_Strings[i]
        New_Tires_Hankook_New_Tires_Columns[Month_Name] = value
    # Get SUM of YTD Values by adding up each month value:
    Sum_YTD = 0
    for i in New_Tires_Hankook_New_Tires_Columns.values():
        Sum_YTD = Sum_YTD + i
    New_Tires_Hankook_New_Tires_Columns['YTD'] = Sum_YTD

    # Print values:
    for column_header_key, value in New_Tires_Hankook_New_Tires_Columns.items():
        print(f"NEW TIRES - Continental {column_header_key} = {value}")



# ==========================================================================================================================================================
    # Misc Tires


    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '4990085')
    -- Exclude Bridgestone Light Truck
    AND TSPD NOT IN (SELECT PDNUMBER FROM TMPROD WHERE (PDSIZE LIKE '015%' OR PDSIZE LIKE '016%' OR PDSIZE LIKE '017%') AND (PDCLASS = '06' OR PDCLASS = '07' OR PDCLASS = '08') AND PDNUMBER IN ('B192659', 'B002038', 'B002043', 'B213518', 'B000027', 'B000017', 'B191826', 'B191860', 'B214606', 'B002036', 'B003489')) 
    -- Exclude Bridgestone 225/70R19.5
    AND TSPD NOT IN (SELECT PDNUMBER FROM TMPROD WHERE PDVENDOR = '007' AND PDSIZE LIKE '%019500225700%' AND PDDESCRIP NOT LIKE '%BILL HOLD%')
    -- Exclude Bridgestone 245/70R17.5
    AND TSPD NOT IN (SELECT PDNUMBER FROM TMPROD WHERE PDVENDOR = '007' AND PDSIZE LIKE '%017500245700%' AND PDDESCRIP NOT LIKE '%BILL HOLD%')
    -- Exclude Bridgestone 9R225
    AND TSPD NOT IN (SELECT PDNUMBER FROM TMPROD WHERE PDVENDOR = '007' AND PDSIZE LIKE '%022500009000%' AND PDDESCRIP NOT LIKE '%BILL HOLD%')
    -- Exclude Bridgestone 295/75R22.5
    AND TSPD NOT IN (SELECT PDNUMBER FROM TMPROD WHERE PDVENDOR = '007' AND PDSIZE LIKE '%022500295750%' AND PDDESCRIP NOT LIKE '%BILL HOLD%')
    -- Exclude Bridgestone 11R22.5
    AND TSPD NOT IN (SELECT PDNUMBER FROM TMPROD WHERE PDVENDOR = '007' AND PDSIZE LIKE '%022500011000%' AND PDDESCRIP NOT LIKE '%BILL HOLD%')
    -- Exclude 10.00-20 New Tires
    AND TSPD NOT IN (SELECT PDNUMBER FROM TMPROD WHERE PDSIZE = '020001000000' AND UPPER(PDDESCRIP) NOT LIKE '%BILL HOLD%' AND PDVENDOR NOT IN ('090', '091'))
    -- Exclude Yokohama New Tires
    AND TSPD NOT IN (SELECT PDNUMBER FROM TMPROD WHERE PDVENDOR = '005' AND UPPER(PDDESCRIP) NOT LIKE '%BILL HOLD%')
    -- Exclude Continental New Tires
    AND TSPD NOT IN (SELECT PDNUMBER FROM TMPROD WHERE PDVENDOR = '021' AND UPPER(PDDESCRIP) NOT LIKE '%BILL HOLD%')
    -- Exclude HANKOOK New Tires:
    AND TSPD NOT IN (SELECT PDNUMBER FROM TMPROD WHERE PDVENDOR = '027' AND UPPER(PDDESCRIP) NOT LIKE '%BILL HOLD%')
    -- Here just make sure we exclude non-inventory and only get the tires which are of relevant classes (not OTRs etc)
    AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE PDNONINV <> 'Y' AND PDCLASS IN ('01', '02', '03', '04', '05','06', '07', '08', '09','10','11','12','13','14','15','16','17','18','19','20','21'))
    --ENSURE PDFLDUSR1 does not have any value (or at least doesn't have "DC")... that way we exlcude junk/commission lines:
    AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE PDFLDUSR1 <> 'DC')
    AND TSCODDEL <> 'D'
    """

    Misc_Tires_Columns = {}
    # Each month values:
    for i, month in enumerate(Months_Numerical):
        value = 0
        if month <= Last_Month:
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
        Month_Name = Months_Strings[i]
        Misc_Tires_Columns[Month_Name] = value
    # Get SUM of YTD Values by adding up each month value:
    Sum_YTD = 0
    for i in Misc_Tires_Columns.values():
        Sum_YTD = Sum_YTD + i
    Misc_Tires_Columns['YTD'] = Sum_YTD
    # Print values:
    for column_header_key, value in Misc_Tires_Columns.items():
        print(f"MISC TIRES {column_header_key} = {value}")


# ======================================================================================================================================================
# ++++++++++++++++++++++++ 3 EXTRA BRIDGESTONE NEW TIRE CATEGORIES +++++++++++++++++++++++++++++++++++++++++++++++++
# ======================================================================================================================================================
    # Bridgestone STEER - NEW TIRE
        # Notes:
        # Lonestar customers are all under Enterprise number 4990085
        # Bridgestone is vendor '007'.
        # Product class for Steer is "10".
        # when I did the raynond alley report, Sandra said PDFLDUSR1 needs to have no values in it.  In this case, it may make no difference but still.

    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '4990085')
    AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE UPPER(PDDESCRIP) NOT LIKE '%BILL HOLD%' AND PDFLDUSR1 = '')
    AND TSVN = '007'
    AND TSPC = '10'
    AND TSCODDEL <> 'D'
    """
    New_Tires_BS_STEER_Columns = {}
    # Each month values:
    for i, month in enumerate(Months_Numerical):
        value = 0
        if month <= Last_Month:
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
        Month_Name = Months_Strings[i]
        New_Tires_BS_STEER_Columns[Month_Name] = value
    # Get SUM of YTD Values by adding up each month value:
    Sum_YTD = 0
    for i in New_Tires_BS_STEER_Columns.values():
        Sum_YTD = Sum_YTD + i
    New_Tires_BS_STEER_Columns['YTD'] = Sum_YTD

    # Print values:
    for column_header_key, value in New_Tires_BS_STEER_Columns.items():
        print(f"NEW TIRES - BS STEER {column_header_key} = {value}")
        

    # Bridgestone DRIVE - NEW TIRE
    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '4990085')
    AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE UPPER(PDDESCRIP) NOT LIKE '%BILL HOLD%' AND PDFLDUSR1 = '')
    AND TSVN = '007'
    AND TSPC = '11'
    AND TSCODDEL <> 'D'
    """
    New_Tires_BS_DRIVE_Columns = {}
    # Each month values:
    for i, month in enumerate(Months_Numerical):
        value = 0
        if month <= Last_Month:
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
        Month_Name = Months_Strings[i]
        New_Tires_BS_DRIVE_Columns[Month_Name] = value
    # Get SUM of YTD Values by adding up each month value:
    Sum_YTD = 0
    for i in New_Tires_BS_DRIVE_Columns.values():
        Sum_YTD = Sum_YTD + i
    New_Tires_BS_DRIVE_Columns['YTD'] = Sum_YTD

    # Print values:
    for column_header_key, value in New_Tires_BS_DRIVE_Columns.items():
        print(f"NEW TIRES - BS DRIVE {column_header_key} = {value}")


    # Bridgestone TRAILER - NEW TIRE
    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '4990085')
    AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE UPPER(PDDESCRIP) NOT LIKE '%BILL HOLD%' AND PDFLDUSR1 = '')
    AND TSVN = '007'
    AND TSPC = '12'
    AND TSCODDEL <> 'D'
    """
    New_Tires_BS_TRAILER_Columns = {}
    # Each month values:
    for i, month in enumerate(Months_Numerical):
        value = 0
        if month <= Last_Month:
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
        Month_Name = Months_Strings[i]
        New_Tires_BS_TRAILER_Columns[Month_Name] = value
    # Get SUM of YTD Values by adding up each month value:
    Sum_YTD = 0
    for i in New_Tires_BS_TRAILER_Columns.values():
        Sum_YTD = Sum_YTD + i
    New_Tires_BS_TRAILER_Columns['YTD'] = Sum_YTD

    # Print values:
    for column_header_key, value in New_Tires_BS_TRAILER_Columns.items():
        print(f"NEW TIRES - BS TRAILER {column_header_key} = {value}")


    


    # ========================================================================================================================================================
    # ++++++++++++++++++++++++ ALL TOTALS CALCULATIONS HERE +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    # ========================================================================================================================================================

    Service_YTD_TOTAL = Flat_Repair_Columns["YTD"] + Tire_Rotation_Columns['YTD'] + Mount_and_Dismount_Columns["YTD"] + Service_Call_Hours_Columns["YTD"] + Scrap_Columns["YTD"]
    Service_JAN_TOTAL = Flat_Repair_Columns["JAN"] + Tire_Rotation_Columns['JAN'] + Mount_and_Dismount_Columns["JAN"] + Service_Call_Hours_Columns["JAN"] + Scrap_Columns["JAN"]
    Service_FEB_TOTAL = Flat_Repair_Columns["FEB"] + Tire_Rotation_Columns['FEB'] + Mount_and_Dismount_Columns["FEB"] + Service_Call_Hours_Columns["FEB"] + Scrap_Columns["FEB"]
    Service_MAR_TOTAL = Flat_Repair_Columns["MAR"] + Tire_Rotation_Columns['MAR'] + Mount_and_Dismount_Columns["MAR"] + Service_Call_Hours_Columns["MAR"] + Scrap_Columns["MAR"]
    Service_APR_TOTAL = Flat_Repair_Columns["APR"] + Tire_Rotation_Columns['APR'] + Mount_and_Dismount_Columns["APR"] + Service_Call_Hours_Columns["APR"] + Scrap_Columns["APR"]
    Service_MAY_TOTAL = Flat_Repair_Columns["MAY"] + Tire_Rotation_Columns['MAY'] + Mount_and_Dismount_Columns["MAY"] + Service_Call_Hours_Columns["MAY"] + Scrap_Columns["MAY"]
    Service_JUN_TOTAL = Flat_Repair_Columns["JUN"] + Tire_Rotation_Columns['JUN'] + Mount_and_Dismount_Columns["JUN"] + Service_Call_Hours_Columns["JUN"] + Scrap_Columns["JUN"]
    Service_JUL_TOTAL = Flat_Repair_Columns["JUL"] + Tire_Rotation_Columns['JUL'] + Mount_and_Dismount_Columns["JUL"] + Service_Call_Hours_Columns["JUL"] + Scrap_Columns["JUL"]
    Service_AUG_TOTAL = Flat_Repair_Columns["AUG"] + Tire_Rotation_Columns['AUG'] + Mount_and_Dismount_Columns["AUG"] + Service_Call_Hours_Columns["AUG"] + Scrap_Columns["AUG"]
    Service_SEPT_TOTAL = Flat_Repair_Columns["SEPT"] + Tire_Rotation_Columns['SEPT'] + Mount_and_Dismount_Columns["SEPT"] + Service_Call_Hours_Columns["SEPT"] + Scrap_Columns["SEPT"]
    Service_OCT_TOTAL = Flat_Repair_Columns["OCT"] + Tire_Rotation_Columns['OCT'] + Mount_and_Dismount_Columns["OCT"] + Service_Call_Hours_Columns["OCT"] + Scrap_Columns["OCT"]
    Service_NOV_TOTAL = Flat_Repair_Columns["NOV"] + Tire_Rotation_Columns['NOV'] + Mount_and_Dismount_Columns["NOV"] + Service_Call_Hours_Columns["NOV"] + Scrap_Columns["NOV"]
    Service_DEC_TOTAL = Flat_Repair_Columns["DEC"] + Tire_Rotation_Columns['DEC'] + Mount_and_Dismount_Columns["DEC"] + Service_Call_Hours_Columns["DEC"] + Scrap_Columns["DEC"]

    # Bandag Retreads Total
    Bandag_Retread_YTD_TOTAL = (Retread_245_slash_70R17_point_5_Columns["YTD"] + Retread_295_slash_75R22_point_5_Columns['YTD'])
    Bandag_Retread_JAN_TOTAL = (Retread_245_slash_70R17_point_5_Columns["JAN"] + Retread_295_slash_75R22_point_5_Columns["JAN"])
    Bandag_Retread_FEB_TOTAL = (Retread_245_slash_70R17_point_5_Columns["FEB"] + Retread_295_slash_75R22_point_5_Columns['FEB'])
    Bandag_Retread_MAR_TOTAL = (Retread_245_slash_70R17_point_5_Columns["MAR"] + Retread_295_slash_75R22_point_5_Columns['MAR'])
    Bandag_Retread_APR_TOTAL = (Retread_245_slash_70R17_point_5_Columns["APR"] + Retread_295_slash_75R22_point_5_Columns['APR'])
    Bandag_Retread_MAY_TOTAL = (Retread_245_slash_70R17_point_5_Columns["MAY"] + Retread_295_slash_75R22_point_5_Columns['MAY'])
    Bandag_Retread_JUN_TOTAL = (Retread_245_slash_70R17_point_5_Columns["JUN"] + Retread_295_slash_75R22_point_5_Columns['JUN'])
    Bandag_Retread_JUL_TOTAL = (Retread_245_slash_70R17_point_5_Columns["JUL"] + Retread_295_slash_75R22_point_5_Columns['JUL'])
    Bandag_Retread_AUG_TOTAL = (Retread_245_slash_70R17_point_5_Columns["AUG"] + Retread_295_slash_75R22_point_5_Columns['AUG'])
    Bandag_Retread_SEPT_TOTAL = (Retread_245_slash_70R17_point_5_Columns["SEPT"] + Retread_295_slash_75R22_point_5_Columns['SEPT'])
    Bandag_Retread_OCT_TOTAL = (Retread_245_slash_70R17_point_5_Columns["OCT"] + Retread_295_slash_75R22_point_5_Columns['OCT'])
    Bandag_Retread_NOV_TOTAL = (Retread_245_slash_70R17_point_5_Columns["NOV"] + Retread_295_slash_75R22_point_5_Columns['NOV'])
    Bandag_Retread_DEC_TOTAL = (Retread_245_slash_70R17_point_5_Columns["DEC"] + Retread_295_slash_75R22_point_5_Columns['DEC'])

    # RETREAD & CASING
    Retread_and_Casing_YTD_TOTAL = Retread_PLUS_CASING_10_point_00_dash_20_Columns["YTD"] + Retread_PLUS_CASING_11R22_point_5_Columns["YTD"] + Retread_PLUS_CASING_295_slash_75R22_point_5_Columns['YTD']
    Retread_and_Casing_JAN_TOTAL = Retread_PLUS_CASING_10_point_00_dash_20_Columns["JAN"] + Retread_PLUS_CASING_11R22_point_5_Columns["JAN"] + Retread_PLUS_CASING_295_slash_75R22_point_5_Columns['JAN']
    Retread_and_Casing_FEB_TOTAL = Retread_PLUS_CASING_10_point_00_dash_20_Columns["FEB"] + Retread_PLUS_CASING_11R22_point_5_Columns["FEB"] + Retread_PLUS_CASING_295_slash_75R22_point_5_Columns['FEB']
    Retread_and_Casing_MAR_TOTAL = Retread_PLUS_CASING_10_point_00_dash_20_Columns["MAR"] + Retread_PLUS_CASING_11R22_point_5_Columns["MAR"] + Retread_PLUS_CASING_295_slash_75R22_point_5_Columns['MAR']
    Retread_and_Casing_APR_TOTAL = Retread_PLUS_CASING_10_point_00_dash_20_Columns["APR"] + Retread_PLUS_CASING_11R22_point_5_Columns["APR"] + Retread_PLUS_CASING_295_slash_75R22_point_5_Columns['APR']
    Retread_and_Casing_MAY_TOTAL = Retread_PLUS_CASING_10_point_00_dash_20_Columns["MAY"] + Retread_PLUS_CASING_11R22_point_5_Columns["MAY"] + Retread_PLUS_CASING_295_slash_75R22_point_5_Columns['MAY']
    Retread_and_Casing_JUN_TOTAL = Retread_PLUS_CASING_10_point_00_dash_20_Columns["JUN"] + Retread_PLUS_CASING_11R22_point_5_Columns["JUN"] + Retread_PLUS_CASING_295_slash_75R22_point_5_Columns['JUN']
    Retread_and_Casing_JUL_TOTAL = Retread_PLUS_CASING_10_point_00_dash_20_Columns["JUL"] + Retread_PLUS_CASING_11R22_point_5_Columns["JUL"] + Retread_PLUS_CASING_295_slash_75R22_point_5_Columns['JUL']
    Retread_and_Casing_AUG_TOTAL = Retread_PLUS_CASING_10_point_00_dash_20_Columns["AUG"] + Retread_PLUS_CASING_11R22_point_5_Columns["AUG"] + Retread_PLUS_CASING_295_slash_75R22_point_5_Columns['AUG']
    Retread_and_Casing_SEPT_TOTAL = Retread_PLUS_CASING_10_point_00_dash_20_Columns["SEPT"] + Retread_PLUS_CASING_11R22_point_5_Columns["SEPT"] + Retread_PLUS_CASING_295_slash_75R22_point_5_Columns['SEPT']
    Retread_and_Casing_OCT_TOTAL = Retread_PLUS_CASING_10_point_00_dash_20_Columns["OCT"] + Retread_PLUS_CASING_11R22_point_5_Columns["OCT"] + Retread_PLUS_CASING_295_slash_75R22_point_5_Columns['OCT']
    Retread_and_Casing_NOV_TOTAL = Retread_PLUS_CASING_10_point_00_dash_20_Columns["NOV"] + Retread_PLUS_CASING_11R22_point_5_Columns["NOV"] + Retread_PLUS_CASING_295_slash_75R22_point_5_Columns['NOV']
    Retread_and_Casing_DEC_TOTAL = Retread_PLUS_CASING_10_point_00_dash_20_Columns["DEC"] +  Retread_PLUS_CASING_11R22_point_5_Columns["DEC"] + Retread_PLUS_CASING_295_slash_75R22_point_5_Columns['DEC']

    # BRIDGESTONE TIRES TOTALS
    Bridgestone_YTD_TOTAL = Bridgestone_Light_Truck_Columns["YTD"] + Bridgestone_225_slash_70R19_point_5_Columns["YTD"] + Bridgestone_245_slash_70R17_point_5_Columns["YTD"] + Bridgestone_9R22_point_5_Columns['YTD'] + Bridgestone_295_slash_75R22_point_5_Columns['YTD'] + Bridgestone_11R22_point_5_Columns['YTD'] + Bridgestone_OTHER_Columns['YTD']
    Bridgestone_JAN_TOTAL = Bridgestone_Light_Truck_Columns["JAN"] + Bridgestone_225_slash_70R19_point_5_Columns["JAN"] + Bridgestone_245_slash_70R17_point_5_Columns["JAN"] + Bridgestone_9R22_point_5_Columns['JAN'] + Bridgestone_295_slash_75R22_point_5_Columns['JAN'] + Bridgestone_11R22_point_5_Columns['JAN'] + Bridgestone_OTHER_Columns['JAN']
    Bridgestone_FEB_TOTAL = Bridgestone_Light_Truck_Columns["FEB"] + Bridgestone_225_slash_70R19_point_5_Columns["FEB"] + Bridgestone_245_slash_70R17_point_5_Columns["FEB"] + Bridgestone_9R22_point_5_Columns['FEB'] + Bridgestone_295_slash_75R22_point_5_Columns['FEB'] + Bridgestone_11R22_point_5_Columns['FEB'] + Bridgestone_OTHER_Columns['FEB']
    Bridgestone_MAR_TOTAL = Bridgestone_Light_Truck_Columns["MAR"] + Bridgestone_225_slash_70R19_point_5_Columns["MAR"] + Bridgestone_245_slash_70R17_point_5_Columns["MAR"] + Bridgestone_9R22_point_5_Columns['MAR'] + Bridgestone_295_slash_75R22_point_5_Columns['MAR'] + Bridgestone_11R22_point_5_Columns['MAR'] + Bridgestone_OTHER_Columns['MAR']
    Bridgestone_APR_TOTAL = Bridgestone_Light_Truck_Columns["APR"] + Bridgestone_225_slash_70R19_point_5_Columns["APR"] + Bridgestone_245_slash_70R17_point_5_Columns["APR"] + Bridgestone_9R22_point_5_Columns['APR'] + Bridgestone_295_slash_75R22_point_5_Columns['APR'] + Bridgestone_11R22_point_5_Columns['APR'] + Bridgestone_OTHER_Columns['APR']
    Bridgestone_MAY_TOTAL = Bridgestone_Light_Truck_Columns["MAY"] + Bridgestone_225_slash_70R19_point_5_Columns["MAY"] + Bridgestone_245_slash_70R17_point_5_Columns["MAY"] + Bridgestone_9R22_point_5_Columns['MAY'] + Bridgestone_295_slash_75R22_point_5_Columns['MAY'] + Bridgestone_11R22_point_5_Columns['MAY'] + Bridgestone_OTHER_Columns['MAY']
    Bridgestone_JUN_TOTAL = Bridgestone_Light_Truck_Columns["JUN"] + Bridgestone_225_slash_70R19_point_5_Columns["JUN"] + Bridgestone_245_slash_70R17_point_5_Columns["JUN"] + Bridgestone_9R22_point_5_Columns['JUN'] + Bridgestone_295_slash_75R22_point_5_Columns['JUN'] + Bridgestone_11R22_point_5_Columns['JUN'] + Bridgestone_OTHER_Columns['JUN']
    Bridgestone_JUL_TOTAL = Bridgestone_Light_Truck_Columns["JUL"] + Bridgestone_225_slash_70R19_point_5_Columns["JUL"] + Bridgestone_245_slash_70R17_point_5_Columns["JUL"] + Bridgestone_9R22_point_5_Columns['JUL'] + Bridgestone_295_slash_75R22_point_5_Columns['JUL'] + Bridgestone_11R22_point_5_Columns['JUL'] + Bridgestone_OTHER_Columns['JUL']
    Bridgestone_AUG_TOTAL = Bridgestone_Light_Truck_Columns["AUG"] + Bridgestone_225_slash_70R19_point_5_Columns["AUG"] + Bridgestone_245_slash_70R17_point_5_Columns["AUG"] + Bridgestone_9R22_point_5_Columns['AUG'] + Bridgestone_295_slash_75R22_point_5_Columns['AUG'] + Bridgestone_11R22_point_5_Columns['AUG'] + Bridgestone_OTHER_Columns['AUG']
    Bridgestone_SEPT_TOTAL = Bridgestone_Light_Truck_Columns["SEPT"] + Bridgestone_225_slash_70R19_point_5_Columns["SEPT"] + Bridgestone_245_slash_70R17_point_5_Columns["SEPT"] + Bridgestone_9R22_point_5_Columns['SEPT'] + Bridgestone_295_slash_75R22_point_5_Columns['SEPT'] + Bridgestone_11R22_point_5_Columns['SEPT'] + Bridgestone_OTHER_Columns['SEPT']
    Bridgestone_OCT_TOTAL = Bridgestone_Light_Truck_Columns["OCT"] + Bridgestone_225_slash_70R19_point_5_Columns["OCT"] + Bridgestone_245_slash_70R17_point_5_Columns["OCT"] + Bridgestone_9R22_point_5_Columns['OCT'] + Bridgestone_295_slash_75R22_point_5_Columns['OCT'] + Bridgestone_11R22_point_5_Columns['OCT'] + Bridgestone_OTHER_Columns['OCT']
    Bridgestone_NOV_TOTAL = Bridgestone_Light_Truck_Columns["NOV"] + Bridgestone_225_slash_70R19_point_5_Columns["NOV"] + Bridgestone_245_slash_70R17_point_5_Columns["NOV"] + Bridgestone_9R22_point_5_Columns['NOV'] + Bridgestone_295_slash_75R22_point_5_Columns['NOV'] + Bridgestone_11R22_point_5_Columns['NOV'] + Bridgestone_OTHER_Columns['NOV']
    Bridgestone_DEC_TOTAL = Bridgestone_Light_Truck_Columns["DEC"] + Bridgestone_225_slash_70R19_point_5_Columns["DEC"] + Bridgestone_245_slash_70R17_point_5_Columns["DEC"] + Bridgestone_9R22_point_5_Columns['DEC'] + Bridgestone_295_slash_75R22_point_5_Columns['DEC'] + Bridgestone_11R22_point_5_Columns['DEC'] + Bridgestone_OTHER_Columns['DEC']

    # NEW TIRES
    New_Tires_YTD_TOTAL = Bridgestone_YTD_TOTAL + New_Tires_10_point_00_dash_20_Columns['YTD'] + New_Tires_Yokohama_New_Tires_Columns['YTD'] + New_Tires_Continental_New_Tires_Columns['YTD'] + New_Tires_Hankook_New_Tires_Columns['YTD'] + Misc_Tires_Columns["YTD"]
    New_Tires_JAN_TOTAL = Bridgestone_JAN_TOTAL + New_Tires_10_point_00_dash_20_Columns['JAN'] + New_Tires_Yokohama_New_Tires_Columns['JAN'] + New_Tires_Continental_New_Tires_Columns['JAN'] + New_Tires_Hankook_New_Tires_Columns['JAN'] + Misc_Tires_Columns["JAN"]
    New_Tires_FEB_TOTAL = Bridgestone_FEB_TOTAL + New_Tires_10_point_00_dash_20_Columns['FEB'] + New_Tires_Yokohama_New_Tires_Columns['FEB'] + New_Tires_Continental_New_Tires_Columns['FEB'] + New_Tires_Hankook_New_Tires_Columns['FEB'] + Misc_Tires_Columns["FEB"]
    New_Tires_MAR_TOTAL = Bridgestone_MAR_TOTAL + New_Tires_10_point_00_dash_20_Columns['MAR'] + New_Tires_Yokohama_New_Tires_Columns['MAR'] + New_Tires_Continental_New_Tires_Columns['MAR'] + New_Tires_Hankook_New_Tires_Columns['MAR'] + Misc_Tires_Columns["MAR"]
    New_Tires_APR_TOTAL = Bridgestone_APR_TOTAL + New_Tires_10_point_00_dash_20_Columns['APR'] + New_Tires_Yokohama_New_Tires_Columns['APR'] + New_Tires_Continental_New_Tires_Columns['APR'] + New_Tires_Hankook_New_Tires_Columns['APR'] + Misc_Tires_Columns["APR"]
    New_Tires_MAY_TOTAL = Bridgestone_MAY_TOTAL + New_Tires_10_point_00_dash_20_Columns['MAY'] + New_Tires_Yokohama_New_Tires_Columns['MAY'] + New_Tires_Continental_New_Tires_Columns['MAY'] + New_Tires_Hankook_New_Tires_Columns['MAY'] + Misc_Tires_Columns["MAY"]
    New_Tires_JUN_TOTAL = Bridgestone_JUN_TOTAL + New_Tires_10_point_00_dash_20_Columns['JUN'] + New_Tires_Yokohama_New_Tires_Columns['JUN'] + New_Tires_Continental_New_Tires_Columns['JUN'] + New_Tires_Hankook_New_Tires_Columns['JUN'] + Misc_Tires_Columns["JUN"]
    New_Tires_JUL_TOTAL = Bridgestone_JUL_TOTAL + New_Tires_10_point_00_dash_20_Columns['JUL'] + New_Tires_Yokohama_New_Tires_Columns['JUL'] + New_Tires_Continental_New_Tires_Columns['JUL'] + New_Tires_Hankook_New_Tires_Columns['JUL'] + Misc_Tires_Columns["JUL"]
    New_Tires_AUG_TOTAL = Bridgestone_AUG_TOTAL + New_Tires_10_point_00_dash_20_Columns['AUG'] + New_Tires_Yokohama_New_Tires_Columns['AUG'] + New_Tires_Continental_New_Tires_Columns['AUG'] + New_Tires_Hankook_New_Tires_Columns['AUG'] + Misc_Tires_Columns["AUG"]
    New_Tires_SEPT_TOTAL = Bridgestone_SEPT_TOTAL + New_Tires_10_point_00_dash_20_Columns['SEPT'] + New_Tires_Yokohama_New_Tires_Columns['SEPT'] + New_Tires_Continental_New_Tires_Columns['SEPT'] + New_Tires_Hankook_New_Tires_Columns['SEPT'] + Misc_Tires_Columns["SEPT"]
    New_Tires_OCT_TOTAL = Bridgestone_OCT_TOTAL +  New_Tires_10_point_00_dash_20_Columns['OCT'] + New_Tires_Yokohama_New_Tires_Columns['OCT'] + New_Tires_Continental_New_Tires_Columns['OCT'] + New_Tires_Hankook_New_Tires_Columns['OCT'] + Misc_Tires_Columns["OCT"]
    New_Tires_NOV_TOTAL = Bridgestone_NOV_TOTAL + New_Tires_10_point_00_dash_20_Columns['NOV'] + New_Tires_Yokohama_New_Tires_Columns['NOV'] + New_Tires_Continental_New_Tires_Columns['NOV'] + New_Tires_Hankook_New_Tires_Columns['NOV'] + Misc_Tires_Columns["NOV"]
    New_Tires_DEC_TOTAL = Bridgestone_DEC_TOTAL + New_Tires_10_point_00_dash_20_Columns['DEC'] + New_Tires_Yokohama_New_Tires_Columns['DEC'] + New_Tires_Continental_New_Tires_Columns['DEC'] + New_Tires_Hankook_New_Tires_Columns['DEC'] + Misc_Tires_Columns["DEC"]

    # OTHER NEW TIRES (BESIDES BRIDGESTONE):
    # Other_New_Tires_YTD_TOTAL = New_Tires_YTD_TOTAL - Bridgestone_YTD_TOTAL
    # Other_New_Tires_JAN_TOTAL = New_Tires_JAN_TOTAL - Bridgestone_JAN_TOTAL
    # Other_New_Tires_FEB_TOTAL = New_Tires_FEB_TOTAL - Bridgestone_FEB_TOTAL
    # Other_New_Tires_MAR_TOTAL = New_Tires_MAR_TOTAL - Bridgestone_MAR_TOTAL
    # Other_New_Tires_APR_TOTAL = New_Tires_APR_TOTAL - Bridgestone_APR_TOTAL
    # Other_New_Tires_MAY_TOTAL = New_Tires_MAY_TOTAL - Bridgestone_MAY_TOTAL
    # Other_New_Tires_JUN_TOTAL = New_Tires_JUN_TOTAL - Bridgestone_JUN_TOTAL
    # Other_New_Tires_JUL_TOTAL = New_Tires_JUL_TOTAL - Bridgestone_JUL_TOTAL
    # Other_New_Tires_AUG_TOTAL = New_Tires_AUG_TOTAL - Bridgestone_AUG_TOTAL
    # Other_New_Tires_SEPT_TOTAL = New_Tires_SEPT_TOTAL - Bridgestone_SEPT_TOTAL
    # Other_New_Tires_OCT_TOTAL = New_Tires_OCT_TOTAL - Bridgestone_OCT_TOTAL
    # Other_New_Tires_NOV_TOTAL = New_Tires_NOV_TOTAL - Bridgestone_NOV_TOTAL
    # Other_New_Tires_DEC_TOTAL = New_Tires_DEC_TOTAL - Bridgestone_DEC_TOTAL

    # NOW, with each loop through the last and current year, we'll create the Columns:

    Sales_Table_df['YTD'] = [Service_YTD_TOTAL, 
                            Flat_Repair_Columns["YTD"], 
                            Tire_Rotation_Columns['YTD'],
                            Mount_and_Dismount_Columns["YTD"], 
                            Service_Call_Hours_Columns["YTD"], 
                            Scrap_Columns["YTD"],
                            Bandag_Retread_YTD_TOTAL,
                            Retread_245_slash_70R17_point_5_Columns["YTD"], 
                            Retread_295_slash_75R22_point_5_Columns['YTD'], 
                            Retread_and_Casing_YTD_TOTAL,
                            Retread_PLUS_CASING_10_point_00_dash_20_Columns["YTD"], 
                            Retread_PLUS_CASING_11R22_point_5_Columns["YTD"], 
                            Retread_PLUS_CASING_295_slash_75R22_point_5_Columns['YTD'],
                            Repair_Only_Columns["YTD"],
                            New_Tires_YTD_TOTAL, 
                            Bridgestone_YTD_TOTAL, 
                            Bridgestone_Light_Truck_Columns["YTD"], 
                            Bridgestone_225_slash_70R19_point_5_Columns["YTD"], 
                            Bridgestone_245_slash_70R17_point_5_Columns["YTD"], 
                            Bridgestone_9R22_point_5_Columns['YTD'], 
                            Bridgestone_295_slash_75R22_point_5_Columns['YTD'], 
                            Bridgestone_11R22_point_5_Columns['YTD'],
                            Bridgestone_OTHER_Columns['YTD'],
                            New_Tires_10_point_00_dash_20_Columns['YTD'], 
                            New_Tires_Yokohama_New_Tires_Columns['YTD'], 
                            New_Tires_Continental_New_Tires_Columns['YTD'],
                            New_Tires_Hankook_New_Tires_Columns['YTD'],
                            Misc_Tires_Columns["YTD"], 
                            New_Tires_BS_STEER_Columns['YTD'], 
                            New_Tires_BS_DRIVE_Columns['YTD'], 
                            New_Tires_BS_TRAILER_Columns['YTD']]

    Sales_Table_df['JAN'] = [Service_JAN_TOTAL, 
                            Flat_Repair_Columns["JAN"], 
                            Tire_Rotation_Columns['JAN'],
                            Mount_and_Dismount_Columns["JAN"], 
                            Service_Call_Hours_Columns["JAN"], 
                            Scrap_Columns["JAN"],
                            Bandag_Retread_JAN_TOTAL, 
                            Retread_245_slash_70R17_point_5_Columns["JAN"], 
                            Retread_295_slash_75R22_point_5_Columns['JAN'], 
                            Retread_and_Casing_JAN_TOTAL,
                            Retread_PLUS_CASING_10_point_00_dash_20_Columns["JAN"], 
                            Retread_PLUS_CASING_11R22_point_5_Columns["JAN"], 
                            Retread_PLUS_CASING_295_slash_75R22_point_5_Columns['JAN'],
                            Repair_Only_Columns["JAN"],
                            New_Tires_JAN_TOTAL, 
                            Bridgestone_JAN_TOTAL, 
                            Bridgestone_Light_Truck_Columns["JAN"], 
                            Bridgestone_225_slash_70R19_point_5_Columns["JAN"], 
                            Bridgestone_245_slash_70R17_point_5_Columns["JAN"], 
                            Bridgestone_9R22_point_5_Columns['JAN'], 
                            Bridgestone_295_slash_75R22_point_5_Columns['JAN'], 
                            Bridgestone_11R22_point_5_Columns['JAN'],
                            Bridgestone_OTHER_Columns['JAN'],
                            New_Tires_10_point_00_dash_20_Columns['JAN'], 
                            New_Tires_Yokohama_New_Tires_Columns['JAN'], 
                            New_Tires_Continental_New_Tires_Columns['JAN'],
                            New_Tires_Hankook_New_Tires_Columns['JAN'],
                            Misc_Tires_Columns["JAN"], 
                            New_Tires_BS_STEER_Columns['JAN'], 
                            New_Tires_BS_DRIVE_Columns['JAN'], 
                            New_Tires_BS_TRAILER_Columns['JAN']]

    Sales_Table_df['FEB'] = [Service_FEB_TOTAL, 
                            Flat_Repair_Columns["FEB"], 
                            Tire_Rotation_Columns['FEB'],
                            Mount_and_Dismount_Columns["FEB"], 
                            Service_Call_Hours_Columns["FEB"], 
                            Scrap_Columns["FEB"],
                            Bandag_Retread_FEB_TOTAL, 
                            Retread_245_slash_70R17_point_5_Columns["FEB"], 
                            Retread_295_slash_75R22_point_5_Columns['FEB'], 
                            Retread_and_Casing_FEB_TOTAL,
                            Retread_PLUS_CASING_10_point_00_dash_20_Columns["FEB"], 
                            Retread_PLUS_CASING_11R22_point_5_Columns["FEB"], 
                            Retread_PLUS_CASING_295_slash_75R22_point_5_Columns['FEB'],
                            Repair_Only_Columns["FEB"],
                            New_Tires_FEB_TOTAL, 
                            Bridgestone_FEB_TOTAL, 
                            Bridgestone_Light_Truck_Columns["FEB"], 
                            Bridgestone_225_slash_70R19_point_5_Columns["FEB"], 
                            Bridgestone_245_slash_70R17_point_5_Columns["FEB"], 
                            Bridgestone_9R22_point_5_Columns['FEB'], 
                            Bridgestone_295_slash_75R22_point_5_Columns['FEB'], 
                            Bridgestone_11R22_point_5_Columns['FEB'],
                            Bridgestone_OTHER_Columns['FEB'],
                            New_Tires_10_point_00_dash_20_Columns['FEB'], 
                            New_Tires_Yokohama_New_Tires_Columns['FEB'], 
                            New_Tires_Continental_New_Tires_Columns['FEB'],
                            New_Tires_Hankook_New_Tires_Columns['FEB'],
                            Misc_Tires_Columns["FEB"], 
                            New_Tires_BS_STEER_Columns['FEB'], 
                            New_Tires_BS_DRIVE_Columns['FEB'], 
                            New_Tires_BS_TRAILER_Columns['FEB']]

    Sales_Table_df['MAR'] = [Service_MAR_TOTAL, 
                            Flat_Repair_Columns["MAR"], 
                            Tire_Rotation_Columns['MAR'],
                            Mount_and_Dismount_Columns["MAR"], 
                            Service_Call_Hours_Columns["MAR"], 
                            Scrap_Columns["MAR"],
                            Bandag_Retread_MAR_TOTAL, 
                            Retread_245_slash_70R17_point_5_Columns["MAR"], 
                            Retread_295_slash_75R22_point_5_Columns['MAR'], 
                            Retread_and_Casing_MAR_TOTAL,
                            Retread_PLUS_CASING_10_point_00_dash_20_Columns["MAR"], 
                            Retread_PLUS_CASING_11R22_point_5_Columns["MAR"], 
                            Retread_PLUS_CASING_295_slash_75R22_point_5_Columns['MAR'],
                            Repair_Only_Columns["MAR"],
                            New_Tires_MAR_TOTAL, 
                            Bridgestone_MAR_TOTAL, 
                            Bridgestone_Light_Truck_Columns["MAR"], 
                            Bridgestone_225_slash_70R19_point_5_Columns["MAR"], 
                            Bridgestone_245_slash_70R17_point_5_Columns["MAR"], 
                            Bridgestone_9R22_point_5_Columns['MAR'], 
                            Bridgestone_295_slash_75R22_point_5_Columns['MAR'], 
                            Bridgestone_11R22_point_5_Columns['MAR'],
                            Bridgestone_OTHER_Columns['MAR'],
                            New_Tires_10_point_00_dash_20_Columns['MAR'], 
                            New_Tires_Yokohama_New_Tires_Columns['MAR'],
                            New_Tires_Continental_New_Tires_Columns['MAR'],
                            New_Tires_Hankook_New_Tires_Columns['MAR'],
                            Misc_Tires_Columns["MAR"], 
                            New_Tires_BS_STEER_Columns['MAR'], 
                            New_Tires_BS_DRIVE_Columns['MAR'], 
                            New_Tires_BS_TRAILER_Columns['MAR']]

    Sales_Table_df['APR'] = [Service_APR_TOTAL, 
                            Flat_Repair_Columns["APR"], 
                            Tire_Rotation_Columns['APR'],
                            Mount_and_Dismount_Columns["APR"], 
                            Service_Call_Hours_Columns["APR"], 
                            Scrap_Columns["APR"],
                            Bandag_Retread_APR_TOTAL, 
                            Retread_245_slash_70R17_point_5_Columns["APR"], 
                            Retread_295_slash_75R22_point_5_Columns['APR'], 
                            Retread_and_Casing_APR_TOTAL,
                            Retread_PLUS_CASING_10_point_00_dash_20_Columns["APR"], 
                            Retread_PLUS_CASING_11R22_point_5_Columns["APR"], 
                            Retread_PLUS_CASING_295_slash_75R22_point_5_Columns['APR'],
                            Repair_Only_Columns["APR"],
                            New_Tires_APR_TOTAL, 
                            Bridgestone_APR_TOTAL, 
                            Bridgestone_Light_Truck_Columns["APR"], 
                            Bridgestone_225_slash_70R19_point_5_Columns["APR"], 
                            Bridgestone_245_slash_70R17_point_5_Columns["APR"], 
                            Bridgestone_9R22_point_5_Columns['APR'], 
                            Bridgestone_295_slash_75R22_point_5_Columns['APR'], 
                            Bridgestone_OTHER_Columns['APR'],
                            Bridgestone_11R22_point_5_Columns['APR'],
                            New_Tires_10_point_00_dash_20_Columns['APR'], 
                            New_Tires_Yokohama_New_Tires_Columns['APR'], 
                            New_Tires_Continental_New_Tires_Columns['APR'],
                            New_Tires_Hankook_New_Tires_Columns['APR'],
                            Misc_Tires_Columns["APR"], 
                            New_Tires_BS_STEER_Columns['APR'], 
                            New_Tires_BS_DRIVE_Columns['APR'], 
                            New_Tires_BS_TRAILER_Columns['APR']]

    Sales_Table_df['MAY'] = [Service_MAY_TOTAL, 
                            Flat_Repair_Columns["MAY"], 
                            Tire_Rotation_Columns['MAY'],
                            Mount_and_Dismount_Columns["MAY"], 
                            Service_Call_Hours_Columns["MAY"], 
                            Scrap_Columns["MAY"],
                            Bandag_Retread_MAY_TOTAL, 
                            Retread_245_slash_70R17_point_5_Columns["MAY"], 
                            Retread_295_slash_75R22_point_5_Columns['MAY'], 
                            Retread_and_Casing_MAY_TOTAL,
                            Retread_PLUS_CASING_10_point_00_dash_20_Columns["MAY"], 
                            Retread_PLUS_CASING_11R22_point_5_Columns["MAY"], 
                            Retread_PLUS_CASING_295_slash_75R22_point_5_Columns['MAY'],
                            Repair_Only_Columns["MAY"],
                            New_Tires_MAY_TOTAL, 
                            Bridgestone_MAY_TOTAL, 
                            Bridgestone_Light_Truck_Columns["MAY"], 
                            Bridgestone_225_slash_70R19_point_5_Columns["MAY"], 
                            Bridgestone_245_slash_70R17_point_5_Columns["MAY"], 
                            Bridgestone_9R22_point_5_Columns['MAY'], 
                            Bridgestone_295_slash_75R22_point_5_Columns['MAY'], 
                            Bridgestone_11R22_point_5_Columns['MAY'],
                            Bridgestone_OTHER_Columns['MAY'],
                            New_Tires_10_point_00_dash_20_Columns['MAY'], 
                            New_Tires_Yokohama_New_Tires_Columns['MAY'], 
                            New_Tires_Continental_New_Tires_Columns['MAY'],
                            New_Tires_Hankook_New_Tires_Columns['MAY'],
                            Misc_Tires_Columns["MAY"], 
                            New_Tires_BS_STEER_Columns['MAY'], 
                            New_Tires_BS_DRIVE_Columns['MAY'], 
                            New_Tires_BS_TRAILER_Columns['MAY']]

    Sales_Table_df['JUN'] = [Service_JUN_TOTAL, 
                            Flat_Repair_Columns["JUN"], 
                            Tire_Rotation_Columns['JUN'],
                            Mount_and_Dismount_Columns["JUN"], 
                            Service_Call_Hours_Columns["JUN"], 
                            Scrap_Columns["JUN"],
                            Bandag_Retread_JUN_TOTAL, 
                            Retread_245_slash_70R17_point_5_Columns["JUN"], 
                            Retread_295_slash_75R22_point_5_Columns['JUN'], 
                            Retread_and_Casing_JUN_TOTAL,
                            Retread_PLUS_CASING_10_point_00_dash_20_Columns["JUN"], 
                            Retread_PLUS_CASING_11R22_point_5_Columns["JUN"], 
                            Retread_PLUS_CASING_295_slash_75R22_point_5_Columns['JUN'],
                            Repair_Only_Columns["JUN"],
                            New_Tires_JUN_TOTAL, 
                            Bridgestone_JUN_TOTAL, 
                            Bridgestone_Light_Truck_Columns["JUN"], 
                            Bridgestone_225_slash_70R19_point_5_Columns["JUN"], 
                            Bridgestone_245_slash_70R17_point_5_Columns["JUN"], 
                            Bridgestone_9R22_point_5_Columns['JUN'], 
                            Bridgestone_295_slash_75R22_point_5_Columns['JUN'], 
                            Bridgestone_11R22_point_5_Columns['JUN'],
                            Bridgestone_OTHER_Columns['JUN'],
                            New_Tires_10_point_00_dash_20_Columns['JUN'], 
                            New_Tires_Yokohama_New_Tires_Columns['JUN'], 
                            New_Tires_Continental_New_Tires_Columns['JUN'],
                            New_Tires_Hankook_New_Tires_Columns['JUN'],
                            Misc_Tires_Columns["JUN"], 
                            New_Tires_BS_STEER_Columns['JUN'], 
                            New_Tires_BS_DRIVE_Columns['JUN'], 
                            New_Tires_BS_TRAILER_Columns['JUN']]

    Sales_Table_df['JUL'] = [Service_JUL_TOTAL, 
                            Flat_Repair_Columns["JUL"], 
                            Tire_Rotation_Columns['JUL'],
                            Mount_and_Dismount_Columns["JUL"], 
                            Service_Call_Hours_Columns["JUL"], 
                            Scrap_Columns["JUL"],
                            Bandag_Retread_JUL_TOTAL, 
                            Retread_245_slash_70R17_point_5_Columns["JUL"], 
                            Retread_295_slash_75R22_point_5_Columns['JUL'], 
                            Retread_and_Casing_JUL_TOTAL,
                            Retread_PLUS_CASING_10_point_00_dash_20_Columns["JUL"], 
                            Retread_PLUS_CASING_11R22_point_5_Columns["JUL"], 
                            Retread_PLUS_CASING_295_slash_75R22_point_5_Columns['JUL'],
                            Repair_Only_Columns["JUL"],
                            New_Tires_JUL_TOTAL, 
                            Bridgestone_JUL_TOTAL, 
                            Bridgestone_Light_Truck_Columns["JUL"], 
                            Bridgestone_225_slash_70R19_point_5_Columns["JUL"], 
                            Bridgestone_245_slash_70R17_point_5_Columns["JUL"], 
                            Bridgestone_9R22_point_5_Columns['JUL'], 
                            Bridgestone_295_slash_75R22_point_5_Columns['JUL'], 
                            Bridgestone_11R22_point_5_Columns['JUL'],
                            Bridgestone_OTHER_Columns['JUL'],
                            New_Tires_10_point_00_dash_20_Columns['JUL'], 
                            New_Tires_Yokohama_New_Tires_Columns['JUL'], 
                            New_Tires_Continental_New_Tires_Columns['JUL'],
                            New_Tires_Hankook_New_Tires_Columns['JUL'],
                            Misc_Tires_Columns["JUL"], 
                            New_Tires_BS_STEER_Columns['JUL'], 
                            New_Tires_BS_DRIVE_Columns['JUL'], 
                            New_Tires_BS_TRAILER_Columns['JUL']]

    Sales_Table_df['AUG'] = [Service_AUG_TOTAL, 
                            Flat_Repair_Columns["AUG"], 
                            Tire_Rotation_Columns['AUG'],
                            Mount_and_Dismount_Columns["AUG"], 
                            Service_Call_Hours_Columns["AUG"], 
                            Scrap_Columns["AUG"],
                            Bandag_Retread_AUG_TOTAL, 
                            Retread_245_slash_70R17_point_5_Columns["AUG"], 
                            Retread_295_slash_75R22_point_5_Columns['AUG'], 
                            Retread_and_Casing_AUG_TOTAL,
                            Retread_PLUS_CASING_10_point_00_dash_20_Columns["AUG"], 
                            Retread_PLUS_CASING_11R22_point_5_Columns["AUG"], 
                            Retread_PLUS_CASING_295_slash_75R22_point_5_Columns['AUG'],
                            Repair_Only_Columns["AUG"],
                            New_Tires_AUG_TOTAL, 
                            Bridgestone_AUG_TOTAL, 
                            Bridgestone_Light_Truck_Columns["AUG"], 
                            Bridgestone_225_slash_70R19_point_5_Columns["AUG"], 
                            Bridgestone_245_slash_70R17_point_5_Columns["AUG"], 
                            Bridgestone_9R22_point_5_Columns['AUG'], 
                            Bridgestone_295_slash_75R22_point_5_Columns['AUG'], 
                            Bridgestone_11R22_point_5_Columns['AUG'],
                            Bridgestone_OTHER_Columns['AUG'],
                            New_Tires_10_point_00_dash_20_Columns['AUG'], 
                            New_Tires_Yokohama_New_Tires_Columns['AUG'], 
                            New_Tires_Continental_New_Tires_Columns['AUG'],
                            New_Tires_Hankook_New_Tires_Columns['AUG'],
                            Misc_Tires_Columns["AUG"], 
                            New_Tires_BS_STEER_Columns['AUG'], 
                            New_Tires_BS_DRIVE_Columns['AUG'], 
                            New_Tires_BS_TRAILER_Columns['AUG']]

    Sales_Table_df['SEPT'] = [Service_SEPT_TOTAL, 
                            Flat_Repair_Columns["SEPT"], 
                            Tire_Rotation_Columns['SEPT'],
                            Mount_and_Dismount_Columns["SEPT"], 
                            Service_Call_Hours_Columns["SEPT"], 
                            Scrap_Columns["SEPT"],
                            Bandag_Retread_SEPT_TOTAL, 
                            Retread_245_slash_70R17_point_5_Columns["SEPT"], 
                            Retread_295_slash_75R22_point_5_Columns['SEPT'], 
                            Retread_and_Casing_SEPT_TOTAL,
                            Retread_PLUS_CASING_10_point_00_dash_20_Columns["SEPT"], 
                            Retread_PLUS_CASING_11R22_point_5_Columns["SEPT"], 
                            Retread_PLUS_CASING_295_slash_75R22_point_5_Columns['SEPT'],
                            Repair_Only_Columns["SEPT"],
                            New_Tires_SEPT_TOTAL, 
                            Bridgestone_SEPT_TOTAL, 
                            Bridgestone_Light_Truck_Columns["SEPT"], 
                            Bridgestone_225_slash_70R19_point_5_Columns["SEPT"], 
                            Bridgestone_245_slash_70R17_point_5_Columns["SEPT"], 
                            Bridgestone_9R22_point_5_Columns['SEPT'], 
                            Bridgestone_295_slash_75R22_point_5_Columns['SEPT'], 
                            Bridgestone_11R22_point_5_Columns['SEPT'],
                            Bridgestone_OTHER_Columns['SEPT'],
                            New_Tires_10_point_00_dash_20_Columns['SEPT'], 
                            New_Tires_Yokohama_New_Tires_Columns['SEPT'], 
                            New_Tires_Continental_New_Tires_Columns['SEPT'],
                            New_Tires_Hankook_New_Tires_Columns['SEPT'],
                            Misc_Tires_Columns["SEPT"], 
                            New_Tires_BS_STEER_Columns['SEPT'], 
                            New_Tires_BS_DRIVE_Columns['SEPT'], 
                            New_Tires_BS_TRAILER_Columns['SEPT']]

    Sales_Table_df['OCT'] = [Service_OCT_TOTAL, 
                            Flat_Repair_Columns["OCT"], 
                            Tire_Rotation_Columns['OCT'],
                            Mount_and_Dismount_Columns["OCT"], 
                            Service_Call_Hours_Columns["OCT"], 
                            Scrap_Columns["OCT"],
                            Bandag_Retread_OCT_TOTAL, 
                            Retread_245_slash_70R17_point_5_Columns["OCT"], 
                            Retread_295_slash_75R22_point_5_Columns['OCT'], 
                            Retread_and_Casing_OCT_TOTAL,
                            Retread_PLUS_CASING_10_point_00_dash_20_Columns["OCT"], 
                            Retread_PLUS_CASING_11R22_point_5_Columns["OCT"], 
                            Retread_PLUS_CASING_295_slash_75R22_point_5_Columns['OCT'],
                            Repair_Only_Columns["OCT"],
                            New_Tires_OCT_TOTAL, 
                            Bridgestone_OCT_TOTAL, 
                            Bridgestone_Light_Truck_Columns["OCT"], 
                            Bridgestone_225_slash_70R19_point_5_Columns["OCT"], 
                            Bridgestone_245_slash_70R17_point_5_Columns["OCT"], 
                            Bridgestone_9R22_point_5_Columns['OCT'], 
                            Bridgestone_295_slash_75R22_point_5_Columns['OCT'], 
                            Bridgestone_11R22_point_5_Columns['OCT'],
                            Bridgestone_OTHER_Columns['OCT'],
                            New_Tires_10_point_00_dash_20_Columns['OCT'], 
                            New_Tires_Yokohama_New_Tires_Columns['OCT'], 
                            New_Tires_Continental_New_Tires_Columns['OCT'],
                            New_Tires_Hankook_New_Tires_Columns['OCT'],
                            Misc_Tires_Columns["OCT"], 
                            New_Tires_BS_STEER_Columns['OCT'], 
                            New_Tires_BS_DRIVE_Columns['OCT'], 
                            New_Tires_BS_TRAILER_Columns['OCT']]

    Sales_Table_df['NOV'] = [Service_NOV_TOTAL, 
                            Flat_Repair_Columns["NOV"], 
                            Tire_Rotation_Columns['NOV'],
                            Mount_and_Dismount_Columns["NOV"], 
                            Service_Call_Hours_Columns["NOV"], 
                            Scrap_Columns["NOV"],
                            Bandag_Retread_NOV_TOTAL, 
                            Retread_245_slash_70R17_point_5_Columns["NOV"], 
                            Retread_295_slash_75R22_point_5_Columns['NOV'], 
                            Retread_and_Casing_NOV_TOTAL,
                            Retread_PLUS_CASING_10_point_00_dash_20_Columns["NOV"], 
                            Retread_PLUS_CASING_11R22_point_5_Columns["NOV"], 
                            Retread_PLUS_CASING_295_slash_75R22_point_5_Columns['NOV'],
                            Repair_Only_Columns["NOV"],
                            New_Tires_NOV_TOTAL, 
                            Bridgestone_NOV_TOTAL, 
                            Bridgestone_Light_Truck_Columns["NOV"], 
                            Bridgestone_225_slash_70R19_point_5_Columns["NOV"], 
                            Bridgestone_245_slash_70R17_point_5_Columns["NOV"], 
                            Bridgestone_9R22_point_5_Columns['NOV'], 
                            Bridgestone_295_slash_75R22_point_5_Columns['NOV'], 
                            Bridgestone_11R22_point_5_Columns['NOV'],
                            Bridgestone_OTHER_Columns['NOV'],
                            New_Tires_10_point_00_dash_20_Columns['NOV'], 
                            New_Tires_Yokohama_New_Tires_Columns['NOV'], 
                            New_Tires_Continental_New_Tires_Columns['NOV'],
                            New_Tires_Hankook_New_Tires_Columns['NOV'],
                            Misc_Tires_Columns["NOV"], 
                            New_Tires_BS_STEER_Columns['NOV'], 
                            New_Tires_BS_DRIVE_Columns['NOV'], 
                            New_Tires_BS_TRAILER_Columns['NOV']]

    Sales_Table_df['DEC'] = [Service_DEC_TOTAL, 
                            Flat_Repair_Columns["DEC"], 
                            Tire_Rotation_Columns['DEC'],
                            Mount_and_Dismount_Columns["DEC"], 
                            Service_Call_Hours_Columns["DEC"], 
                            Scrap_Columns["DEC"],
                            Bandag_Retread_DEC_TOTAL, 
                            Retread_245_slash_70R17_point_5_Columns["DEC"], 
                            Retread_295_slash_75R22_point_5_Columns['DEC'], 
                            Retread_and_Casing_DEC_TOTAL,
                            Retread_PLUS_CASING_10_point_00_dash_20_Columns["DEC"], 
                            Retread_PLUS_CASING_11R22_point_5_Columns["DEC"], 
                            Retread_PLUS_CASING_295_slash_75R22_point_5_Columns['DEC'],
                            Repair_Only_Columns["DEC"],
                            New_Tires_DEC_TOTAL, 
                            Bridgestone_DEC_TOTAL, 
                            Bridgestone_Light_Truck_Columns["DEC"], 
                            Bridgestone_225_slash_70R19_point_5_Columns["DEC"], 
                            Bridgestone_245_slash_70R17_point_5_Columns["DEC"], 
                            Bridgestone_9R22_point_5_Columns['DEC'], 
                            Bridgestone_295_slash_75R22_point_5_Columns['DEC'], 
                            Bridgestone_11R22_point_5_Columns['DEC'],
                            Bridgestone_OTHER_Columns['DEC'],
                            New_Tires_10_point_00_dash_20_Columns['DEC'], 
                            New_Tires_Yokohama_New_Tires_Columns['DEC'], 
                            New_Tires_Continental_New_Tires_Columns['DEC'],
                            New_Tires_Hankook_New_Tires_Columns['DEC'],
                            Misc_Tires_Columns["DEC"], 
                            New_Tires_BS_STEER_Columns['DEC'], 
                            New_Tires_BS_DRIVE_Columns['DEC'], 
                            New_Tires_BS_TRAILER_Columns['DEC']]


    # Save last and current year's tables:
    if year == Last_Year:
        Sales_Table_df.to_csv(f'CSVs/Sales_Table_df_{Last_Year}.csv', index=False)
    elif year == Current_Year:
        Sales_Table_df.to_csv(f'CSVs/Sales_Table_df_{Current_Year}.csv', index=False)

-------------------------------------------
year: 2020
Months_Numerical:  [202001, 202002, 202003, 202004, 202005, 202006, 202007, 202008, 202009, 202010, 202011, 202012]


In [ ]:
Last_Year_df = pd.read_csv(f'CSVs/Sales_Table_df_{Last_Year}.csv')
Last_Year_df

In [ ]:
Current_Year_df = pd.read_csv(f'CSVs/Sales_Table_df_{Current_Year}.csv')
Current_Year_df

# Create the HTML Sales Table:

In [ ]:
Sales_df_for_html = Sales_Table_df.copy()

for columns in Sales_df_for_html.iloc[:,1:].columns:
    Sales_df_for_html[columns] = Sales_df_for_html[columns].astype(float)
    Sales_df_for_html[columns] = Sales_df_for_html.apply(lambda x: "{:,.0f}".format(x[columns]), axis=1)

Sales_df_for_html

## Now add dashes to month values where time doesn't exist yet:

In [ ]:
# ----------NOW ADD DASHES TO ANY MONTH THAT DOESN'T EXIST IN TIME YET---------------------------------------------------

Sales_df_for_html_Transposed = Sales_df_for_html.T
Sales_df_for_html_Transposed = Sales_df_for_html_Transposed.reset_index()

# Recall each month name here represents a numerical date like 202101 is January 2021
List_of_Dates = [0, 0, Jan, Feb, Mar, Apr,
                    May, Jun, Jul, Aug, Sept, Oct, Nov, Dec]
Sales_df_for_html_Transposed_EXTRA_COLUMN = Sales_df_for_html_Transposed
# Create a new column that has the values from the list above
Sales_df_for_html_Transposed_EXTRA_COLUMN['Date'] = List_of_Dates
Sales_df_for_html_Transposed_EXTRA_COLUMN

# ---------------------------------------------------------------------------------
# This section will filx YTD to NOT include the current month IF the current month is in the df according to the month variables we assigned at the top of the notebook and, accordingly, the new "Date" column in the df.  In other words, this section will only be used if we are testing for the current year and it allows the test to be able to run smoothly for any year:
if Current_Month in Sales_df_for_html_Transposed_EXTRA_COLUMN.Date:
    # Assign a variable to the row where the date column is equal to the current month:
    df_of_row_for_current_month = Sales_df_for_html_Transposed_EXTRA_COLUMN.loc[
        Sales_df_for_html_Transposed_EXTRA_COLUMN['Date'] == Current_Month]
    # Assign a variable to filter the df to omit the first column which is a string for the month name.  We just want the numbers from current month (so we can subtract it from YTD if the current month is in the df since we don't want to include it in the YTD sum)
    df_current_month_row_numbers = df_of_row_for_current_month.iloc[0, 1:]
    # Fix the YTD row numbers (so they don't include this month in the calc) by subtracting the current month's values from the YTD's values:
    Sales_df_for_html_Transposed_EXTRA_COLUMN.iloc[1:2, 1:-
                                            1] = Sales_df_for_html_Transposed_EXTRA_COLUMN.iloc[1:2, 1:-1] - df_current_month_row_numbers
# ----------------------------------------------------------------------------------

# Replace all zeros in all the df that occured before the current numerical month with dashes... except don't change the column called "index" which has the month names
Sales_df_for_html_Transposed_EXTRA_COLUMN.loc[Sales_df_for_html_Transposed_EXTRA_COLUMN["Date"]
                                    >= Current_Month, Sales_df_for_html_Transposed_EXTRA_COLUMN.columns != 'index'] = '-'
# Drop the date column:
df_dropped_date_column = Sales_df_for_html_Transposed_EXTRA_COLUMN.drop(columns=[
    'Date'])
df_dropped_date_column

# Retranspose the df back to normal (which will now have dashes rather than zeros for all months before the current month)
df_DASHES_for_HTML = df_dropped_date_column.T
df_DASHES_for_HTML = df_DASHES_for_HTML.iloc[1:, :]
df_DASHES_for_HTML = df_DASHES_for_HTML.rename(
    columns={0: 'Category', 1: 'YTD', 2: 'JAN', 3: 'FEB', 4: 'MAR', 5: 'APR', 6: 'MAY', 7: 'JUN', 8: 'JUL', 9: 'AUG', 10: 'SEPT', 11: 'OCT', 12: 'NOV', 13: 'DEC'})
df_DASHES_for_HTML

In [ ]:
# Although we named lots of long names in the Category column to help distinguish them in grpahing etc. We'll rename them the shorter ones for HTML to display in the report.  We'll use the replace method to do this.

df_DASHES_for_HTML['Category'] = df_DASHES_for_HTML['Category'].replace(['245/70R17.5 RETREAD', '295/75R22.5 RETREAD', '10.00-20 CAP & CASING', '11R22.5 CAP & CASING', '295/75R22.5 CAP & CASING', 'Light Truck BRIDGESTONE', '225/70R19.5 BRIDGESTONE', '245/70R17.5 BRIDGESTONE', '9R22.5 BRIDGESTONE', '295/75R22.5 BRIDGESTONE', '11R22.5 BRIDGESTONE', 'OTHER BRIDGESTONE'], ['245/70R17.5', '295/75R22.5', '10.00-20', '11R22.5', '295/75R22.5', 'Light Truck', '225/70R19.5', '245/70R17.5', '9R22.5', '295/75R22.5', '11R22.5', 'OTHER'])

df_DASHES_for_HTML

In [ ]:

# For the html table only, we'll get rid of the name "Category" as the first column since we want that part blank in the final pdf table:
df_just_for_html = df_DASHES_for_HTML.rename(
    columns={'Category': ''})
# If we don't want the "SERVICE" category to have totals, we can replace all the totals with empty strings:
df_just_for_html.iloc[0, 1:] = ''

# BUILD THE HTML TABLE for the MAIN TABLE:

def create_html_table(Primary_table):

    # Create headers for HTML
    row_data = '<!-- Start Sales Table --><thead>\n<tr class="header">'

    # Create columns for HTML:    
    for column in range(Primary_table.shape[1]):
        adjust_header_css = "header"
        if column in [0,1]:
            adjust_header_css += " make_white"
        if column not in [0, 1]:
            adjust_header_css += " lonestar_gold_color"
        if column not in []:
            adjust_header_css += " center"    
        row_data += '\n <th class = "'+adjust_header_css+'">'+df_just_for_html.columns[column]+'</th>'
    row_data += '\n</tr> \n </head> \n <tbody>'
    
    # Create Body (rows for table) for html
    for row in range(Primary_table.shape[0]):
        if row in [0, 6, 9, 13, 14, 28, 29, 30]:
            row_data += '\n<tr class = "lonestar_gold_color make_bold">'
        else: row_data += '\n<tr>'
        for column in range(Primary_table.shape[1]):
            td_class_name = 'text_column'
            if column in [0] and row in [1, 2, 3, 4, 5, 7, 8, 10, 11, 12, 16, 17, 18, 19, 20, 21, 22, 28, 29, 30]:
                td_class_name += " right_align"
            if column not in [0]:
                td_class_name += " center"
            if column in [0] and row in [0, 6, 9, 13, 14, 15, 23, 24, 25, 26, 27]:
                td_class_name += " make_bold left_align"
            if column not in [0] and row in [14, 15]:
                td_class_name += " lonestar_gold_color"
            if column in [0] and row in [28, 29, 30]:
                td_class_name += " make_bold right_align"
            if column not in [] and row in [15]:
                td_class_name += " make_bold"            
            row_data += '\n <td class = "'+td_class_name+'">'+str(Primary_table.iloc[row,column])+'</td>'
    row_data += '\n</tr> \n</tbody><!-- End Sales Table -->'
    # -------------------------------------------------------------------------------------------------

    return row_data

# Call the function:
html_table = create_html_table(df_just_for_html)

# Create Current Year Percentage Sales Bar Chart:

In [ ]:
Total_of_Categories = Bandag_Retread_YTD_TOTAL + Retread_and_Casing_YTD_TOTAL + Repair_Only_Columns["YTD"] + New_Tires_YTD_TOTAL

if Bandag_Retread_YTD_TOTAL > 0:
    Retreads_percentage = round(100 * (Bandag_Retread_YTD_TOTAL/Total_of_Categories), 2)
    Retreads_plus_Casings_percentage = round(100 * (Retread_and_Casing_YTD_TOTAL/Total_of_Categories), 2)
    Repair_Only_percentage = round(100 * (Repair_Only_Columns["YTD"]/Total_of_Categories), 2)
    New_Tires_percentage = round(100 * (New_Tires_YTD_TOTAL/Total_of_Categories), 2)

else:
    Retreads_percentage = 0
    Retreads_plus_Casings_percentage = 0
    Repair_Only_percentage = 0
    New_Tires_percentage = 0

In [ ]:
import plotly.graph_objects as go

Categories = ['RETREADS', 'RETREAD + CASING', 'REPAIR ONLY', 'NEW TIRES']

# Create blank figure
fig1 = go.Figure()

# Add bar for RETREADS:
if Retreads_percentage >= 1:
    fig1.add_trace(go.Bar(
        x=['RETREADS'],
        y=[round(Retreads_percentage, 0)],
        name='Retreads', 
        marker_color = 'black'))

else: 
    fig1.add_trace(go.Bar(
        x=['RETREADS'],
        y=[round(Retreads_percentage, 1)],
        name='Retreads', 
        marker_color = 'black'))

# Add Bar for RETREAD + CASINGS
if Retreads_plus_Casings_percentage >= 1:
    fig1.add_trace(go.Bar(
        x=['RETREAD + CASING'],
        y=[round(Retreads_plus_Casings_percentage, 0)],
        name='Retread + Casing',
        marker_color='dimgray'))
else:
    fig1.add_trace(go.Bar(
    x=['RETREAD + CASING'],
    y=[round(Retreads_plus_Casings_percentage, 1)],
    name='Retread + Casing',
    marker_color='dimgray'))

# Add Bar for REPAIR ONLY
if Repair_Only_percentage >= 1:
    fig1.add_trace(go.Bar(
        x=['REPAIR ONLY'],
        y=[round(Repair_Only_percentage, 0)],
        name='Repair Only',
        marker_color='royalblue'))
else:
    fig1.add_trace(go.Bar(
        x=['REPAIR ONLY'],
        y=[round(Repair_Only_percentage, 1)],
        name='Repair Only',
        marker_color='royalblue'))

# Add Bar for NEW TIRES
if New_Tires_percentage >= 1 and New_Tires_percentage <= 99:
    fig1.add_trace(go.Bar(
        x=['NEW TIRES'],
        y=[round(New_Tires_percentage, 0)],
        name='New Tires',
        marker_color='#caa74d'))
else:
    fig1.add_trace(go.Bar(
        x=['NEW TIRES'],
        y=[round(New_Tires_percentage, 1)],
        name='New Tires',
        marker_color='#caa74d'))


# Change grid color and axis colors
fig1.update_yaxes(gridwidth=2, gridcolor='white')

fig1.update_traces(texttemplate='%{y:.s}%', textposition= 'auto', textfont = {'size': [27]} )

# Here let's setup fig2 = fig1 so that we can use fig2 as a different size JUST for the website.  We'll use fig1 for the FIRST index.html to generate the pdf.
# Since the website needs smaller graphs, we'll need to do this last part twice JUST to change the size amounts.


# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig1.update_layout(
    barmode='group', 
    title_text=f"<b>YTD {Current_Year} - Purchases Percentage Breakdown<b>", 
    title_font_size=38,
    title_font_color="#351C15",
    title_x = 0.5,
    xaxis_title="Purchase Categories",
    yaxis_title="Percentage",
    autosize=False, width=1445, height=900,
    # xaxis_tickangle=-45,
    showlegend=False, 
    font=dict(
            family=None,
            size=25,
            color="black"
        ))



# Write the image that we'll use for index.html to generate the pdf:
fig1.write_image(f"Images_graphs/{Current_Year}_Sales_Percentage_Breakdown.svg")
fig1.show()

# NOW write fig2's size to be used for plotly interactive in the website afterwards
# PLOTLY INTERACTIVE:
fig2 = fig1
fig1.update_traces(texttemplate='%{y:.s}%', textposition= 'auto', textfont = {'size': [20]} )
# NOW do the same thing but with different image sizes for fig2 (for the website)!!!
fig2.update_layout(
    barmode='group', 
    title_text=f"<b>YTD {Current_Year} - Purchases Percentage Breakdown<b>", 
    title_font_size=25,
    title_font_color="#351C15",
    title_x = 0.5,
    xaxis_title="Purchase Categories",
    yaxis_title="Percentage",
    autosize=False, width=900, height=650,
    # xaxis_tickangle=-45,
    showlegend=False, 
    font=dict(
            family=None,
            size=16,
            color="black"
        ))


purchase_percentage_breakdown = fig2
Lonestar_Purchase_Percentage_Breakdown_url = py.plot(purchase_percentage_breakdown, filename = 'Lonestar_Purchase_Percentage_Breakdown', auto_open=False)


fig2.show()

# Now Make a Percentage Breakdown for Retreads bar chart:

In [ ]:
if Bandag_Retread_YTD_TOTAL > 0:
    Retread_245_percentage = round(100 * (Retread_245_slash_70R17_point_5_Columns["YTD"]/Bandag_Retread_YTD_TOTAL), 2)
    Retread_295_percentage = round(100 * (Retread_295_slash_75R22_point_5_Columns['YTD']/Bandag_Retread_YTD_TOTAL), 2)
else:
    Retread_245_percentage = 0
    Retread_295_percentage = 0

In [ ]:
import plotly.graph_objects as go

Categories = ['245/70R17.5', '295/70R22.5']

# Create blank figure
fig1 = go.Figure()

# Add bar for 245/70R17.5 RETREADS:
if Retread_245_percentage >= 1:
    fig1.add_trace(go.Bar(
        x=['245/70R17.5'],
        y=[round(Retread_245_percentage, 0)],
        name='245/70R17.5', 
        marker_color = 'black'))
else: 
    fig1.add_trace(go.Bar(
        x=['245/70R17.5'],
        y=[round(Retread_245_percentage, 1)],
        name='245/70R17.5', 
        marker_color = 'black'))

# Add bar for 295/70R22.5 RETREADS:
if Retread_295_percentage >= 1:
    fig1.add_trace(go.Bar(
        x=['295/70R22.5'],
        y=[round(Retread_295_percentage, 0)],
        name='295/70R22.5', 
        marker_color = '#caa74d'))
else: 
    fig1.add_trace(go.Bar(
        x=['295/70R22.5'],
        y=[round(Retread_295_percentage, 1)],
        name='295/70R22.5', 
        marker_color = '#caa74d'))


fig1.update_traces(texttemplate='%{y:.s}%', textposition= 'auto', textfont = {'size': [32]} )

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig1.update_layout(
    barmode='group', 
    title_text=f"<b>YTD {Current_Year} - Retreads Percentage Breakdown<b>", 
    title_font_size=38,
    title_font_color="#351C15",
    title_x = 0.5,
    xaxis_title="Tire Size",
    yaxis_title="Percentage",
    autosize=False, width=1445, height=900,
    # xaxis_tickangle=-45,
    showlegend=False, 
    font=dict(
            family=None,
            size=29,
            color="black"
        ))

# Change grid color and axis colors
fig1.update_yaxes(gridwidth=2, gridcolor='white')


fig1.write_image(f"Images_graphs/{Current_Year}_Retreads_Percentage_Breakdown.svg")
fig1.show()

# Create 3 Trend Line charts:

In [ ]:
Current_Year_df_Transposed = Current_Year_df.T
Current_Year_df_Transposed = Current_Year_df_Transposed.reset_index()

# Recall each month name here represents a numerical date like 202101 is January 2021
List_of_Dates = [0, 0, Jan, Feb, Mar, Apr,
                    May, Jun, Jul, Aug, Sept, Oct, Nov, Dec]
Current_Year_df_Transposed_EXTRA_COLUMN = Current_Year_df_Transposed
# Create a new column that has the values from the list above
Current_Year_df_Transposed_EXTRA_COLUMN['Date'] = List_of_Dates
Current_Year_df_Transposed_EXTRA_COLUMN

# ---------------------------------------------------------------------------------
# This section will filx YTD to NOT include the current month IF the current month is in the df according to the month variables we assigned at the top of the notebook and, accordingly, the new "Date" column in the df.  In other words, this section will only be used if we are testing for the current year and it allows the test to be able to run smoothly for any year:
if Current_Month in Current_Year_df_Transposed_EXTRA_COLUMN.Date:
    # Assign a variable to the row where the date column is equal to the current month:
    df_of_row_for_current_month = Current_Year_df_Transposed_EXTRA_COLUMN.loc[Current_Year_df_Transposed_EXTRA_COLUMN['Date'] == Current_Month]
    # Assign a variable to filter the df to omit the first column which is a string for the month name.  We just want the numbers from current month (so we can subtract it from YTD if the current month is in the df since we don't want to include it in the YTD sum)
    df_current_month_row_numbers = df_of_row_for_current_month.iloc[0, 1:]
    # Fix the YTD row numbers (so they don't include this month in the calc) by subtracting the current month's values from the YTD's values:
    Current_Year_df_Transposed_EXTRA_COLUMN.iloc[1:2, 1:-1] = Current_Year_df_Transposed_EXTRA_COLUMN.iloc[1:2, 1:-1] - df_current_month_row_numbers
# ----------------------------------------------------------------------------------

# Replace all zeros in all the df that occured before the current numerical month with dashes... except don't change the column called "index" which has the month names
Current_Year_df_Transposed_EXTRA_COLUMN.loc[Current_Year_df_Transposed_EXTRA_COLUMN["Date"]
                                    >= Current_Month, Current_Year_df_Transposed_EXTRA_COLUMN.columns != 'index'] = '-'
# Drop the date column:
df_Current_Year_dropped_date_column = Current_Year_df_Transposed_EXTRA_COLUMN.drop(columns=[
    'Date'])
df_Current_Year_dropped_date_column

# Retranspose the df back to normal (which will now have dashes rather than zeros for all months before the current month)
df_Current_Year_DASHES = df_Current_Year_dropped_date_column.T
df_Current_Year_DASHES = df_Current_Year_DASHES.iloc[1:, :]
df_Current_Year_DASHES = df_Current_Year_DASHES.rename(
    columns={0: 'Category', 1: 'YTD', 2: 'JAN', 3: 'FEB', 4: 'MAR', 5: 'APR', 6: 'MAY', 7: 'JUN', 8: 'JUL', 9: 'AUG', 10: 'SEPT', 11: 'OCT', 12: 'NOV', 13: 'DEC'})
df_Current_Year_DASHES

In [ ]:
# Drop the YTD column from each df to prepare it for graphing.
Last_Year_df = Last_Year_df.drop(columns=['YTD'])
Current_Year_df = df_Current_Year_DASHES.drop(columns=['YTD'])

Current_Year_df

In [ ]:
# Rename the columns on Each Year's data so it shows not only the month but also the year:
# Last Year:
Last_Year_df.columns = ['Category', f'JAN {Last_Year}', f'FEB {Last_Year}', f'MAR {Last_Year}', f'APR {Last_Year}', f'MAY {Last_Year}',
                                f'JUN {Last_Year}', f'JUL {Last_Year}', f'AUG {Last_Year}', f'SEPT {Last_Year}', f'OCT {Last_Year}', f'NOV {Last_Year}', f'DEC {Last_Year}']
# Current Year:
Current_Year_df.columns = ['Category', f'JAN {Current_Year}', f'FEB {Current_Year}', f'MAR {Current_Year}', f'APR {Current_Year}', f'MAY {Current_Year}',
                                f'JUN {Current_Year}', f'JUL {Current_Year}', f'AUG {Current_Year}', f'SEPT {Current_Year}', f'OCT {Current_Year}', f'NOV {Current_Year}', f'DEC {Current_Year}']

# For the Current Year, drop columns with dashes:
Current_Year_df = Current_Year_df.loc[:, ~(Current_Year_df.astype(str) == '-').all()]
# To merge, we ONLY want "Category" on ONE df.  We'll drop "Category" on ALL dfs that are not the oldest year we're testing.
Current_Year_df = Current_Year_df.drop(columns=['Category'])
# Now add a new TEMPORARY column for ALL dfs we're testing which will be ALL unique values but identical for each df, so we can merge "on" that temporary column
Last_Year_df['Temp Col'] = range(len(Last_Year_df))#
Current_Year_df['Temp Col'] = range(len(Current_Year_df))
# Merge each year together into one DF:
Merged_Years_df = pd.merge(Last_Year_df, Current_Year_df, how='outer', on='Temp Col')
# Now delete the "Temp Col" from the merged df:
Merged_Years_df = Merged_Years_df.drop(columns=['Temp Col'])

# For last year's df, create individual dfs for each category we want to graph (Services, Bandag Retreads, and Bridgestone):
Merged_Service_df = Merged_Years_df.iloc[1:6, :].reset_index(drop=True)
Merged_Bandag_Retread_df = Merged_Years_df.iloc[7:9, :].reset_index(drop=True)
Merged_Bridgestone_df = Merged_Years_df.iloc[16:22, :].reset_index(drop=True)

Merged_Bridgestone_df

## 1. Service Trend Line Graph

In [ ]:
# Transpose the Merged_Service_df to make it easier for graphing:
Service_df_TRANSPOSED = Merged_Service_df.T

Description_list_from_first_row = list(Service_df_TRANSPOSED.iloc[0])
Service_df_TRANSPOSED = Service_df_TRANSPOSED.iloc[1:, :]
Service_df_TRANSPOSED.columns = Description_list_from_first_row

Service_df_TRANSPOSED = Service_df_TRANSPOSED.reset_index()
Service_df_TRANSPOSED = Service_df_TRANSPOSED.rename(columns={'index': 'month-yr'})

Service_df_TRANSPOSED

In [ ]:

# Setup independent X Variables (which is just the month and save a list):
Month_LIST = Service_df_TRANSPOSED['month-yr'].to_list()

# Setup a dependent Y Variables (By Creating separate lists for each type of service):
Flat_Repair_Values = Service_df_TRANSPOSED['Flat Repair'].to_list()

Tire_Rotation_Values = Service_df_TRANSPOSED["Tire Rotation"].tolist()

Mount_and_Dismount_Values = Service_df_TRANSPOSED["Mount and Dismount"].to_list()

Service_Call_Hours_Values = Service_df_TRANSPOSED["Service Call Hours"].to_list()

Scrap_Values = Service_df_TRANSPOSED["Scrap"].to_list()

fig = go.Figure()
# Add a light blue filter over last year's graph to distinguish it from this year:
fig.add_vrect(x0=0, x1=12,
                annotation_text=" Last Year ", annotation_position="top left",
                annotation=dict(
    font_size=27, font_family="Times New Roman"),
    fillcolor=None, opacity=0.25, line_width=0)
# Optionally add a filter color for this yr too, lightgreen works (or choose "white" or comment out this block for none)
fig.add_vrect(x0=12, x1=12,
                annotation_text=" Current Year ", annotation_position="top left",
                annotation=dict(
    font_size=27, font_family="Times New Roman"),
    fillcolor=None, opacity=0.15, line_width=0)

# Add a green vertical line the graph to separate last year from the current one:
fig.add_vline(x=11.5, line_width=3.5,
                line_dash="dash", line_color="green")

# Create and Style Traces:
fig.add_trace(go.Scatter(x=Month_LIST, y=Flat_Repair_Values, name='Flat Repair',
                            line=dict(color='purple', width=2)))
fig.add_trace(go.Scatter(x=Month_LIST, y=Tire_Rotation_Values, name='Tire Rotation',
                            line=dict(color='#FFB500', width=2)))
fig.add_trace(go.Scatter(x=Month_LIST, y=Mount_and_Dismount_Values, name='Mount and Dismount',
                            line=dict(color='royalblue', width=2)))
fig.add_trace(go.Scatter(x=Month_LIST, y=Service_Call_Hours_Values, name='Service Call Hours',
                            line=dict(color='red', width=2)))
fig.add_trace(go.Scatter(x=Month_LIST, y=Scrap_Values, name='Scrap',
                            line=dict(color='#351C15', width=2)))

# Edit the Layout:
fig.update_layout(
    title_text=f"<b>Service Purchases Trend {Last_Year} - {Current_Year}</b>",
    title_x=0.5,
    title_font_size=24,
    title_font_color="#351C15",
    xaxis_title="Month",
    yaxis_title='Services',
    autosize=False, width=1350, height=675,
    showlegend=True,
    legend=dict(
        x=1,
        y=1,
        title_font_family="Times New Roman",
        font=dict(
            family=None,
            size=15,
            color="black"
        ),
        bgcolor="white",
        bordercolor="dimgray",
        borderwidth=.05
    ),
    # General axes and axes labels fonts:
    font=dict(
        family=None,
        size=16,
        color="black"
    ))


# Add a green vertical line the graph to separate last year from the current one:
fig.add_vline(x=11.5, line_width=3.5,
                line_dash="dash", line_color="green")

# Change grid color and axis colors
fig.update_xaxes(gridwidth=1, gridcolor='white')
fig.update_xaxes(gridwidth=1, gridcolor='white')

# Save the file to use in HTML, PDF, etc.
fig.write_image(f"Images_graphs/{Current_Year}_Service_Purchase_Analysis.svg")
fig.show()









# NOW.... we have to do all that again to create the same graph but different size for our interactive webpage version:
fig2 = go.Figure()
# Add a light blue filter over last year's graph to distinguish it from this year:
fig2.add_vrect(x0=0, x1=12,
                annotation_text=" Last Year ", annotation_position="top left",
                annotation=dict(
    font_size=23, font_family="Times New Roman"),
    fillcolor=None, opacity=0.25, line_width=0)
# Optionally add a filter color for this yr too, lightgreen works (or choose "white" or comment out this block for none)
fig2.add_vrect(x0=12, x1=12,
                annotation_text=" Current Year ", annotation_position="top left",
                annotation=dict(
    font_size=23, font_family="Times New Roman"),
    fillcolor=None, opacity=0.15, line_width=0)

# Add a green vertical line the graph to separate last year from the current one:
fig2.add_vline(x=11.5, line_width=3.5,
                line_dash="dash", line_color="green")

# Create and Style Traces:
fig2.add_trace(go.Scatter(x=Month_LIST, y=Flat_Repair_Values, name='Flat Repair',
                            line=dict(color='purple', width=2)))
fig2.add_trace(go.Scatter(x=Month_LIST, y=Tire_Rotation_Values, name='Tire Rotation',
                            line=dict(color='#FFB500', width=2)))
fig2.add_trace(go.Scatter(x=Month_LIST, y=Mount_and_Dismount_Values, name='Mount and Dismount',
                            line=dict(color='royalblue', width=2)))
fig2.add_trace(go.Scatter(x=Month_LIST, y=Service_Call_Hours_Values, name='Service Call Hours',
                            line=dict(color='red', width=2)))
fig2.add_trace(go.Scatter(x=Month_LIST, y=Scrap_Values, name='Scrap',
                            line=dict(color='#351C15', width=2)))

# Edit the Layout:
fig2.update_layout(
    title_text=f"<b>Service Purchases Trend {Last_Year} - {Current_Year}</b>",
    title_x=0.5,
    title_font_size=20,
    title_font_color="#351C15",
    xaxis_title="Month",
    yaxis_title='Services',
    autosize=False, width=1050, height=525,
    showlegend=True,
    legend=dict(
        x=1,
        y=1,
        title_font_family="Times New Roman",
        font=dict(
            family=None,
            size=13,
            color="black"
        ),
        bgcolor="white",
        bordercolor="dimgray",
        borderwidth=.05
    ),
    # General axes and axes labels fonts:
    font=dict(
        family=None,
        size=13,
        color="black"
    ))


# Add a green vertical line the graph to separate last year from the current one:
fig2.add_vline(x=11.5, line_width=3.5,
                line_dash="dash", line_color="green")



# PLOTLY INTERACTIVE:
Service_Purchases_Trend_graph = fig2
Service_Purchases_Trend_graph_for_iframe = py.plot(Service_Purchases_Trend_graph, filename = 'Service_Purchases_Trend_graph', auto_open=False)
Service_Purchases_Trend_graph_for_iframe


## 2. Bandag Retread Purchase Analysis Line Chart:

In [ ]:
# Transpose the Merged_Bandag_Retread_df to make it easier for graphing:
Merged_Bandag_Retread_df_TRANSPOSED = Merged_Bandag_Retread_df.T

Description_list_from_first_row = list(Merged_Bandag_Retread_df_TRANSPOSED.iloc[0])
Merged_Bandag_Retread_df_TRANSPOSED = Merged_Bandag_Retread_df_TRANSPOSED.iloc[1:, :]
Merged_Bandag_Retread_df_TRANSPOSED.columns = Description_list_from_first_row

Merged_Bandag_Retread_df_TRANSPOSED = Merged_Bandag_Retread_df_TRANSPOSED.reset_index()
Merged_Bandag_Retread_df_TRANSPOSED = Merged_Bandag_Retread_df_TRANSPOSED.rename(columns={'index': 'month-yr'})

Merged_Bandag_Retread_df_TRANSPOSED

In [ ]:
# Create Bandag Purchase Analysis Graph:

# Setup independent X Variables (which is just the month and save a list):
Month_LIST2 = Merged_Bandag_Retread_df_TRANSPOSED['month-yr'].to_list()

# Setup a dependent Y Variables (By Creating separate lists for each type of Bandag Retread):
Two_Forty_Five_LIST = Merged_Bandag_Retread_df_TRANSPOSED['245/70R17.5 RETREAD'].to_list()
Two_Ninety_Five_LIST = Merged_Bandag_Retread_df_TRANSPOSED['295/75R22.5 RETREAD'].to_list()


fig = go.Figure()
# Add a filter over last year's graph to distinguish it from this year:
fig.add_vrect(x0=0, x1=12,
                annotation_text=" Last Year ", annotation_position="top left",
                annotation=dict(
    font_size=27, font_family="Times New Roman"),
    fillcolor=None, opacity=0.25, line_width=0)

# Optionally add a filter color for this yr too, lightgreen works (or choose "white" or comment out this block for none)
fig.add_vrect(x0=12, x1=12,
                annotation_text=" Current Year ", annotation_position="top left",
                annotation=dict(
    font_size=27, font_family="Times New Roman"),
    fillcolor=None, opacity=0.15, line_width=0)

# Create and Style Traces:
fig.add_trace(go.Scatter(x=Month_LIST2, y=Two_Forty_Five_LIST, name='245/70R17.5',
                            line=dict(color='black', width=2.5)))
fig.add_trace(go.Scatter(x=Month_LIST2, y=Two_Ninety_Five_LIST, name='295/75R22.5',
                            line=dict(color='#caa74d', width=2.5)))

# HERE IS A GREAT Hack TO MAKE SURE ALL THE MONTHS SHOW UP!!!!  ADD A WHITE LINE THAT HAS all zeros and name it as a blank string so the legend doesn't show it!
# fig.add_trace(go.Scatter(x=Month_LIST2, y=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], name='',
#                          line=dict(color='white', width=1)))
# Edit the Layout:
fig.update_layout(
    title_text=f"<b>Bandag Retread Purchases Trend {Last_Year} - {Current_Year}</b>",
    title_x=.5,
    title_font_size=24,
    title_font_color="#351C15",
    xaxis_title='Month',
    yaxis_title='Purchases',
    autosize=False, width=1350, height=675,
    showlegend=True,
    legend=dict(
        x=1,
        y=1,
        title_font_family="Times New Roman",
        font=dict(
            family=None,
            size=15,
            color="black"
        ),
        bgcolor="white",
        bordercolor="dimgray",
        borderwidth=.05
    ),
    # General axes and axes labels fonts:
    font=dict(
        family=None,
        size=16,
        color="black"
    ))

# Add a green vertical line the graph to separate last year from the current one:
fig.add_vline(x=11.5, line_width=3.5,
                line_dash="dash", line_color="green")


# Change grid color and axis colors
fig.update_xaxes(gridwidth=1, gridcolor='white')
fig.update_xaxes(gridwidth=1, gridcolor='white')

# Save the file to use in HTML, PDF, etc.
fig.write_image(f"Images_graphs/{Current_Year}_Bandag_Purchase_Analysis.svg")
fig.show()





# ============================================================================================================================================
# AGAIN, we'll need to create a graph for the webpage version as well that will be different sizes and interactive with plotly:
# ============================================================================================================================================
fig2 = go.Figure()
# Add a filter over last year's graph to distinguish it from this year:
fig2.add_vrect(x0=0, x1=12,
                annotation_text=" Last Year ", annotation_position="top left",
                annotation=dict(
    font_size=20, font_family="Times New Roman"),
    fillcolor=None, opacity=0.25, line_width=0)

# Optionally add a filter color for this yr too, lightgreen works (or choose "white" or comment out this block for none)
fig2.add_vrect(x0=12, x1=12,
                annotation_text=" Current Year ", annotation_position="top left",
                annotation=dict(
    font_size=20, font_family="Times New Roman"),
    fillcolor=None, opacity=0.15, line_width=0)

# Create and Style Traces:
fig2.add_trace(go.Scatter(x=Month_LIST2, y=Two_Forty_Five_LIST, name='245/70R17.5',
                            line=dict(color='black', width=2)))
fig2.add_trace(go.Scatter(x=Month_LIST2, y=Two_Ninety_Five_LIST, name='295/75R22.5',
                            line=dict(color='#caa74d', width=2)))

# HERE IS A GREAT Hack TO MAKE SURE ALL THE MONTHS SHOW UP!!!!  ADD A WHITE LINE THAT HAS all zeros and name it as a blank string so the legend doesn't show it!
# fig.add_trace(go.Scatter(x=Month_LIST2, y=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], name='',
#                          line=dict(color='white', width=1)))
# Edit the Layout:
fig2.update_layout(
    title_text=f"<b>Bandag Retread Purchases Trend {Last_Year} - {Current_Year}</b>",
    title_x=.5,
    title_font_size=20,
    title_font_color="#351C15",
    xaxis_title='Month',
    yaxis_title='Purchases',
    autosize=False, width=1050, height=525,
    showlegend=True,
    legend=dict(
        x=1,
        y=1,
        title_font_family="Times New Roman",
        font=dict(
            family=None,
            size=13,
            color="black"
        ),
        bgcolor="white",
        bordercolor="dimgray",
        borderwidth=.05
    ),
    # General axes and axes labels fonts:
    font=dict(
        family=None,
        size=13,
        color="black"
    ))

# Add a green vertical line the graph to separate last year from the current one:
fig2.add_vline(x=11.5, line_width=3.5,
                line_dash="dash", line_color="green")

# Save the file to use in HTML, PDF, etc.
fig2.write_image(f"Images_graphs/{Current_Year}_Bandag_Purchase_Analysis.svg")
fig2.show()





# PLOTLY INTERACTIVE:
Bandag_Purchases_Trend_graph = fig2
Bandag_Purchases_Trend_graph_for_iframe = py.plot(Bandag_Purchases_Trend_graph, filename = 'Bandag_Purchases_Trend_graph', auto_open=False)

## 3. Bridgestone Purchases Trend Analysis Line Chart:

In [ ]:
# Transpose the Merged_Service_df to make it easier for graphing:
Bridgestone_df_TRANSPOSED = Merged_Bridgestone_df.T

Description_list_from_first_row = list(Bridgestone_df_TRANSPOSED.iloc[0])
Bridgestone_df_TRANSPOSED = Bridgestone_df_TRANSPOSED.iloc[1:, :]
Bridgestone_df_TRANSPOSED.columns = Description_list_from_first_row

Bridgestone_df_TRANSPOSED = Bridgestone_df_TRANSPOSED.reset_index()
Bridgestone_df_TRANSPOSED = Bridgestone_df_TRANSPOSED.rename(columns={'index': 'month-yr'})

Bridgestone_df_TRANSPOSED

In [ ]:
# Create Bridgestone Purchase Analysis Graph:

# Setup independent X Variables (which is just the month and save a list):
Month_LIST2 = Bridgestone_df_TRANSPOSED['month-yr'].to_list()

# Setup a dependent Y Variables (By Creating separate lists for each type of Bandag Retread):
Light_Truck_List = Bridgestone_df_TRANSPOSED['Light Truck BRIDGESTONE'].to_list()
Two_Twenty_Five_LIST = Bridgestone_df_TRANSPOSED['225/70R19.5 BRIDGESTONE'].to_list()
Two_Forty_Five_LIST = Bridgestone_df_TRANSPOSED['245/70R17.5 BRIDGESTONE'].to_list()
Nine_R_Twenty_Two_Five_LIST = Bridgestone_df_TRANSPOSED['9R22.5 BRIDGESTONE'].to_list()
Two_Ninety_Five_LIST = Bridgestone_df_TRANSPOSED['295/75R22.5 BRIDGESTONE'].to_list()
Eleven_R_Twenty_Two_Point_Five_List = Bridgestone_df_TRANSPOSED['11R22.5 BRIDGESTONE'].to_list()

fig = go.Figure()
# Add a filter over last year's graph to distinguish it from this year:
fig.add_vrect(x0=0, x1=12,
                annotation_text=" Last Year ", annotation_position="top left",
                annotation=dict(
    font_size=29, font_family="Times New Roman"),
    fillcolor=None, opacity=0.25, line_width=0)

# Optionally add a filter color for this yr too, lightgreen works (or choose "white" or comment out this block for none)
fig.add_vrect(x0=12, x1=12,
                annotation_text=" Current Year ", annotation_position="top left",
                annotation=dict(
    font_size=29, font_family="Times New Roman"),
    fillcolor=None, opacity=0.15, line_width=0)

# Create and Style Traces:
fig.add_trace(go.Scatter(x=Month_LIST2, y=Light_Truck_List, name='Light Truck',
                            line=dict(color='red', width=2.5)))
fig.add_trace(go.Scatter(x=Month_LIST2, y=Two_Twenty_Five_LIST, name='225/70R19.5',
                            line=dict(color='green', width=2.5)))
fig.add_trace(go.Scatter(x=Month_LIST2, y=Two_Forty_Five_LIST, name='245/70R17.5',
                            line=dict(color='black', width=2.5)))
fig.add_trace(go.Scatter(x=Month_LIST2, y=Nine_R_Twenty_Two_Five_LIST, name='9R22.5',
                            line=dict(color='purple', width=2.5)))
fig.add_trace(go.Scatter(x=Month_LIST2, y=Two_Ninety_Five_LIST, name='295/75R22.5',
                            line=dict(color='#caa74d', width=2.5)))
fig.add_trace(go.Scatter(x=Month_LIST2, y=Eleven_R_Twenty_Two_Point_Five_List, name='11R22.5',
                            line=dict(color='royalblue', width=2.5)))

# HERE IS A GREAT Hack TO MAKE SURE ALL THE MONTHS SHOW UP!!!!  ADD A WHITE LINE THAT HAS all zeros and name it as a blank string so the legend doesn't show it!
# fig.add_trace(go.Scatter(x=Month_LIST2, y=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], name='',
#                          line=dict(color='white', width=1)))
# Edit the Layout:
fig.update_layout(
    title_text=f"<b>Bridgestone Purchases Trend {Last_Year} - {Current_Year}</b>",
    title_x=.5,
    title_font_size=24,
    title_font_color="#351C15",
    xaxis_title='Month',
    yaxis_title='Purchases',
    autosize=False, width=1350, height=675,
    showlegend=True,
    legend=dict(
        x=1,
        y=1,
        title_font_family="Times New Roman",
        font=dict(
            family=None,
            size=15,
            color="black"
        ),
        bgcolor="white",
        bordercolor="dimgray",
        borderwidth=.05
    ),
    # General axes and axes labels fonts:
    font=dict(
        family=None,
        size=16,
        color="black"
    ))

# Add a green vertical line the graph to separate last year from the current one:
fig.add_vline(x=11.5, line_width=3.5,
                line_dash="dash", line_color="green")


# Change grid color and axis colors
fig.update_xaxes(gridwidth=1, gridcolor='white')
fig.update_xaxes(gridwidth=1, gridcolor='white')


# Save the file to use in HTML, PDF, etc.
fig.write_image(f"Images_graphs/{Current_Year}_Bridgestone_Purchase_Analysis.svg")
fig.show()





# =================================================================================================================
# Now let's make ANOTHER version for our webpage which will need to be resized and customized:
fig2 = go.Figure()
# Add a filter over last year's graph to distinguish it from this year:
fig2.add_vrect(x0=0, x1=12,
                annotation_text=" Last Year ", annotation_position="top left",
                annotation=dict(
    font_size=20, font_family="Times New Roman"),
    fillcolor=None, opacity=0.25, line_width=0)

# Optionally add a filter color for this yr too, lightgreen works (or choose "white" or comment out this block for none)
fig2.add_vrect(x0=12, x1=12,
                annotation_text=" Current Year ", annotation_position="top left",
                annotation=dict(
    font_size=20, font_family="Times New Roman"),
    fillcolor=None, opacity=0.15, line_width=0)

# Create and Style Traces:
fig2.add_trace(go.Scatter(x=Month_LIST2, y=Light_Truck_List, name='Light Truck',
                            line=dict(color='red', width=2)))
fig2.add_trace(go.Scatter(x=Month_LIST2, y=Two_Twenty_Five_LIST, name='225/70R19.5',
                            line=dict(color='green', width=2)))
fig2.add_trace(go.Scatter(x=Month_LIST2, y=Two_Forty_Five_LIST, name='245/70R17.5',
                            line=dict(color='black', width=2)))
fig2.add_trace(go.Scatter(x=Month_LIST2, y=Nine_R_Twenty_Two_Five_LIST, name='9R22.5',
                            line=dict(color='purple', width=2)))
fig2.add_trace(go.Scatter(x=Month_LIST2, y=Two_Ninety_Five_LIST, name='295/75R22.5',
                            line=dict(color='#caa74d', width=2)))
fig2.add_trace(go.Scatter(x=Month_LIST2, y=Eleven_R_Twenty_Two_Point_Five_List, name='11R22.5',
                            line=dict(color='royalblue', width=2)))

fig2.update_layout(
    title_text=f"<b>Bridgestone Purchases Trend {Last_Year} - {Current_Year}</b>",
    title_x=.5,
    title_font_size=20,
    title_font_color="#351C15",
    xaxis_title='Month',
    yaxis_title='Purchases',
    autosize=False, width=1050, height=525,
    showlegend=True,
    legend=dict(
        x=1,
        y=1,
        title_font_family="Times New Roman",
        font=dict(
            family=None,
            size=13,
            color="black"
        ),
        bgcolor="white",
        bordercolor="dimgray",
        borderwidth=.05
    ),
    # General axes and axes labels fonts:
    font=dict(
        family=None,
        size=13,
        color="black"
    ))

# Add a green vertical line the graph to separate last year from the current one:
fig2.add_vline(x=11.5, line_width=3.5,
                line_dash="dash", line_color="green")

# PLOTLY INTERACTIVE:
Bridgestone_Purchases_Trend_graph = fig2
Bridgestone_Purchases_Trend_graph_for_iframe = py.plot(Bridgestone_Purchases_Trend_graph, filename = 'Bridgestone_Purchases_Trend_graph', auto_open=False)

fig2.show()

# Get SWIFT NRT DATA by scraping BFI site.  

### NOTE:  We will scrape both last year and the current YTD bc we want to compare last year with current YTD in graphs:

## Since the website appears to do better the second attempt of selecting the NRT template, we'll 
## do a mock selection here prior to doing it for real in the loop afterwards:

In [ ]:
# ---------Set the executable path and initialize the chrome browser in splinter -------
executable_path = {'executable_path': 'C:/webdrivers/chromedriver'}
browser = Browser('chrome', **executable_path)

# --------------------------- LOGIN ------------------------------------------
# ---Set Variables for login:
Page_usrname = 'stmreport'
Page_pword = 'stmreport'

attempts = 0

# Go to BFI login page:
browser.visit('https://fleetinventory.com')

#Find the username and password boxes and fill them in:
browser.find_by_id('username').fill(Page_usrname)
browser.find_by_id('password').fill(Page_pword)
# Click on login button:
browser.find_by_name('submit').first.click()

# ---------------- Navigate to the NRT Summary page ---------------------------
# Click on "Reporting" tab and then click "NRT Summary":
time.sleep(3)
browser.find_by_text('Reporting', wait_time=4).first.click()
browser.find_by_text('NRT Summary', wait_time=4).first.click()
time.sleep(10)

# Click on the "Report Profile" drop down box.  This will give you options from saved profiles you can choose from (for example, RR-UPS SCRAP SUMMARY)
browser.find_by_id('id_report_profile', wait_time=6).first.click()

# ----------------- To select the region on the BFI site, we'll SELECT ONE OF THE SAVED TEMPLATES FROM THE DROP-DOWN WE JUST CLICKED ON -----------------
time.sleep(2)
browser.find_by_text('LONESTAR NRT SUMMARY WALTON REPORT', wait_time=10).first.click()
time.sleep(15)
# Once done, logout and quit the browser:
browser.find_by_id('logout').first.click()
browser.quit()

In [ ]:
# Loop through last year and YTD of current year:
years = [Last_Year, Current_Year]

for year in years:

    Jan = year * 100 + 1
    Feb = year * 100 + 2
    Mar = year * 100 + 3
    Apr = year * 100 + 4
    May = year * 100 + 5
    Jun = year * 100 + 6
    Jul = year * 100 + 7
    Aug = year * 100 + 8
    Sept = year * 100 + 9
    Oct = year * 100 + 10
    Nov = year * 100 + 11
    Dec = year * 100 + 12

    # We can also define a list of these numerical Months:
    # Again, these will be in format like Jan = 202101.
    Months = [Jan, Feb, Mar, Apr, May, Jun, Jul, Aug, Sept, Oct, Nov, Dec]

    # ---------Set the executable path and initialize the chrome browser in splinter -------
    executable_path = {'executable_path': 'C:/webdrivers/chromedriver'}
    browser = Browser('chrome', **executable_path)

    # --------------------------- LOGIN ------------------------------------------
    # ---Set Variables for login:
    Page_usrname = 'stmreport'
    Page_pword = 'stmreport'

    attempts = 0

    # Go to BFI login page:
    browser.visit('https://fleetinventory.com')

    #Find the username and password boxes and fill them in:
    browser.find_by_id('username').fill(Page_usrname)
    browser.find_by_id('password').fill(Page_pword)
    # Click on login button:
    browser.find_by_name('submit').first.click()

    # ---------------- Navigate to the NRT Summary page ---------------------------
    # Click on "Reporting" tab and then click "NRT Summary":
    time.sleep(3)
    browser.find_by_text('Reporting', wait_time=4).first.click()
    browser.find_by_text('NRT Summary', wait_time=4).first.click()
    time.sleep(10)

    # Click on the "Report Profile" drop down box.  This will give you options from saved profiles you can choose from (for example, RR-UPS SCRAP SUMMARY)
    browser.find_by_id('id_report_profile', wait_time=6).first.click()

    # ----------------- To select the region on the BFI site, we'll SELECT ONE OF THE SAVED TEMPLATES FROM THE DROP-DOWN WE JUST CLICKED ON -----------------
    time.sleep(2)
    browser.find_by_text('LONESTAR NRT SUMMARY WALTON REPORT', wait_time=10).first.click()
    time.sleep(15)

    while attempts < 4:
        attempts += 1

        browser.find_by_id('id_date_range', wait_time=15).first.click()
        browser.find_by_text('Range', wait_time=1).first.click()


        # ------------- To fill in the date range on the BFI site, we'll first need to define Start and End Date variables for each month --------------
        January_Start = f'01-01-{year}'
        January_End = f'01-31-{year}'

        February_Start = f'02-01-{year}'
        # ----------------------
        #Adjust for Leap year:
        if (year % 400 == 0) and (year % 100 != 0) and (year % 4 != 0): 
            # (If all the above are true, it's a leap year)
            February_End = f'02-29-{year}'
        else:
            February_End = f'02-28-{year}'
        # ----------------------

        March_Start = f'03-01-{year}'
        March_End = f'03-31-{year}'

        April_Start = f'04-01-{year}'
        April_End = f'04-30-{year}'

        May_Start = f'05-01-{year}'
        May_End = f'05-31-{year}'

        June_Start = f'06-01-{year}'
        June_End = f'06-30-{year}'

        July_Start = f'07-01-{year}'
        July_End = f'07-31-{year}'

        August_Start = f'08-01-{year}'
        August_End = f'08-31-{year}'

        September_Start = f'09-01-{year}'
        September_End = f'09-30-{year}'

        October_Start = f'10-01-{year}'
        October_End = f'10-31-{year}'

        November_Start = f'11-01-{year}'
        November_End = f'11-30-{year}'

        December_Start = f'12-01-{year}'
        December_End = f'12-31-{year}'

        # ------ Create a List of lists.  The inner lists will each be for each month's start, end, and number (for instacn Jan's number could be 202101 if 2021 is year).
        Months_Start_End_Numerical = [[January_Start, January_End, Jan],
                                    [February_Start, February_End, Feb],
                                    [March_Start, March_End, Mar],
                                    [April_Start, April_End, Apr],
                                    [May_Start, May_End, May],
                                    [June_Start, June_End, Jun], 
                                    [July_Start, July_End, Jul],
                                    [August_Start, August_End, Aug],
                                    [September_Start,September_End, Sept],
                                    [October_Start, October_End, Oct], 
                                    [November_Start, November_End, Nov], 
                                    [December_Start, December_End, Dec]]

        #Give the webpage a moment to continue loading:
        time.sleep(2)
        prompt = browser.get_alert()
        # Now loop through the list of lists to download the CSVs for only the months that have been completed (Current_Month > month_abbr)
        Was_Successful = True
        for start, end, month_as_YYYYMM in Months_Start_End_Numerical:
            if Current_Month > month_as_YYYYMM:
                try:
                    # Fill in the date range for the NRT CSV:
                    time.sleep(2)
                    browser.find_by_id('id_start_date', wait_time=4).fill(start)
                    browser.find_by_id('id_end_date', wait_time=4).fill(end)
                    # Click the "run report" button to download the NRT CSV
                    browser.find_by_id('form_submit').first.click()
                    # Give the dowload 2 seconds to complete:
                    time.sleep(1)
                    # Move the file from "downloads" folder 
                    # and rename it from nrt_analysis.csv to specify the region and date
                    original = r'C:/Users/alex.conerly/Downloads/nrt_analysis.csv'
                    target = f'{os.getcwd()}/NRT_CSVs_BFI_SCRAPE/LONESTAR_NRT_{month_as_YYYYMM}.csv'

                    # Attempt to move the file
                    shutil.move(original,target)
                    time.sleep(1)

                except:
                    print('FileNotFoundError, likely because page did not fully load')
                    Was_Successful = False
                    break

        if Was_Successful == True:
            break

    # Once done, logout and quit the browser:
    browser.find_by_id('logout').first.click()
    browser.quit()

## Now that we've scraped and imported the NRT data for two years and saved them as CSVs, we'll loop through both years again
## and format the data to prepare it for html and also graphing.  Of course, html will only show the current year.  The graph will show both.

In [ ]:

years = [Last_Year, Current_Year]

for year in years:

    Jan = year * 100 + 1
    Feb = year * 100 + 2
    Mar = year * 100 + 3
    Apr = year * 100 + 4
    May = year * 100 + 5
    Jun = year * 100 + 6
    Jul = year * 100 + 7
    Aug = year * 100 + 8
    Sept = year * 100 + 9
    Oct = year * 100 + 10
    Nov = year * 100 + 11
    Dec = year * 100 + 12


    # JAN
    if Jan < Current_Month:
        Nrt_JAN = pd.read_csv(f'NRT_CSVs_BFI_SCRAPE/LONESTAR_NRT_{Jan}.csv', skiprows=24)
        # Get rid of the Top 25 unecessary rows and the bottom 2 rows for "Total" and "Grand Total"
        Nrt_JAN = Nrt_JAN.iloc[:-2, :]
        # Get rid of the "% of Total" column.  We will calculate percentages manually later after merging all month dfs etc.
        Nrt_JAN = Nrt_JAN.drop(columns=[r'% of Total'])
        # Rename the "Total" column to the month from which this NRT data was derived:
        Nrt_JAN = Nrt_JAN.rename(columns = {'Total': 'JAN'})

    # For all the rest of the months, do the same as above BUT drop the Description column to 
    # prepare for merging with the January df (which already has the Description column):
    # FEB
    if Feb < Current_Month:
        Nrt_FEB = pd.read_csv(f'NRT_CSVs_BFI_SCRAPE/LONESTAR_NRT_{Feb}.csv', skiprows=24)
        Nrt_FEB = Nrt_FEB.iloc[:-2, :]
        Nrt_FEB = Nrt_FEB.drop(columns=[r'% of Total'])
        Nrt_FEB = Nrt_FEB.rename(columns = {'Total': 'FEB'})
        Nrt_FEB = Nrt_FEB.drop(columns = ['Description'])

    # MAR
    if Mar < Current_Month:
        Nrt_MAR = pd.read_csv(f'NRT_CSVs_BFI_SCRAPE/LONESTAR_NRT_{Mar}.csv', skiprows=24)
        Nrt_MAR = Nrt_MAR.iloc[:-2, :]
        Nrt_MAR = Nrt_MAR.drop(columns=[r'% of Total'])
        Nrt_MAR = Nrt_MAR.rename(columns = {'Total': 'MAR'})
        Nrt_MAR = Nrt_MAR.drop(columns = ['Description'])

    # APR
    if Apr < Current_Month:
        Nrt_APR = pd.read_csv(f'NRT_CSVs_BFI_SCRAPE/LONESTAR_NRT_{Apr}.csv', skiprows=24)
        Nrt_APR = Nrt_APR.iloc[:-2, :]
        Nrt_APR = Nrt_APR.drop(columns=[r'% of Total'])
        Nrt_APR = Nrt_APR.rename(columns = {'Total': 'APR'})
        Nrt_APR = Nrt_APR.drop(columns = ['Description'])

    # MAY
    if May < Current_Month:
        Nrt_MAY = pd.read_csv(f'NRT_CSVs_BFI_SCRAPE/LONESTAR_NRT_{May}.csv', skiprows=24)
        Nrt_MAY = Nrt_MAY.iloc[:-2, :]
        Nrt_MAY = Nrt_MAY.drop(columns=[r'% of Total'])
        Nrt_MAY = Nrt_MAY.rename(columns = {'Total': 'MAY'})
        Nrt_MAY = Nrt_MAY.drop(columns = ['Description'])

    # JUN
    if Jun < Current_Month:
        Nrt_JUN = pd.read_csv(f'NRT_CSVs_BFI_SCRAPE/LONESTAR_NRT_{Jun}.csv', skiprows=24)
        Nrt_JUN = Nrt_JUN.iloc[:-2, :]
        Nrt_JUN = Nrt_JUN.drop(columns=[r'% of Total'])
        Nrt_JUN = Nrt_JUN.rename(columns = {'Total': 'JUN'})
        Nrt_JUN = Nrt_JUN.drop(columns = ['Description'])

    # JUL
    if Jul < Current_Month:
        Nrt_JUL = pd.read_csv(f'NRT_CSVs_BFI_SCRAPE/LONESTAR_NRT_{Jul}.csv', skiprows=24)
        Nrt_JUL = Nrt_JUL.iloc[:-2, :]
        Nrt_JUL = Nrt_JUL.drop(columns=[r'% of Total'])
        Nrt_JUL = Nrt_JUL.rename(columns = {'Total': 'JUL'})
        Nrt_JUL = Nrt_JUL.drop(columns = ['Description'])

    # AUG
    if Aug < Current_Month:
        Nrt_AUG = pd.read_csv(f'NRT_CSVs_BFI_SCRAPE/LONESTAR_NRT_{Aug}.csv', skiprows=24)
        Nrt_AUG = Nrt_AUG.iloc[:-2, :]
        Nrt_AUG = Nrt_AUG.drop(columns=[r'% of Total'])
        Nrt_AUG = Nrt_AUG.rename(columns = {'Total': 'AUG'})
        Nrt_AUG = Nrt_AUG.drop(columns = ['Description'])

    # SEPT
    if Sept < Current_Month:
        Nrt_SEPT = pd.read_csv(f'NRT_CSVs_BFI_SCRAPE/LONESTAR_NRT_{Sept}.csv', skiprows=24)
        Nrt_SEPT = Nrt_SEPT.iloc[:-2, :]
        Nrt_SEPT = Nrt_SEPT.drop(columns=[r'% of Total'])
        Nrt_SEPT = Nrt_SEPT.rename(columns = {'Total': 'SEPT'})
        Nrt_SEPT = Nrt_SEPT.drop(columns = ['Description'])

    # OCT
    if Oct < Current_Month:
        Nrt_OCT = pd.read_csv(f'NRT_CSVs_BFI_SCRAPE/LONESTAR_NRT_{Oct}.csv', skiprows=24)
        Nrt_OCT = Nrt_OCT.iloc[:-2, :]
        Nrt_OCT = Nrt_OCT.drop(columns=[r'% of Total'])
        Nrt_OCT = Nrt_OCT.rename(columns = {'Total': 'OCT'})
        Nrt_OCT = Nrt_OCT.drop(columns = ['Description'])

    # NOV
    if Nov < Current_Month:
        Nrt_NOV = pd.read_csv(f'NRT_CSVs_BFI_SCRAPE/LONESTAR_NRT_{Nov}.csv', skiprows=24)
        Nrt_NOV = Nrt_NOV.iloc[:-2, :]
        Nrt_NOV = Nrt_NOV.drop(columns=[r'% of Total'])
        Nrt_NOV = Nrt_NOV.rename(columns = {'Total': 'NOV'})
        Nrt_NOV = Nrt_NOV.drop(columns = ['Description'])

    # DEC
    if Dec < Current_Month:
        Nrt_DEC = pd.read_csv(f'NRT_CSVs_BFI_SCRAPE/LONESTAR_NRT_{Dec}.csv', skiprows=24)
        Nrt_DEC = Nrt_DEC.iloc[:-2, :]
        Nrt_DEC = Nrt_DEC.drop(columns=[r'% of Total'])
        Nrt_DEC = Nrt_DEC.rename(columns = {'Total': 'DEC'})
        Nrt_DEC = Nrt_DEC.drop(columns = ['Description'])


    # Merge each month if it exists:
    if Current_Month > Jan:
        Lonestar_Nrt_Merged = Nrt_JAN
    if Current_Month > Feb:
        Lonestar_Nrt_Merged = pd.merge(Lonestar_Nrt_Merged, Nrt_FEB, how='outer', on="NRT Code") # Merged JAN - FEB
    if Current_Month > Mar:
        Lonestar_Nrt_Merged = pd.merge(Lonestar_Nrt_Merged, Nrt_MAR, how='outer', on="NRT Code") # Merged JAN - MAR
    if Current_Month > Apr:
        Lonestar_Nrt_Merged = pd.merge(Lonestar_Nrt_Merged, Nrt_APR, how='outer', on="NRT Code") # Merged JAN - APR
    if Current_Month > May:
        Lonestar_Nrt_Merged = pd.merge(Lonestar_Nrt_Merged, Nrt_MAY, how='outer', on="NRT Code") # Merged JAN - MAY
    if Current_Month > Jun:
        Lonestar_Nrt_Merged = pd.merge(Lonestar_Nrt_Merged, Nrt_JUN, how='outer', on="NRT Code") # Merged JAN - JUN
    if Current_Month > Jul:
        Lonestar_Nrt_Merged = pd.merge(Lonestar_Nrt_Merged, Nrt_JUL, how='outer', on="NRT Code") # Merged JAN - JUL
    if Current_Month > Aug:
        Lonestar_Nrt_Merged = pd.merge(Lonestar_Nrt_Merged, Nrt_AUG, how='outer', on="NRT Code") # Merged JAN - AUG
    if Current_Month > Sept:
        Lonestar_Nrt_Merged = pd.merge(Lonestar_Nrt_Merged, Nrt_SEPT, how='outer', on="NRT Code") # Merged JAN - SEPT
    if Current_Month > Oct:
        Lonestar_Nrt_Merged = pd.merge(Lonestar_Nrt_Merged, Nrt_OCT, how='outer', on="NRT Code") # Merged JAN - OCT
    if Current_Month > Nov:
        Lonestar_Nrt_Merged = pd.merge(Lonestar_Nrt_Merged, Nrt_NOV, how='outer', on="NRT Code") # Merged JAN - NOV
    if Current_Month > Dec:
        Lonestar_Nrt_Merged = pd.merge(Lonestar_Nrt_Merged, Nrt_DEC, how='outer', on="NRT Code") # Merged JAN - DEC

    Lonestar_Nrt_Merged = Lonestar_Nrt_Merged.fillna(0)



    # Create the YTD COlUMN by adding all the months (if they have) completed:
    if Current_Month > Dec:
        Lonestar_Nrt_Merged['YTD'] = Lonestar_Nrt_Merged['JAN'] + \
                            Lonestar_Nrt_Merged['FEB'] + \
                            Lonestar_Nrt_Merged['MAR'] + \
                            Lonestar_Nrt_Merged['APR'] + \
                            Lonestar_Nrt_Merged['MAY'] + \
                            Lonestar_Nrt_Merged['JUN'] + \
                            Lonestar_Nrt_Merged['JUL'] + \
                            Lonestar_Nrt_Merged['AUG'] + \
                            Lonestar_Nrt_Merged['SEPT'] + \
                            Lonestar_Nrt_Merged['OCT'] + \
                            Lonestar_Nrt_Merged['NOV'] + \
                            Lonestar_Nrt_Merged['DEC']
    elif Current_Month > Nov:
        Lonestar_Nrt_Merged['YTD'] = Lonestar_Nrt_Merged['JAN'] + \
                            Lonestar_Nrt_Merged['FEB'] + \
                            Lonestar_Nrt_Merged['MAR'] + \
                            Lonestar_Nrt_Merged['APR'] + \
                            Lonestar_Nrt_Merged['MAY'] + \
                            Lonestar_Nrt_Merged['JUN'] + \
                            Lonestar_Nrt_Merged['JUL'] + \
                            Lonestar_Nrt_Merged['AUG'] + \
                            Lonestar_Nrt_Merged['SEPT'] + \
                            Lonestar_Nrt_Merged['OCT'] + \
                            Lonestar_Nrt_Merged['NOV']
    elif Current_Month > Oct:
        Lonestar_Nrt_Merged['YTD'] = Lonestar_Nrt_Merged['JAN'] + \
                            Lonestar_Nrt_Merged['FEB'] + \
                            Lonestar_Nrt_Merged['MAR'] + \
                            Lonestar_Nrt_Merged['APR'] + \
                            Lonestar_Nrt_Merged['MAY'] + \
                            Lonestar_Nrt_Merged['JUN'] + \
                            Lonestar_Nrt_Merged['JUL'] + \
                            Lonestar_Nrt_Merged['AUG'] + \
                            Lonestar_Nrt_Merged['SEPT'] + \
                            Lonestar_Nrt_Merged['OCT']
    elif Current_Month > Sept:
        Lonestar_Nrt_Merged['YTD'] = Lonestar_Nrt_Merged['JAN'] + \
                            Lonestar_Nrt_Merged['FEB'] + \
                            Lonestar_Nrt_Merged['MAR'] + \
                            Lonestar_Nrt_Merged['APR'] + \
                            Lonestar_Nrt_Merged['MAY'] + \
                            Lonestar_Nrt_Merged['JUN'] + \
                            Lonestar_Nrt_Merged['JUL'] + \
                            Lonestar_Nrt_Merged['AUG'] + \
                            Lonestar_Nrt_Merged['SEPT']
    elif Current_Month > Aug:
        Lonestar_Nrt_Merged['YTD'] = Lonestar_Nrt_Merged['JAN'] + \
                            Lonestar_Nrt_Merged['FEB'] + \
                            Lonestar_Nrt_Merged['MAR'] + \
                            Lonestar_Nrt_Merged['APR'] + \
                            Lonestar_Nrt_Merged['MAY'] + \
                            Lonestar_Nrt_Merged['JUN'] + \
                            Lonestar_Nrt_Merged['JUL'] + \
                            Lonestar_Nrt_Merged['AUG']
    elif Current_Month > Jul:
        Lonestar_Nrt_Merged['YTD'] = Lonestar_Nrt_Merged['JAN'] + \
                            Lonestar_Nrt_Merged['FEB'] + \
                            Lonestar_Nrt_Merged['MAR'] + \
                            Lonestar_Nrt_Merged['APR'] + \
                            Lonestar_Nrt_Merged['MAY'] + \
                            Lonestar_Nrt_Merged['JUN'] + \
                            Lonestar_Nrt_Merged['JUL']
    elif Current_Month > Jun:
        Lonestar_Nrt_Merged['YTD'] = Lonestar_Nrt_Merged['JAN'] + \
                            Lonestar_Nrt_Merged['FEB'] + \
                            Lonestar_Nrt_Merged['MAR'] + \
                            Lonestar_Nrt_Merged['APR'] + \
                            Lonestar_Nrt_Merged['MAY'] + \
                            Lonestar_Nrt_Merged['JUN']
    elif Current_Month > May:
        Lonestar_Nrt_Merged['YTD'] = Lonestar_Nrt_Merged['JAN'] + \
                            Lonestar_Nrt_Merged['FEB'] + \
                            Lonestar_Nrt_Merged['MAR'] + \
                            Lonestar_Nrt_Merged['APR'] + \
                            Lonestar_Nrt_Merged['MAY']
    elif Current_Month > Apr:
        # Lonestar_Nrt_Merged['YTD'] = Lonestar_Nrt_Merged.sum(numeric_only=True)
        Lonestar_Nrt_Merged['YTD'] = Lonestar_Nrt_Merged['JAN'] + \
                            Lonestar_Nrt_Merged['FEB'] + \
                            Lonestar_Nrt_Merged['MAR'] + \
                            Lonestar_Nrt_Merged['APR']
    elif Current_Month > Mar:
        Lonestar_Nrt_Merged['YTD'] = Lonestar_Nrt_Merged['JAN'] + \
                            Lonestar_Nrt_Merged['FEB'] + \
                            Lonestar_Nrt_Merged['MAR']
    elif Current_Month > Feb:
        Lonestar_Nrt_Merged['YTD'] = Lonestar_Nrt_Merged['JAN'] + \
                            Lonestar_Nrt_Merged['FEB']
    elif Current_Month >= Jan:
        Lonestar_Nrt_Merged['YTD'] = Lonestar_Nrt_Merged['JAN']

    # Reorder YTD to be right after description:
    if Current_Month > Dec:
        Lonestar_Nrt_Merged = Lonestar_Nrt_Merged[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEPT', 'OCT', 'NOV', 'DEC']]
    elif Current_Month > Nov:
        Lonestar_Nrt_Merged = Lonestar_Nrt_Merged[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEPT', 'OCT', 'NOV']]
    elif Current_Month > Oct:
        Lonestar_Nrt_Merged = Lonestar_Nrt_Merged[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEPT', 'OCT']]
    elif Current_Month > Sept:
        Lonestar_Nrt_Merged = Lonestar_Nrt_Merged[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEPT']]
    elif Current_Month > Aug:
        Lonestar_Nrt_Merged = Lonestar_Nrt_Merged[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG']]
    elif Current_Month > Jul:
        Lonestar_Nrt_Merged = Lonestar_Nrt_Merged[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL']]
    elif Current_Month > Jun:
        Lonestar_Nrt_Merged = Lonestar_Nrt_Merged[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN']]
    elif Current_Month > May:
        Lonestar_Nrt_Merged = Lonestar_Nrt_Merged[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY']]
    elif Current_Month > Apr:
        Lonestar_Nrt_Merged = Lonestar_Nrt_Merged[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR']]
    elif Current_Month > Mar:
        Lonestar_Nrt_Merged = Lonestar_Nrt_Merged[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR']]
    elif Current_Month > Feb:
        Lonestar_Nrt_Merged = Lonestar_Nrt_Merged[['NRT Code', "Description", 'YTD', 'JAN', 'FEB']]
    elif Current_Month >= Jan:
        Lonestar_Nrt_Merged = Lonestar_Nrt_Merged[['NRT Code', "Description", 'YTD', 'JAN']]


    # Sort Nrt Codes in order.  Then Rename Missing Descriptions:
    Lonestar_Nrt_Merged = Lonestar_Nrt_Merged.sort_values(['NRT Code'])
    Lonestar_Nrt_Merged = Lonestar_Nrt_Merged.reset_index().drop(columns=['index'])

    # In some cases, where a particular NRT Code had all zeros, the "Description" column value will have a '0'
    # Therefore, we will manually ensure each NRT Code has the correct description
    # by replacing whatever was there prior with the correct Description for that NRT Code. 
    # Note, we won't fix EVERY POSSIBLE Description because these are the only NRT Code names we'll 
    # need for all the for the NRT Summary Table.

    Lonestar_Nrt_Merged.loc[Lonestar_Nrt_Merged['NRT Code'] == '2002', 'Description'] = 'Sidewall Cut(s)/Snag(s)'
    Lonestar_Nrt_Merged.loc[Lonestar_Nrt_Merged['NRT Code'] == '2004', 'Description'] = 'Run Under-Inflated'
    Lonestar_Nrt_Merged.loc[Lonestar_Nrt_Merged['NRT Code'] == '2005', 'Description'] = 'Run To Destruction'
    Lonestar_Nrt_Merged.loc[Lonestar_Nrt_Merged['NRT Code'] == '2006', 'Description'] = 'Impact Split(s)'
    Lonestar_Nrt_Merged.loc[Lonestar_Nrt_Merged['NRT Code'] == '2009', 'Description'] = 'Pinch Shock'
    Lonestar_Nrt_Merged.loc[Lonestar_Nrt_Merged['NRT Code'] == '2010', 'Description'] = 'Rock Drilling'
    Lonestar_Nrt_Merged.loc[Lonestar_Nrt_Merged['NRT Code'] == '2101', 'Description'] = 'Bead Damage - Mount/Dismount R'
    Lonestar_Nrt_Merged.loc[Lonestar_Nrt_Merged['NRT Code'] == '2102', 'Description'] = 'Worn Too Far'
    Lonestar_Nrt_Merged.loc[Lonestar_Nrt_Merged['NRT Code'] == '2205', 'Description'] = 'Shoulder Separation'


    if '2002' not in Lonestar_Nrt_Merged.values:
        line = pd.DataFrame({"NRT Code": '2002', "Description": 'Sidewall Cut(s)/Snag(s)'}, index=[1.1])
        Lonestar_Nrt_Merged = Lonestar_Nrt_Merged.append(line, ignore_index=False)
        Lonestar_Nrt_Merged = Lonestar_Nrt_Merged.sort_index().reset_index(drop=True)
    if '2004' not in Lonestar_Nrt_Merged.values:
        line = pd.DataFrame({"NRT Code": '2004', "Description": 'Run Under-Inflated'}, index=[2.1])
        Lonestar_Nrt_Merged = Lonestar_Nrt_Merged.append(line, ignore_index=False)
        Lonestar_Nrt_Merged = Lonestar_Nrt_Merged.sort_index().reset_index(drop=True)
    if '2005' not in Lonestar_Nrt_Merged.values:
        line = pd.DataFrame({"NRT Code": '2005', "Description": 'Run to Destruction'}, index=[2.1])
        Lonestar_Nrt_Merged = Lonestar_Nrt_Merged.append(line, ignore_index=False)
        Lonestar_Nrt_Merged = Lonestar_Nrt_Merged.sort_index().reset_index(drop=True)
    if '2006' not in Lonestar_Nrt_Merged.values:
        line = pd.DataFrame({"NRT Code": '2006', "Description": 'Impact Split(s)'}, index=[4.1])
        Lonestar_Nrt_Merged = Lonestar_Nrt_Merged.append(line, ignore_index=False)
        Lonestar_Nrt_Merged = Lonestar_Nrt_Merged.sort_index().reset_index(drop=True)
    if '2009' not in Lonestar_Nrt_Merged.values:
        line = pd.DataFrame({"NRT Code": '2009', "Description": 'Pinch Shock'}, index=[4.1])
        Lonestar_Nrt_Merged = Lonestar_Nrt_Merged.append(line, ignore_index=False)
        Lonestar_Nrt_Merged = Lonestar_Nrt_Merged.sort_index().reset_index(drop=True)
    if '2010' not in Lonestar_Nrt_Merged.values:
        line = pd.DataFrame({"NRT Code": '2010', "Description": 'Rock Drilling'}, index=[4.1])
        Lonestar_Nrt_Merged = Lonestar_Nrt_Merged.append(line, ignore_index=False)
        Lonestar_Nrt_Merged = Lonestar_Nrt_Merged.sort_index().reset_index(drop=True)
    if '2101' not in Lonestar_Nrt_Merged.values:
        line = pd.DataFrame({"NRT Code": '2101', "Description": 'Bead Damage - Mount/Dismount R'}, index=[4.1])
        Lonestar_Nrt_Merged = Lonestar_Nrt_Merged.append(line, ignore_index=False)
        Lonestar_Nrt_Merged = Lonestar_Nrt_Merged.sort_index().reset_index(drop=True)
    if '2102' not in Lonestar_Nrt_Merged.values:
        line = pd.DataFrame({"NRT Code": '2102', "Description": 'Worn Too Far'}, index=[10.1])
        Lonestar_Nrt_Merged = Lonestar_Nrt_Merged.append(line, ignore_index=False)
        Lonestar_Nrt_Merged = Lonestar_Nrt_Merged.sort_index().reset_index(drop=True)
    if '2205' not in Lonestar_Nrt_Merged.values:
        line = pd.DataFrame({"NRT Code": '2205', "Description": 'Shoulder Separation'}, index=[16.1])
        Lonestar_Nrt_Merged = Lonestar_Nrt_Merged.append(line, ignore_index=False)
        Lonestar_Nrt_Merged = Lonestar_Nrt_Merged.sort_index().reset_index(drop=True)

    # -------------------------------------------------
    # Here we need to check the df to see if any NRT Code from 2300-2309 (or 2400-2409) exists and if at least one NRT Code doesn't appear to exist, then we will create a placeholder value for the range(s).
    # As a placeholder for 2300-2309 (or 2400-2409), we'll use 2300 (or 2400) and run a loop with regex to check if the value occured somewhere and, if so, we won't create a new row.

    Is_there_a_2300_based_row = False

    for row_number in range(len(Lonestar_Nrt_Merged)):
        if re.match('230\d', Lonestar_Nrt_Merged['NRT Code'].values[row_number]):
            Is_there_a_2300_based_row = True

    if Is_there_a_2300_based_row == False:
        line = pd.DataFrame({"NRT Code": '2300', "Description": 'Outside of Fleet Specs'}, index=[17.1])
        Lonestar_Nrt_Merged = Lonestar_Nrt_Merged.append(line, ignore_index=False)
        Lonestar_Nrt_Merged = Lonestar_Nrt_Merged.sort_index().reset_index(drop=True)

    Is_there_a_2400_based_row = False

    for row_number in range(len(Lonestar_Nrt_Merged)):
        if re.match('240\d', Lonestar_Nrt_Merged['NRT Code'].values[row_number]):
            Is_there_a_2400_based_row = True

    if Is_there_a_2400_based_row == False:
        line = pd.DataFrame({"NRT Code": '2400', "Description": 'Injuries too Close/Large'}, index=[17.1])
        Lonestar_Nrt_Merged = Lonestar_Nrt_Merged.append(line, ignore_index=False)
        Lonestar_Nrt_Merged = Lonestar_Nrt_Merged.sort_index().reset_index(drop=True)


    # ---------------------------------------------------------------------------------------------------------------------------------------------------------------------
    # Group all codes that start with 23** to 2300 and change description to "Outside of Fleet Specifications" AND
    # Group all codes that start with 24** to 2400 and change description to "Injuries too Close/Large"

    List_of_2300 = ['2300', '2301', '2302', '2303', '2304', '2305', '2306', '2307', '2308', '2309']
    List_of_2400 = ['2400', '2401', '2402', '2403', '2404', '2405', '2406', '2407', '2408', '2409']

    # Create two TEMPORARY dfs, one for 2300 based codes and the other for 2400 based codes:
    Nrt_2300_df = Lonestar_Nrt_Merged.loc[Lonestar_Nrt_Merged['NRT Code'].isin(List_of_2300)]
    Nrt_2400_df = Lonestar_Nrt_Merged.loc[Lonestar_Nrt_Merged['NRT Code'].isin(List_of_2400)]

    # Drop all ungrouped 2300 and 2400 based codes from main Nrt df:
    Lonestar_Nrt_Merged = Lonestar_Nrt_Merged.loc[~Lonestar_Nrt_Merged["NRT Code"].isin(List_of_2300)]
    Lonestar_Nrt_Merged = Lonestar_Nrt_Merged.loc[~Lonestar_Nrt_Merged["NRT Code"].isin(List_of_2400)]

    # Set ALL values for the first TEMPORARY df to 2300 and all values for the second TEMPORARY df to 2400:
    Nrt_2300_df["NRT Code"] = '2300'
    Nrt_2400_df["NRT Code"] = '2400'

    # Group all the NRT Codes that are 2300 into ONE (Summed) row... each column will be summed (YTD, JAN, etc):
    Nrt_2300_df = Nrt_2300_df.groupby(Nrt_2300_df["NRT Code"]).sum()
    Nrt_2300_df = Nrt_2300_df.reset_index()
    Nrt_2300_df['Description'] = "Outside of Fleet Specifications"

    # Group all the NRT Codes that are 2400 into ONE (Summed) row... each column will be summed (YTD, Jan, etc):
    Nrt_2400_df = Nrt_2400_df.groupby(Nrt_2400_df["NRT Code"]).sum()
    Nrt_2400_df = Nrt_2400_df.reset_index()
    Nrt_2400_df['Description'] = "Injuries too Close/Large"

    #---------------------------- REORDER the columns depending on whether the month exists yet:
    if Current_Month > Dec:
        Nrt_2300_df = Nrt_2300_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEPT', 'OCT', 'NOV', 'DEC']]
        Nrt_2400_df = Nrt_2400_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEPT', 'OCT', 'NOV', 'DEC']]

    elif Current_Month > Nov:
        Nrt_2300_df = Nrt_2300_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEPT', 'OCT', 'NOV']]
        Nrt_2400_df = Nrt_2400_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEPT', 'OCT', 'NOV']]

    elif Current_Month > Oct:
        Nrt_2300_df = Nrt_2300_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEPT', 'OCT']]
        Nrt_2400_df = Nrt_2400_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEPT', 'OCT']]

    elif Current_Month > Sept:
        Nrt_2300_df = Nrt_2300_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEPT']]
        Nrt_2400_df = Nrt_2400_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEPT']]

    elif Current_Month > Aug:
        Nrt_2300_df = Nrt_2300_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG']]
        Nrt_2400_df = Nrt_2400_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG']]

    elif Current_Month > Jul:
        Nrt_2300_df = Nrt_2300_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL']]
        Nrt_2400_df = Nrt_2400_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL']]

    elif Current_Month > Jun:
        Nrt_2300_df = Nrt_2300_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN']]
        Nrt_2400_df = Nrt_2400_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN']]

    elif Current_Month > May:
        Nrt_2300_df = Nrt_2300_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY']]
        Nrt_2400_df = Nrt_2400_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY']]

    elif Current_Month > Apr:
        Nrt_2300_df = Nrt_2300_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR']]
        Nrt_2400_df = Nrt_2400_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR']]

    elif Current_Month > Mar:
        Nrt_2300_df = Nrt_2300_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR']]
        Nrt_2400_df = Nrt_2400_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR']]

    elif Current_Month > Feb:
        Nrt_2300_df = Nrt_2300_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB']]
        Nrt_2400_df = Nrt_2400_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB']]

    elif Current_Month > Jan:
        Nrt_2300_df = Nrt_2300_df[['NRT Code', "Description", 'YTD', 'JAN']]
        Nrt_2400_df = Nrt_2400_df[['NRT Code', "Description", 'YTD', 'JAN']]

    # # NOW, Append the "2300" and "2400" one-row dataframes to the main NRT df:
    Lonestar_Nrt_Merged = Lonestar_Nrt_Merged.append(Nrt_2300_df)
    Lonestar_Nrt_Merged = Lonestar_Nrt_Merged.append(Nrt_2400_df)
    Lonestar_Nrt_Merged = Lonestar_Nrt_Merged.reset_index().drop(columns=['index'])

    #Convert YTD and Each Month from object to int:
    Lonestar_Nrt_Merged = Lonestar_Nrt_Merged.fillna(0)
    Lonestar_Nrt_Merged.iloc[:, 2:] = Lonestar_Nrt_Merged.iloc[:, 2:].astype(int)
    # ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

    # ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

    # Create the short list that Lonestar Nrt df needs:
    Lonestar_Codes_List = ['2002', '2004', '2005', '2006', '2009', '2010', '2101', '2102', '2205', '2300', '2400']


    #---------------------------------------------------------------------------------------------------------------
    # Assign to variable a filtered merged df that only includes Codes we want according to Lonestar:
    Lonestar_Nrt_Codes_df = Lonestar_Nrt_Merged[Lonestar_Nrt_Merged['NRT Code'].isin(Lonestar_Codes_List)]
    Lonestar_Nrt_Codes_df = Lonestar_Nrt_Codes_df.reset_index(drop=True)

    # Check either df if desired:
    Lonestar_Nrt_Codes_df

    # -------------------------------------------------------------------------------------------------------------------------------------------------------------------
    # NOW we need to create a new row with a Description called  "OTHER" which will be a total of all other codes that were not part of the list above.

    # Assign a TEMPORARY df of all OTHER codes NOT in relevant list:
    Nrt_df_NOT_in_Lonestar_CODES = Lonestar_Nrt_Merged[~Lonestar_Nrt_Merged['NRT Code'].isin(Lonestar_Codes_List)]

    # Make all "NRT Code" values in this new df for OTHER NRT Codes to ALL be OTHER:
    Nrt_df_NOT_in_Lonestar_CODES["NRT Code"] = 'OTHER'

    # Group by ALL of the OTHER codes to sum into ONE ROW:
    Nrt_df_NOT_in_Lonestar_CODES = Nrt_df_NOT_in_Lonestar_CODES.groupby(Nrt_df_NOT_in_Lonestar_CODES['NRT Code']).sum()
    Nrt_df_NOT_in_Lonestar_CODES = Nrt_df_NOT_in_Lonestar_CODES.reset_index()
    Nrt_df_NOT_in_Lonestar_CODES['Description'] = 'OTHER'

    # NOW, Append the "OTHER" df (the one-row df we just created) to the main NRT df:
    Lonestar_Nrt_Codes_df = Lonestar_Nrt_Codes_df.append(Nrt_df_NOT_in_Lonestar_CODES)
    Lonestar_Nrt_Codes_df = Lonestar_Nrt_Codes_df.reset_index().drop(columns=['index'])
 
                    # ------------------------------------------------- Now this will set it up for HTML aesthetically---------------------------------------
    # SUM COLUMNS FOR NUMERIC ONLY!
    Lonestar_Nrt_Codes_df.loc['this_value_will_be_in_the_index_and_thus_removed_in_the_next_step'] = Lonestar_Nrt_Codes_df.sum(numeric_only=True)

    # Reset the index and then drop the extra index column to make it clean:
    Lonestar_Nrt_Codes_df = Lonestar_Nrt_Codes_df.reset_index().drop(columns=['index'])

    # Replace NaNs with empty strings:
    Lonestar_Nrt_Codes_df = Lonestar_Nrt_Codes_df.fillna('')
    Lonestar_Nrt_Codes_df

    # Replace the blank Description with the word "TOTAL", then replace OTHER in row 10 with blank and "OTHER NRT CODES"
    Lonestar_Nrt_Codes_df['Description'] = Lonestar_Nrt_Codes_df["Description"].replace('',"TOTAL")
    Lonestar_Nrt_Codes_df['NRT Code'] = Lonestar_Nrt_Codes_df["NRT Code"].replace('OTHER',"")
    Lonestar_Nrt_Codes_df['Description'] = Lonestar_Nrt_Codes_df["Description"].replace('OTHER',"OTHER NRT CODES")

    # Ensure the YTD TOTAL at the very bottom of the df is a sum of each of the months total:
    Lonestar_Nrt_Codes_df.iloc[-1,2] = Lonestar_Nrt_Codes_df.iloc[-1, 3:].sum()
        # Convert all YTD and months columns to Integers:
    for column in Lonestar_Nrt_Codes_df.columns:
        if column != 'Description' and column != 'NRT Code':
            Lonestar_Nrt_Codes_df[f'{column}'] = Lonestar_Nrt_Codes_df[f'{column}'].astype(int)

    # Save the NRT table of BOTH YEARS.
    Lonestar_Nrt_Codes_df.to_csv(f'./CSVs/Lonestar_NRT_TABLE_{year}.csv', index=False)


Lonestar_Nrt_Codes_df

In [ ]:
# Now, ensure each numerical column in the NRT TABLE has commas as separators:
Lonestar_Nrt_Codes_df_with_COMMAS_and_DASHES_for_HTML = Lonestar_Nrt_Codes_df.copy()
Lonestar_Nrt_Codes_df_with_DASHES = Lonestar_Nrt_Codes_df.copy()

columns_in_df = Lonestar_Nrt_Codes_df_with_COMMAS_and_DASHES_for_HTML.iloc[:,2:].columns
for columns in columns_in_df:
    Lonestar_Nrt_Codes_df_with_COMMAS_and_DASHES_for_HTML[columns] = Lonestar_Nrt_Codes_df_with_COMMAS_and_DASHES_for_HTML.apply(lambda x: "{:,}".format(x[columns]), axis=1)

# Finally, we'll add dashes for months that don't exist for the html table:

if Current_Month < Jan:
    Lonestar_Nrt_Codes_df_with_COMMAS_and_DASHES_for_HTML['JAN'] = '-'
    Lonestar_Nrt_Codes_df_with_DASHES['JAN'] = '-'
if Current_Month <= Feb:
    Lonestar_Nrt_Codes_df_with_COMMAS_and_DASHES_for_HTML['FEB'] = '-'
    Lonestar_Nrt_Codes_df_with_DASHES['FEB'] = '-'
if Current_Month <= Mar:
    Lonestar_Nrt_Codes_df_with_COMMAS_and_DASHES_for_HTML['MAR'] = '-'
    Lonestar_Nrt_Codes_df_with_DASHES['MAR'] = '-'
if Current_Month <= Apr:
    Lonestar_Nrt_Codes_df_with_COMMAS_and_DASHES_for_HTML['APR'] = '-'
    Lonestar_Nrt_Codes_df_with_DASHES['APR'] = '-'
if Current_Month <= May:
    Lonestar_Nrt_Codes_df_with_COMMAS_and_DASHES_for_HTML['MAY'] = '-'
    Lonestar_Nrt_Codes_df_with_DASHES['MAY'] = '-'
if Current_Month <= Jun:
    Lonestar_Nrt_Codes_df_with_COMMAS_and_DASHES_for_HTML['JUN'] = '-'
    Lonestar_Nrt_Codes_df_with_DASHES['JUN'] = '-'
if Current_Month <= Jul:
    Lonestar_Nrt_Codes_df_with_COMMAS_and_DASHES_for_HTML['JUL'] = '-'
    Lonestar_Nrt_Codes_df_with_DASHES['JUL'] = '-'
if Current_Month <= Aug:
    Lonestar_Nrt_Codes_df_with_COMMAS_and_DASHES_for_HTML['AUG'] = '-'
    Lonestar_Nrt_Codes_df_with_DASHES['AUG'] = '-'
if Current_Month <= Sept:
    Lonestar_Nrt_Codes_df_with_COMMAS_and_DASHES_for_HTML['SEPT'] = '-'
    Lonestar_Nrt_Codes_df_with_DASHES['SEPT'] = '-'
if Current_Month <= Oct:
    Lonestar_Nrt_Codes_df_with_COMMAS_and_DASHES_for_HTML['OCT'] = '-'
    Lonestar_Nrt_Codes_df_with_DASHES['OCT'] = '-'
if Current_Month <= Nov:
    Lonestar_Nrt_Codes_df_with_COMMAS_and_DASHES_for_HTML['NOV'] = '-'
    Lonestar_Nrt_Codes_df_with_DASHES['NOV'] = '-'
if Current_Month <= Dec:
    Lonestar_Nrt_Codes_df_with_COMMAS_and_DASHES_for_HTML['DEC'] = '-'
    Lonestar_Nrt_Codes_df_with_DASHES['DEC'] = '-'


# Check the df that only has dashes to be used for graphing (we'll use the dashes to our advantage to remove incompleted month columns):
Lonestar_Nrt_Codes_df_with_DASHES
# Or check the df that has commas and dashes used for html (the commas, of course, won't appear unless 4 or more digits in a value):
Lonestar_Nrt_Codes_df_with_COMMAS_and_DASHES_for_HTML

## Sort the NRT Table for the html:

In [ ]:
Lonestar_NRT_df_sorted_for_html = Lonestar_Nrt_Codes_df_with_COMMAS_and_DASHES_for_HTML.copy()

# Create a temp varaible that ONLY holds the "TOTALS" row:
temp_TOTALS_row = Lonestar_NRT_df_sorted_for_html.loc[Lonestar_NRT_df_sorted_for_html['Description'] == 'TOTAL', :]

# Assign ALL rows and columns EXCEPT the TOTALS row:
Lonestar_NRT_df_sorted_for_html = Lonestar_NRT_df_sorted_for_html.iloc[0:-1, :]
Lonestar_NRT_df_sorted_for_html['YTD'] = Lonestar_NRT_df_sorted_for_html['YTD'].astype(int)
# Sort all rows baased on the the YTD column:
Lonestar_NRT_df_sorted_for_html = Lonestar_NRT_df_sorted_for_html.sort_values('YTD', ascending=False)
Lonestar_NRT_df_sorted_for_html = Lonestar_NRT_df_sorted_for_html.reset_index(drop=True)

# Add the "TOTALS" row back into the df:
Lonestar_NRT_df_sorted_for_html = pd.concat([Lonestar_NRT_df_sorted_for_html, temp_TOTALS_row], ignore_index=True)

# For NRT Code of "OTHER NRT CODES", let's add a dash since there's no NRT Code for that:
Lonestar_NRT_df_sorted_for_html.loc[Lonestar_NRT_df_sorted_for_html['Description'] == 'OTHER NRT CODES', 'NRT Code'] = '-'

Lonestar_NRT_df_sorted_for_html

In [ ]:
# ------------------------------------------------------------------

# BUILD THE HTML TABLE for the NRT TABLE:

def create_html_table(Nrt_table):
    
    # Create headers for HTML
    row_data = '<!-- Start NRT Table --><thead>\n<tr class="NRT_header">'

    # Create columns for HTML:    
    for column in range(Nrt_table.shape[1]):
        adjust_header_css = "NRT_header"
        if column not in [2]:
            adjust_header_css += " black_color make_bold make_text_white"
        if column in [2]:
            adjust_header_css += " lonestar_gold_color make_bold"
        if column not in []:
            adjust_header_css += " center"    
        row_data += '\n <th class = "'+adjust_header_css+'">'+Lonestar_NRT_df_sorted_for_html.columns[column]+'</th>'
    row_data += '\n</tr> \n </head> \n <tbody>'
    
    # Create Body (rows for table) for html
    for row in range(Nrt_table.shape[0]):
        if row in [0,1,2,3,4,5,6,7,8,9,10,11,12]:
            row_data += '\n<tr class = "make_bold">'
        else: row_data += '\n<tr>'
        for column in range(Nrt_table.shape[1]):
            td_class_name = 'NRT_text_column'
            if column not in []:
                td_class_name += " center"
            if row in [12]:
                td_class_name += " black_color make_bold make_text_white"
            # if column not in [2] and row in [13]:
            #     td_class_name += " blue_color make_text_white"
            if column in [2]:
                td_class_name += " lonestar_gold_color make_bold make_text_black"
            row_data += '\n <td class = "'+td_class_name+'">'+str(Nrt_table.iloc[row,column])+'</td>'
    row_data += '\n</tr> \n</tbody><!-- End NRT Table -->'
                # -------------------------------------------------------------------------------------------------

    return row_data

# Call the function:
html_table2 = create_html_table(Lonestar_NRT_df_sorted_for_html)

## Now create NRT Year-to_Year Line GRAPH:

In [ ]:

# Pull back in last year's data and reformat it again:
if os.path.isfile(f'CSVs/LONESTAR_NRT_TABLE_{Last_Year}.csv'):
    Last_Year_NRT_df = pd.read_csv(f'CSVs/LONESTAR_NRT_TABLE_{Last_Year}.csv')
    Last_Year_NRT_df['NRT Code'] = Last_Year_NRT_df['NRT Code'].fillna(0)
    Last_Year_NRT_df['NRT Code'] = Last_Year_NRT_df['NRT Code'].astype(int)
    Last_Year_NRT_df.iloc[-2:, 0] = ''

Last_Year_NRT_df
# Note, you already have this year's data you need, you can use the Swift_Nrt_Codes_df dataframe.

In [ ]:
# Drop YTD column for each df:
Last_Year_NRT_df = Last_Year_NRT_df.drop(columns=['YTD'])
Current_Year_NRT_df = Lonestar_Nrt_Codes_df_with_DASHES.drop(columns=['YTD'])

# Rename the columns on Each Year's data so it shows not only the month but also the year:
    # Last Year:
Last_Year_NRT_df.columns = ['NRT Code', 'Description', f'JAN {Last_Year}', f'FEB {Last_Year}', f'MAR {Last_Year}', f'APR {Last_Year}', f'MAY {Last_Year}', f'JUN {Last_Year}', f'JUL {Last_Year}', f'AUG {Last_Year}', f'SEPT {Last_Year}', f'OCT {Last_Year}', f'NOV {Last_Year}', f'DEC {Last_Year}']
    # Current Year:
Current_Year_NRT_df.columns = ['NRT Code', 'Description', f'JAN {Current_Year}', f'FEB {Current_Year}', f'MAR {Current_Year}', f'APR {Current_Year}', f'MAY {Current_Year}', f'JUN {Current_Year}', f'JUL {Current_Year}', f'AUG {Current_Year}', f'SEPT {Current_Year}', f'OCT {Current_Year}', f'NOV {Current_Year}', f'DEC {Current_Year}']

# For the Current Year, drop columns with dashes:
Current_Year_NRT_df = Current_Year_NRT_df.loc[:, ~(Current_Year_NRT_df.astype(str) == '-').all()]

Current_Year_NRT_df

In [ ]:
# To merge, we ONLY want "Category" on ONE df.  We'll drop "NRT Code" and "Description" on ALL dfs that are not the oldest year we're testing.
Current_Year_NRT_df = Current_Year_NRT_df.drop(columns = ['NRT Code', 'Description'])

# Now add a new TEMPORARY column for ALL dfs we're testing which will be ALL unique values but identical for each df, so we can merge "on" that temporary column
Last_Year_NRT_df['Temp Col'] = range(len(Last_Year_NRT_df))
Current_Year_NRT_df['Temp Col'] = range(len(Current_Year_NRT_df))

# Merge each year together into one DF:
Merged_NRT_df = pd.merge(Last_Year_NRT_df, Current_Year_NRT_df, how='outer', on='Temp Col')

# Now delete the "Temp Col" from the merged df:
Merged_NRT_df = Merged_NRT_df.drop(columns = ['Temp Col'])

pd.set_option('display.max_columns', None)
Merged_NRT_df

In [ ]:
# Transpose the Merged_Service_df to make it easier for graphing:
Merged_NRT_df_TRANSPOSED = Merged_NRT_df.T

Description_list_from_second_row = list(Merged_NRT_df_TRANSPOSED.iloc[1])
Merged_NRT_df_TRANSPOSED = Merged_NRT_df_TRANSPOSED.iloc[2:, :]
Merged_NRT_df_TRANSPOSED.columns = Description_list_from_second_row

Merged_NRT_df_TRANSPOSED = Merged_NRT_df_TRANSPOSED.reset_index()
Merged_NRT_df_TRANSPOSED = Merged_NRT_df_TRANSPOSED.rename(columns = {'index': 'month-yr'})
Merged_NRT_df_TRANSPOSED = Merged_NRT_df_TRANSPOSED.drop(columns=['TOTAL'])


Merged_NRT_df_TRANSPOSED

In [ ]:
# Setup independent X Variables (which is just the month and save a list):
Month_LIST = Merged_NRT_df_TRANSPOSED['month-yr'].to_list()


#Setup a dependent Y Variables (By Creating separate lists for each type of service):

Sidewall_Cut_List =  Merged_NRT_df_TRANSPOSED['Sidewall Cut(s)/Snag(s)'].to_list()
Run_Under_Inflated_List = Merged_NRT_df_TRANSPOSED["Run Under-Inflated"].to_list()
Run_to_Destruction_List = Merged_NRT_df_TRANSPOSED["Run to Destruction"].to_list()
Impact_Split_List =  Merged_NRT_df_TRANSPOSED["Impact Split(s)"].to_list()
Pinch_Shock_List =  Merged_NRT_df_TRANSPOSED["Pinch Shock"].to_list()
Rock_Drilling_List =  Merged_NRT_df_TRANSPOSED["Rock Drilling"].to_list()
Bead_Damage_Mount_Dismount_R_List =  Merged_NRT_df_TRANSPOSED["Bead Damage - Mount/Dismount R"].to_list()
Worn_Too_Far_List =  Merged_NRT_df_TRANSPOSED["Worn Too Far"].to_list()
Shoulder_Separation_List =  Merged_NRT_df_TRANSPOSED["Shoulder Separation"].to_list()
Outside_Of_Fleet_Specifications_List =  Merged_NRT_df_TRANSPOSED["Outside of Fleet Specifications"].to_list()
Injuries_Too_Close_or_Large_List =  Merged_NRT_df_TRANSPOSED["Injuries too Close/Large"].to_list()
Other_NRT_Codes_List =  Merged_NRT_df_TRANSPOSED["OTHER NRT CODES"].to_list()

fig = go.Figure()

# Create and Style Traces:
fig.add_trace(go.Scatter(x=Month_LIST, y=Sidewall_Cut_List, name = 'Sidewall Cut(s)/Snag(s)',
                         line=dict(color='red', width=2.2),
                         # change the marker style to be so small it's unseen
                        marker=dict(symbol='diamond', size=2, line=dict(width=0))))
fig.add_trace(go.Scatter(x=Month_LIST, y=Run_Under_Inflated_List, name = 'Run Under-Inflated',
                         line=dict(color='orange', width=2.2),
                         # change the marker style to be so small it's unseen
                        marker=dict(symbol='diamond', size=2, line=dict(width=0))))
fig.add_trace(go.Scatter(x=Month_LIST, y=Run_to_Destruction_List, name = "Run to Destruction",
                         line=dict(color='blue', width=2.2),
                         # change the marker style to be so small it's unseen
                        marker=dict(symbol='diamond', size=2, line=dict(width=0))))
fig.add_trace(go.Scatter(x=Month_LIST, y=Pinch_Shock_List, name = "Impact Split(s)",
                         line=dict(color='purple', width=2.2),
                         # change the marker style to be so small it's unseen
                        marker=dict(symbol='diamond', size=2, line=dict(width=0))))
fig.add_trace(go.Scatter(x=Month_LIST, y=Worn_Too_Far_List, name = "Pinch Shock",
                         line=dict(color='darkgreen', width=2.2),
                         # change the marker style for this one to "triangle-up"
                        marker=dict(symbol='diamond', size=2, line=dict(width=0))))

fig.add_trace(go.Scatter(x=Month_LIST, y=Rock_Drilling_List, name = "Rock Drilling",
                         line=dict(color='brown', width=2.2, dash='dot'),
                         # change the marker style for this one to "triangle-up"
                        marker=dict(symbol='diamond', size=6, line=dict(width=0))))
fig.add_trace(go.Scatter(x=Month_LIST, y=Bead_Damage_Mount_Dismount_R_List, name = "Bead Damage - Mount/Dismount R",
                         line=dict(color='orange', width=2.2, dash='dot'),
                         # change the marker style for this one to "triangle-up"
                        marker=dict(symbol='diamond', size=6, line=dict(width=0))))
fig.add_trace(go.Scatter(x=Month_LIST, y=Worn_Too_Far_List, name = 'Worn Too Far',
                         line=dict(color='royalblue', width=2.2, dash='dot'),
                         # change the marker style for this one to "triangle-up"
                        marker=dict(symbol='diamond', size=6, line=dict(width=0))))
fig.add_trace(go.Scatter(x=Month_LIST, y=Shoulder_Separation_List, name = 'Shoulder Separation',
                         line=dict(color='purple', width=2.2, dash='dot'),
                         # change the marker style for this one to "triangle-up"
                        marker=dict(symbol='diamond', size=6, line=dict(width=0))))
fig.add_trace(go.Scatter(x=Month_LIST, y=Outside_Of_Fleet_Specifications_List, name = 'Outside of Fleet Specifications',
                         line=dict(color='darkgreen', width=2.2, dash='dot'),
                         # change the marker style for this one to "triangle-up"
                        marker=dict(symbol='diamond', size=6, line=dict(width=0))))
fig.add_trace(go.Scatter(x=Month_LIST, y=Injuries_Too_Close_or_Large_List, name = 'Injuries too Close/Large',
                         line=dict(color='dimgray', width=3),
                         # change the marker style for this one to "star"
                        marker=dict(symbol='triangle-up', size=6, line=dict(width=0))))
fig.add_trace(go.Scatter(x=Month_LIST, y=Other_NRT_Codes_List, name = 'OTHER NRT CODES',
                         line=dict(color='turquoise', width=2.2),
                         # change the marker style for this one to "triangle-up"
                        marker=dict(symbol='diamond', size=2, line=dict(width=0))))



# Edit the Layout:
fig.update_layout(
    title_text=f"<b>SCRAP TREND  {Last_Year} - {Current_Year}</b>", 
    title_x= 0.5,
    title_font_size=29,
    title_font_color="black",
    xaxis_title= "Month-Year",
    yaxis_title= 'NUMBER OF NRT',
    autosize=False, width=1445, height=1330,
    showlegend=True,
     legend=dict(
        x=0,
        y=-.76,
        title_font_family="Times New Roman",
        font=dict(
            family=None,
            size=23,
            color="black"
        ),
        bgcolor="white",
        bordercolor="black",
        borderwidth=.03
    ),
    # General axes and axes labels fonts:
    font=dict(
        family='Arial',
        size=19,
        color="black"
    ))

fig.add_vline(x=11.5, line_width=6, line_dash="dash", line_color="green")

# For aesthetics, force the graph to display the zero point near the bottom IF THERE ARE NO VALUES.  Otherwise, plotly will automatically put the zero point in the middle.
# if (sum(Crown_Cut_List) + sum(Sidewall_Cut_List) + sum(Run_Under_Inflated_List) + sum(Impact_Split_List) + sum(Bead_Damage_Mount_Dismount_List) + sum(Worn_Too_Far_List) + sum(Belt_Separation_Radial_Only_List) + sum(Bead_Separation_Splitting_List) + sum(Shoulder_Separation_List) + sum(Sidewall_Blisters_Or_Bumps_List) + sum(Outside_Of_Fleet_Specifications_List) + sum(Other_NRT_Codes_List)) == 0:
#     fig.layout['yaxis1'] = {'range': [-.5, 5]}


fig.add_vrect(x0=0, x1=12, 
              annotation_text="<b>LAST YEAR<b>", annotation_position="top left",
              annotation=dict(font_size=30, font_family="Helvetica"),
              fillcolor=None, opacity=0.25, line_width=0)

fig.add_vrect(x0=12, x1=12, 
              annotation_text="<b>CURRENT YEAR<b>", annotation_position="top left",
              annotation=dict(font_size=30, font_family="Helvetica"),
              fillcolor=None, opacity=0.15, line_width=0)



# Change grid color and axis colors
fig.update_xaxes(gridwidth=1, gridcolor='white')
fig.update_xaxes(gridwidth=1, gridcolor='white')
# fig.update_yaxes(showline=True, linewidth=2, linecolor='orange', gridcolor='Red')


# file_name = f"{Region_Name}_Service_Trend_Analysis"
fig.write_image(f'Images_graphs/NRT_Graph_{Last_Year}-{Current_Year}.svg')
fig.show()






# ============================================================================================================================================
# ============================================================================================================================================
# ============================================================================================================================================
# ============================================================================================================================================
# NOW that we've created to version for the initial index.html for the pdf... we can now create the image version with a smaller size along with interactivity for plotly for web page version:

fig2 = go.Figure()

# Create and Style Traces:
fig2.add_trace(go.Scatter(x=Month_LIST, y=Sidewall_Cut_List, name = 'Sidewall Cut(s)/Snag(s)',
                         line=dict(color='red', width=2),
                         # change the marker style to be so small it's unseen
                        marker=dict(symbol='diamond', size=2, line=dict(width=0))))
fig2.add_trace(go.Scatter(x=Month_LIST, y=Run_Under_Inflated_List, name = 'Run Under-Inflated',
                         line=dict(color='orange', width=2),
                         # change the marker style to be so small it's unseen
                        marker=dict(symbol='diamond', size=2, line=dict(width=0))))
fig2.add_trace(go.Scatter(x=Month_LIST, y=Run_to_Destruction_List, name = "Run to Destruction",
                         line=dict(color='blue', width=2),
                         # change the marker style to be so small it's unseen
                        marker=dict(symbol='diamond', size=2, line=dict(width=0))))
fig2.add_trace(go.Scatter(x=Month_LIST, y=Pinch_Shock_List, name = "Impact Split(s)",
                         line=dict(color='purple', width=2),
                         # change the marker style to be so small it's unseen
                        marker=dict(symbol='diamond', size=2, line=dict(width=0))))
fig2.add_trace(go.Scatter(x=Month_LIST, y=Worn_Too_Far_List, name = "Pinch Shock",
                         line=dict(color='darkgreen', width=2),
                         # change the marker style for this one to "triangle-up"
                        marker=dict(symbol='diamond', size=2, line=dict(width=0))))

fig2.add_trace(go.Scatter(x=Month_LIST, y=Rock_Drilling_List, name = "Rock Drilling",
                         line=dict(color='brown', width=2, dash='dot'),
                         # change the marker style for this one to "triangle-up"
                        marker=dict(symbol='diamond', size=6, line=dict(width=0))))
fig2.add_trace(go.Scatter(x=Month_LIST, y=Bead_Damage_Mount_Dismount_R_List, name = "Bead Damage - Mount/Dismount R",
                         line=dict(color='orange', width=2, dash='dot'),
                         # change the marker style for this one to "triangle-up"
                        marker=dict(symbol='diamond', size=6, line=dict(width=0))))
fig2.add_trace(go.Scatter(x=Month_LIST, y=Worn_Too_Far_List, name = 'Worn Too Far',
                         line=dict(color='royalblue', width=2, dash='dot'),
                         # change the marker style for this one to "triangle-up"
                        marker=dict(symbol='diamond', size=6, line=dict(width=0))))
fig2.add_trace(go.Scatter(x=Month_LIST, y=Shoulder_Separation_List, name = 'Shoulder Separation',
                         line=dict(color='purple', width=2, dash='dot'),
                         # change the marker style for this one to "triangle-up"
                        marker=dict(symbol='diamond', size=6, line=dict(width=0))))
fig2.add_trace(go.Scatter(x=Month_LIST, y=Outside_Of_Fleet_Specifications_List, name = 'Outside of Fleet Specifications',
                         line=dict(color='darkgreen', width=2, dash='dot'),
                         # change the marker style for this one to "triangle-up"
                        marker=dict(symbol='diamond', size=6, line=dict(width=0))))
fig2.add_trace(go.Scatter(x=Month_LIST, y=Injuries_Too_Close_or_Large_List, name = 'Injuries too Close/Large',
                         line=dict(color='dimgray', width=2),
                         # change the marker style for this one to "star"
                        marker=dict(symbol='triangle-up', size=6, line=dict(width=0))))
fig2.add_trace(go.Scatter(x=Month_LIST, y=Other_NRT_Codes_List, name = 'OTHER NRT CODES',
                         line=dict(color='turquoise', width=2),
                         # change the marker style for this one to "triangle-up"
                        marker=dict(symbol='diamond', size=2, line=dict(width=0))))



# Edit the Layout:
fig2.update_layout(
    title_text=f"<b>SCRAP TREND  {Last_Year} - {Current_Year}</b>", 
    title_x= 0.5,
    title_font_size=20,
    title_font_color="black",
    xaxis_title= "Month-Year",
    yaxis_title= 'NUMBER OF NRT',
    autosize=False, width=1100, height=1020,
    showlegend=True, 
     legend=dict(
        x=0,
        y=-.9,
        title_font_family="Times New Roman",
        font=dict(
            family=None,
            size=19,
            color="black"
        ),
        bgcolor="white",
        bordercolor="black",
        borderwidth=.03
    ),
    # General axes and axes labels fonts:
    font=dict(
        family='Arial',
        size=13,
        color="black"
    ))

fig2.add_vline(x=11.5, line_width=6, line_dash="dash", line_color="green")

# For aesthetics, force the graph to display the zero point near the bottom IF THERE ARE NO VALUES.  Otherwise, plotly will automatically put the zero point in the middle.
# if (sum(Crown_Cut_List) + sum(Sidewall_Cut_List) + sum(Run_Under_Inflated_List) + sum(Impact_Split_List) + sum(Bead_Damage_Mount_Dismount_List) + sum(Worn_Too_Far_List) + sum(Belt_Separation_Radial_Only_List) + sum(Bead_Separation_Splitting_List) + sum(Shoulder_Separation_List) + sum(Sidewall_Blisters_Or_Bumps_List) + sum(Outside_Of_Fleet_Specifications_List) + sum(Other_NRT_Codes_List)) == 0:
#     fig.layout['yaxis1'] = {'range': [-.5, 5]}


fig2.add_vrect(x0=0, x1=12, 
              annotation_text="<b>LAST YEAR<b>", annotation_position="top left",
              annotation=dict(font_size=20, font_family="Helvetica"),
              fillcolor=None, opacity=0.25, line_width=0)

fig2.add_vrect(x0=12, x1=12, 
              annotation_text="<b>CURRENT YEAR<b>", annotation_position="top left",
              annotation=dict(font_size=20, font_family="Helvetica"),
              fillcolor=None, opacity=0.15, line_width=0)


# PLOTLY INTERACTIVE:
Scrap_Trend_graph = fig2
Scrap_Trend_graph_for_iframe = py.plot(Scrap_Trend_graph, filename = 'Scrap_Trend_graph', auto_open=False)

fig2.show()




# First, Create the index.html for JUST the pdf:

In [ ]:
Create_html = f"""

<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <meta http-equiv="X-UA-Compatible" content="ie=edge" />
    <!--Bootstrap CSS using CDN-->
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" />
    <link rel="stylesheet" href="css/main.css">

    <title>Report</title>
</head>

<body>
    <!--------------------------------------INTRO PDF PAGE----------------------------------------------------------------->
    <!-- STM LOGO -->
    <br>
    <br>
    <div class="STM_logo">
        <img src=".\Images_logos\STM_logo.svg">
    </div>
    <br>
    <br>
    <br>
    <br>
    <!--YEAR TO DATE BANNER text-->
    <h1 class="first_banner_text">YEAR TO DATE - {Last_Month_Descriptive_Short} {Current_Year}</h1>
    <br>
    <br>
    <br>
    <br>
    <br>
    <!-- Lonestar LOGO -->
    <div class="Customer_logo_image">
        <img src='./Images_logos/Lonestar_logo.jpg'>
    </div>
    <br>
    <br>
    <!--  Second Blue Banner Text -->
    <h1 class="second_banner_text">PURCHASE AND SCRAP PROFILE</h1>
    <br>
    <!--  BRIDGESTONE AND BANDAG LOGOS-->
    <div class = "bridgestone_and_bandag_logo_container">
    <img class= "bridgestone_logo" src= "./Images_logos/Bridgestone_logo.svg">
    <img class= "bandag_logo" src= "./Images_logos/Bandag_logo.svg">
    </div> 
    <!-----------------------------PURCHASES SUMMARY TABLE pdf page--------------------------------------------------------------->
    <br>
    <br>
    <br>
    <h1 class=page_title>{Current_Year} YTD PURCHASES</h1>
    <br>
    <br>
    <br>
    <br>
    <table>
        <!-- Start Sales Table -->
        [[DATA_MAIN_TABLE]]
        <!-- End Sales Table -->
    </table>
    <br>
    <br>
    <br>
    <br>
    <br>
    <br>
    <br>
    <br>
    <br>
    <br>
    <br>
    <br>
    <!----------------------------------------- SALES BREAKDOWN PERCENTAGE COLUMN GRAPHS pdf page --------------------------------------->
    <br>
    <div class = "sales_percentage_breakdown_bar">
        <img src="Images_graphs/{Current_Year}_Sales_Percentage_Breakdown.svg">
        <br>
        <br>
        <br>
        <br>
        <br>
        <br>
        <img src="Images_graphs/{Current_Year}_Retreads_Percentage_Breakdown.svg">
    </div>
    <br>
    <!----------------------------------------- TREND LINE GRAPHS pdf page --------------------------------------------------------------------->
    <div class = "fit_three_graphs_on_pdf_page">
        <img src="Images_graphs/{Current_Year}_Service_Purchase_Analysis.svg">
        <img src="Images_graphs/{Current_Year}_Bandag_Purchase_Analysis.svg">
        <img src="Images_graphs/{Current_Year}_Bridgestone_Purchase_Analysis.svg">
    </div>
    <!--------------------------- SCRAPS ANALYSIS pdf page ---------------------------------------------------------------------------------------->
    <h1 class=page_title>{Current_Year} YTD SCRAP</h1>
    <br>
    <table>
        <!-- Start NRT Table -->
        [[DATA_MAIN_TABLE]]
        <!-- End NRT Table -->
    </table>
    <br>
    <br>
    <div class = "NRT_year_to_year_line_graph">
        <img src='Images_graphs/NRT_Graph_{Last_Year}-{Current_Year}.png'>
    </div>

</body>

</html>

"""
# ------------------------------------------------------------------------------------------------------------------------------
# ADD THE SALES TABLE:
with open('index.html', 'w') as f:
    f.write(re.sub(r"<\!-- Start Sales Table -->.*<\!-- End Sales Table -->", html_table, Create_html, flags=re.DOTALL))

# Then add NRT TABLE:
index_exists = os.path.exists('index.html')
current_index = ''
with open('index.html', 'r') as f:
    current_index = f.read()
with open('index.html', 'w') as f:
    f.write(re.sub(r"<\!-- Start NRT Table -->.*<\!-- End NRT Table -->",
            html_table2, current_index, flags=re.DOTALL))



# BUILD THE PDF:

In [ ]:
# !pip3 install pdfkit
import pdfkit
options = {"enable-local-file-access": None}

# Save the a pdf with a name according to whether it's the current year or a previous year AND whether it's for all districts of a region or if it's for a subregion of desert mountain:

# ------------ If running the report for the current year ----------------------:
pdfkit.from_file("index.html", f"{Last_Month}_Lonestar_YTD_Purchase_and_Scrap_Report.pdf", options=options)

# Now that we have the PDF done, we'll RECREATE the index.html so it is more conducive to an interactive website
# for both sizes of graphs and INTERACTIVITY of plotly graphs:

### First, though, let's create some new html files for a navigation bar:

In [ ]:
# Create 3 new html files and write them as blank.
# If they already exist before this code, when you run this cell each will be overwritten as blank:
with open('index.html', 'w') as new_file:
    new_file.write('')

with open('Sales_Table.html', 'w') as new_file:
    new_file.write('')

with open('Sales_Breakdown_Column_Charts.html', 'w') as new_file:
    new_file.write('')

with open('Trend_Analysis_Line_Charts.html', 'w') as new_file:
    new_file.write('')

with open('BFI_Scraps_Analysis.html', 'w') as new_file:
    new_file.write('')

with open('Data_Downloads.html', 'w') as new_file:
    new_file.write('')

# NOW, we'll need to create EACH page of the website separately.  The next few cells will pertain to the following:
## 1. Home page
## 2. Sales Summary Table
## 3. Sales Breakdown Graphs
## 4. Sales Trend Line Graphs
## 5. BFI Scrap Trend Analysis

#### Note:  Every page will also include the Nav Bar to navigate to all of those pages.

# First, BUILD HOME PAGE:

In [ ]:
# Build Home Page (this will recreate index.html but NOT formatted for the pdf (since we already built that pdf)...instead it will be formatted for the website)
# This will mostly just be the home page logos, intro, etc.

Create_html = f"""

<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <meta http-equiv="X-UA-Compatible" content="ie=edge" />
    <!--Bootstrap CSS using CDN-->
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" />
    <link rel="stylesheet" href="css/main.css">

    <title>Report</title>
</head>

<body class = 'container'>
    <header>
        <nav>
            <ul class="./index_NAV_BAR_files/nav_bar_for_website">
                <li><a href="index.html">Home</a></li>
                <li><a href="Sales_Table.html">Sales Summary Table</a></li>
                <li><a href="Sales_Breakdown_Column_Charts.html">Sales Breakdown</a></li>
                <li><a href="Trend_Analysis_Line_Charts.html">Sales Trend Lines</a></li>
                <li><a href="BFI_Scraps_Analysis.html">BFI Scraps Trend Analysis</a></li>
                <li><a href="Data_Downloads.html">Data Downloads</a></li>
            </ul>
        </nav>
    </header>
    <!--------------------------------------HOME PAGE----------------------------------------------------------------->
    <!-- STM LOGO -->
    <div class="STM_logo_for_website">
        <img src=".\Images_logos\STM_logo.svg">
    </div>
    <br>
    <br>
    <br>
    <br>
    <!--YEAR TO DATE BANNER text-->
    <h1 class="first_banner_text_for_website">YEAR TO DATE - {Last_Month_Descriptive_Short} {Current_Year}</h1>
    <br>
    <br>
    <br>
    <br>
    <br>
    <!-- Lonestar LOGO -->
    <div class="Customer_logo_image_for_website">
        <img src='./Images_logos/Lonestar_logo.jpg'>
    </div>
    <br>
    <br>
    <!--  Second Blue Banner Text -->
    <h1 class="second_banner_text_for_website">PURCHASE AND SCRAP PROFILE</h1>
    <br>
    <!--  BRIDGESTONE AND BANDAG LOGOS-->
    <div class = "bridgestone_and_bandag_logo_container_for_website">
    <img class= "bridgestone_logo_for_website" src= "./Images_logos/Bridgestone_logo.svg">
    <img class= "bandag_logo_for_website" src= "./Images_logos/Bandag_logo.svg">
    </div> 
</body>

</html>

"""
# ------------------------------------------------------------------------------------------------------------------------------
with open('index.html', 'w') as f:
    f.write(Create_html)


# # Then add NRT TABLE:
# index_exists = os.path.exists('index.html')
# current_index = ''
# with open('index.html', 'r') as f:
#     current_index = f.read()
# with open('index.html', 'w') as f:
#     f.write(re.sub(r"<\!-- Start NRT Table -->.*<\!-- End NRT Table -->",
#             html_table2, current_index, flags=re.DOTALL))

# 2. Build the Sales Summary Table web page:

In [ ]:
# For the html table only, we'll get rid of the name "Category" as the first column since we want that part blank in the final pdf table:
df_just_for_html = df_DASHES_for_HTML.rename(
    columns={'Category': ''})
# If we don't want the "SERVICE" category to have totals, we can replace all the totals with empty strings:
df_just_for_html.iloc[0, 1:] = ''

# BUILD THE HTML TABLE for the MAIN TABLE:

def create_html_table(Primary_table):

    # Create headers for HTML
    row_data = '<!-- Start Sales Table --><thead>\n<tr class="header_for_website">'

    # Create columns for HTML:    
    for column in range(Primary_table.shape[1]):
        adjust_header_css = "header_for_website"
        if column in [0,1]:
            adjust_header_css += " make_white_for_website"
        if column not in [0, 1]:
            adjust_header_css += " lonestar_gold_color_for_website"
        if column not in []:
            adjust_header_css += " center_for_website"    
        row_data += '\n <th class = "'+adjust_header_css+'">'+df_just_for_html.columns[column]+'</th>'
    row_data += '\n</tr> \n </head> \n <tbody>'
    
    # Create Body (rows for table) for html
    for row in range(Primary_table.shape[0]):
        if row in [0, 6, 9, 13, 14, 28, 29, 30]:
            row_data += '\n<tr class = "lonestar_gold_color_for_website make_bold_for_website">'
        else: row_data += '\n<tr>'
        for column in range(Primary_table.shape[1]):
            td_class_name = 'text_column_for_website'
            if column in [0] and row in [1, 2, 3, 4, 5, 7, 8, 10, 11, 12, 16, 17, 18, 19, 20, 21, 22, 28, 29, 30]:
                td_class_name += " right_align_for_website"
            if column not in [0]:
                td_class_name += " center_for_website"
            if column in [0] and row in [0, 6, 9, 13, 14, 15, 23, 24, 25, 26, 27]:
                td_class_name += " make_bold left_align_for_website"
            if column not in [0] and row in [14, 15]:
                td_class_name += " lonestar_gold_color_for_website"
            if column in [0] and row in [28, 29, 30]:
                td_class_name += " make_bold right_align_for_website"
            if column not in [] and row in [15]:
                td_class_name += " make_bold_for_website"            
            row_data += '\n <td class = "'+td_class_name+'">'+str(Primary_table.iloc[row,column])+'</td>'
    row_data += '\n</tr> \n</tbody><!-- End Sales Table -->'
    # -------------------------------------------------------------------------------------------------

    return row_data

# Call the function:
html_sales_table_for_website = create_html_table(df_just_for_html)

Create_html = f"""

<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <meta http-equiv="X-UA-Compatible" content="ie=edge" />
    <!--Bootstrap CSS using CDN-->
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" />
    <link rel="stylesheet" href="css/main.css">

    <title>Report</title>
</head>

<body class = 'container'>
    <header>
        <nav>
            <ul class="./index_NAV_BAR_files/nav_bar_for_website">
                <li><a href="index.html">Home</a></li>
                <li><a href="Sales_Table.html">Sales Summary Table</a></li>
                <li><a href="Sales_Breakdown_Column_Charts.html">Sales Breakdown</a></li>
                <li><a href="Trend_Analysis_Line_Charts.html">Sales Trend Lines</a></li>
                <li><a href="BFI_Scraps_Analysis.html">BFI Scraps Trend Analysis</a></li>
                <li><a href="Data_Downloads.html">Data Downloads</a></li>
            </ul>
        </nav>
    </header>
<!-------------------------SALES SUMMARY TABLE PAGE---------------------------------------------->
    <br>
    <br>
    <h1 class=page_title_for_website>{Current_Year} YTD PURCHASES</h1>
    <br>
    <table class = "make_table_smaller_for_website">
        <!-- Start Sales Table -->
        [[DATA_MAIN_TABLE]]
        <!-- End Sales Table -->
    </table>

</body>

</html>

"""

# ADD THE SALES TABLE:
# index_exists = os.path.exists('./index_NAV_BAR_files/Sales_Table.html')
# current_index = ''
# with open('./index_NAV_BAR_files/Sales_Table.html', 'r') as f:
#     current_index = f.read()
# with open('./index_NAV_BAR_files/Sales_Table.html', 'w') as f:
#     f.write(re.sub(r"<\!-- Start NRT Table -->.*<\!-- End NRT Table -->",
#             html_table2, current_index, flags=re.DOTALL))

# ADD THE SALES TABLE:
html_page_exists = os.path.exists('Sales_Table.html')
current_index = ''
with open('Sales_Table.html', 'r') as f:
    current_index = f.read()
with open('Sales_Table.html', 'w') as f:
    f.write(re.sub(r"<\!-- Start Sales Table -->.*<\!-- End Sales Table -->", html_sales_table_for_website, Create_html, flags=re.DOTALL))

print(html_page_exists)

# 3. Build SALES BREAKDOWN percentage graphs page for website:

In [ ]:
Create_html_for_sales_breakdown = f"""

<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <meta http-equiv="X-UA-Compatible" content="ie=edge" />
    <!--Bootstrap CSS using CDN-->
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" />
    <link rel="stylesheet" href="css/main.css">

    <title>Report</title>
</head>

<body class = 'container'>
    <header>
        <nav>
            <ul class="./index_NAV_BAR_files/nav_bar_for_website">
                <li><a href="index.html">Home</a></li>
                <li><a href="Sales_Table.html">Sales Summary Table</a></li>
                <li><a href="Sales_Breakdown_Column_Charts.html">Sales Breakdown</a></li>
                <li><a href="Trend_Analysis_Line_Charts.html">Sales Trend Lines</a></li>
                <li><a href="BFI_Scraps_Analysis.html">BFI Scraps Trend Analysis</a></li>
                <li><a href="Data_Downloads.html">Data Downloads</a></li>
            </ul>
        </nav>
    </header>
   <!----------------------------------------- SALES BREAKDOWN PERCENTAGE COLUMN GRAPHS for web page --------------------------------------->
    <br>
    <h1 class = "page_title_smaller_for_website">PURCHASES BREAKDOWN BY CATEGORY</h1>
    <br>
    <br>
    <br>
    <div class = "sales_percentage_breakdown_bar_for_website">
        <img src="Images_graphs/{Current_Year}_Sales_Percentage_Breakdown.svg">
        <br>
        <br>
        <br>
        <img src="Images_graphs/{Current_Year}_Retreads_Percentage_Breakdown.svg">
    </div>
</body>

</html>

"""

# html_page_exists = os.path.exists('Sales_Table.html')
# current_index = ''
# with open('Sales_Table.html', 'r') as f:
#     current_index = f.read()

with open('Sales_Breakdown_Column_Charts.html', 'w') as new_file:
    new_file.write(Create_html_for_sales_breakdown)

# Build webpage for TREND LINE GRAPHS:

In [ ]:
Create_html_for_trend_lines = f"""

<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <meta http-equiv="X-UA-Compatible" content="ie=edge" />
    <!--Bootstrap CSS using CDN-->
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" />
    <link rel="stylesheet" href="css/main.css">

    <title>Report</title>
</head>

<body class = 'container'>
    <header>
        <nav>
            <ul class="./index_NAV_BAR_files/nav_bar_for_website">
                <li><a href="index.html">Home</a></li>
                <li><a href="Sales_Table.html">Sales Summary Table</a></li>
                <li><a href="Sales_Breakdown_Column_Charts.html">Sales Breakdown</a></li>
                <li><a href="Trend_Analysis_Line_Charts.html">Sales Trend Lines</a></li>
                <li><a href="BFI_Scraps_Analysis.html">BFI Scraps Trend Analysis</a></li>
                <li><a href="Data_Downloads.html">Data Downloads</a></li>
            </ul>
        </nav>
    </header>
   <!----------------------------------------- SALES TREND LINE GRAPHS for web page --------------------------------------->
    <br>
    <h1 class = "page_title_smaller_for_website">SALES TREND GRAPHS</h1>
    <br>
    <br>
    <p>Note:  The graphs on this page are interactive.  A few common examples include:
                    <br>- Single-click an item in the legend to exclude that category from view.
                    <br>- Double-click an item in the legend to view ONLY that category.
                    <br>- As you hover over the graph, additional functions will appear above the graph (feel free to experiment)
                    <br>- To reset a graph, double-click the legend (or graph) or just refresh the webpage
    <div class = "fit_three_graphs_on_pdf_page_for_website">
        <iframe width="1050px" height="525px" frameborder="0" scrolling="no" src="{Service_Purchases_Trend_graph_for_iframe}"></iframe>
        <iframe width="1050px" height="525px" frameborder="0" scrolling="no" src="{Bandag_Purchases_Trend_graph_for_iframe}"></iframe>
        <iframe width="1050px" height="525px" frameborder="0" scrolling="no" src="{Bridgestone_Purchases_Trend_graph_for_iframe}"></iframe>
    </div>
    <br>
    <br>
    <br>
    <br>
</body>

</html>

"""

# html_page_exists = os.path.exists('Sales_Table.html')
# current_index = ''
# with open('Sales_Table.html', 'r') as f:
#     current_index = f.read()

with open('Trend_Analysis_Line_Charts.html', 'w') as new_file:
    new_file.write(Create_html_for_trend_lines)

# BUILD THE SCRAPS ANALYSIS Web Page:

In [ ]:
# BUILD THE HTML TABLE for the NRT TABLE:

def create_html_table(Nrt_table):
    
    # Create headers for HTML
    row_data = '<!-- Start NRT Table --><thead>\n<tr class="NRT_header_for_website">'

    # Create columns for HTML:    
    for column in range(Nrt_table.shape[1]):
        adjust_header_css = "NRT_header_for_website"
        if column not in [2]:
            adjust_header_css += " black_color_for_website make_bold_for_website make_text_white_for_website"
        if column in [2]:
            adjust_header_css += " lonestar_gold_color_for_website make_bold_for_website"
        if column not in []:
            adjust_header_css += " center_for_website"    
        row_data += '\n <th class = "'+adjust_header_css+'">'+Lonestar_NRT_df_sorted_for_html.columns[column]+'</th>'
    row_data += '\n</tr> \n </head> \n <tbody>'
    
    # Create Body (rows for table) for html
    for row in range(Nrt_table.shape[0]):
        if row in [0,1,2,3,4,5,6,7,8,9,10,11,12]:
            row_data += '\n<tr class = "make_bold_for_website">'
        else: row_data += '\n<tr>'
        for column in range(Nrt_table.shape[1]):
            td_class_name = 'NRT_text_column_for_website'
            if column not in []:
                td_class_name += " center_for_website"
            if row in [12]:
                td_class_name += " black_color_for_website make_bold_for_website make_text_white_for_website"
            # if column not in [2] and row in [13]:
            #     td_class_name += " blue_color make_text_white"
            if column in [2]:
                td_class_name += " lonestar_gold_color_for_website make_bold_for_website make_text_black_for_website"
            row_data += '\n <td class = "'+td_class_name+'">'+str(Nrt_table.iloc[row,column])+'</td>'
    row_data += '\n</tr> \n</tbody><!-- End NRT Table -->'
                # -------------------------------------------------------------------------------------------------

    return row_data

# Call the function:
html_table2 = create_html_table(Lonestar_NRT_df_sorted_for_html)

Create_html_for_the_scraps_analysis = f"""

<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <meta http-equiv="X-UA-Compatible" content="ie=edge" />
    <!--Bootstrap CSS using CDN-->
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" />
    <link rel="stylesheet" href="css/main.css">

    <title>Report</title>
</head>

<body class = 'container'>
    <header>
        <nav>
            <ul class="./index_NAV_BAR_files/nav_bar_for_website">
                <li><a href="index.html">Home</a></li>
                <li><a href="Sales_Table.html">Sales Summary Table</a></li>
                <li><a href="Sales_Breakdown_Column_Charts.html">Sales Breakdown</a></li>
                <li><a href="Trend_Analysis_Line_Charts.html">Sales Trend Lines</a></li>
                <li><a href="BFI_Scraps_Analysis.html">BFI Scraps Trend Analysis</a></li>
                <li><a href="Data_Downloads.html">Data Downloads</a></li>
            </ul>
        </nav>
    </header>
   <!----------------------------------------- SCRAPS TREND TABLE AND GRAPH for web page --------------------------------------->
    <br>
        <h1 class = "page_title_smaller_for_website">BFI SCRAP TREND ANALYSIS</h1>
    <br>
    <br>
    <p>Note:  The graphs on this page are interactive.  A few common examples include:
                    <br>- Single-click an item in the legend to exclude that category from view.
                    <br>- Double-click an item in the legend to view ONLY that category.
                    <br>- As you hover over the graph, additional functions will appear above the graph (feel free to experiment)
                    <br>- To reset a graph, double-click the legend (or graph) or just refresh the webpage
    <br>
    <br>
    <h2 class=scrap_summary_table_heading_for_website>SCRAP SUMMARY TABLE:<h2>
    <table>
        <!-- Start NRT Table -->
        [[DATA_MAIN_TABLE]]
        <!-- End NRT Table -->
    </table>
    <br>
    <br>
    <br>
    <iframe width="1100px" height="1020px" frameborder="0" scrolling="no" src="{Scrap_Trend_graph_for_iframe}"></iframe>
    <br>
    <br>
    <br>
</body>

</html>

"""

# 
with open('BFI_Scraps_Analysis.html', 'w') as new_file:
    new_file.write(Create_html_for_the_scraps_analysis)

index_exists = os.path.exists('BFI_Scraps_Analysis.html')
current_index = ''
with open('BFI_Scraps_Analysis.html', 'r') as f:
    current_index = f.read()
with open('BFI_Scraps_Analysis.html', 'w') as f:
    f.write(re.sub(r"<\!-- Start NRT Table -->.*<\!-- End NRT Table -->", html_table2, current_index, flags=re.DOTALL))


# CREATE A PAGE THAT WILL HOUSE A LINK TO DOWNLOADS:

In [ ]:
Create_html_for_the_data_downloads_page = f"""

<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <meta http-equiv="X-UA-Compatible" content="ie=edge" />
    <!--Bootstrap CSS using CDN-->
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" />
    <link rel="stylesheet" href="css/main.css">

    <title>Report</title>
</head>

<body class = 'container'>
    <header>
        <nav>
            <ul class="./index_NAV_BAR_files/nav_bar_for_website">
                <li><a href="index.html">Home</a></li>
                <li><a href="Sales_Table.html">Sales Summary Table</a></li>
                <li><a href="Sales_Breakdown_Column_Charts.html">Sales Breakdown</a></li>
                <li><a href="Trend_Analysis_Line_Charts.html">Sales Trend Lines</a></li>
                <li><a href="BFI_Scraps_Analysis.html">BFI Scraps Trend Analysis</a></li>
                <li><a href="Data_Downloads.html">Data Downloads</a></li>
            </ul>
        </nav>
    </header>
   <!----------------------------------------- Downloads data for web page --------------------------------------->
    <br>
        <h1 class = "page_title_smaller_for_website">DOWNLOADS</h1>
    <br>
    <br>
    <br>
    <form>
        <input class = "download_button" type="button" value="Download PDF Report" onClick="window.location.href='{Last_Month}_Lonestar_YTD_Purchase_and_Scrap_Report.pdf'">
    </form>
    <br>
    <form>
    <input class = "download_button" type="button" value="Download Excel Data" onClick="window.location.href='SAM_Steve_Walton_Customers_Purchases_Breakdown_10-22-2021_Starting_Template.xlsx'">
    </form>
    <br>
    <br>
    <br>
</body>

</html>

"""

# 
with open('Data_Downloads.html', 'w') as new_file:
    new_file.write(Create_html_for_the_data_downloads_page)


In [ ]:
Last_Month

## Delete the contents no longer needed from certain folders after successfully generating website and pdf:

In [ ]:
# # Delete contents from various folders that we don't want to keep now that we have the report done:

# folder = './CSVs'
# for filename in os.listdir(folder):
#     file_path = os.path.join(folder, filename)
#     try:
#         if os.path.isfile(file_path) or os.path.islink(file_path):
#             os.unlink(file_path)
#         elif os.path.isdir(file_path):
#             shutil.rmtree(file_path)
#     except Exception as e:
#         print('Failed to delete CSVs' % (file_path, e))

# # Delete contents form the "Images_graphs" folder:

# folder = './Images_graphs'
# for filename in os.listdir(folder):
#     file_path = os.path.join(folder, filename)
#     try:
#         if os.path.isfile(file_path) or os.path.islink(file_path):
#             os.unlink(file_path)
#         elif os.path.isdir(file_path):
#             shutil.rmtree(file_path)
#     except Exception as e:
#         print('Failed to delete CSVs' % (file_path, e))

# # Delete contents form the "NRT_CSVs_BFI_SCRAPE" folder:

# folder = './NRT_CSVs_BFI_SCRAPE'
# for filename in os.listdir(folder):
#     file_path = os.path.join(folder, filename)
#     try:
#         if os.path.isfile(file_path) or os.path.islink(file_path):
#             os.unlink(file_path)
#         elif os.path.isdir(file_path):
#             shutil.rmtree(file_path)
#     except Exception as e:
#         print('Failed to delete CSVs' % (file_path, e))